In [8]:
import re
import os
import sys

from pymongo import MongoClient
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [19]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 2000)

In [110]:
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DATABASE = os.getenv("MONGO_DATABASE")
client = MongoClient(MONGO_URI)
db = client[MONGO_DATABASE]

selected_keys = {"acordaoId": 1, "index": 1, "observacao": 1, "partesTexto": 1,
                 "citacoesObs": 1, "publicacao": 1, "citacoes_revistas": 1}
page_ranks_cursor = db["acordaos"].find({}, selected_keys)
df_before = pd.DataFrame(page_ranks_cursor)

db = client["DJs_v7"]
page_ranks_cursor = db["acordaos"].find({}, selected_keys)
df_after = pd.DataFrame(page_ranks_cursor)

In [4]:
def getMatchText(text, regexExp):
    match = re.search(regexExp, text)
    if match == None:
        return ""
    else:
        return (match.group(1)).strip()

def normalizeId(Id):
    acNo = getMatchText(Id, "[A-Z]+\s+(\d+)\s*.*")
    typeInv = Id.replace(acNo, " ").upper().strip()
    idList = typeInv.split()
    Id = " ".join(reversed(idList))
    return Id + " " + acNo

In [7]:
df_before["citacoesObs"] = df_before["citacoesObs"].apply(lambda ids: [normalizeId(id) for id in ids])

### Medidas descritivas da rede de acórdãos como era antes 

In [8]:
# Número de decisões
len(df_before)

76378

In [9]:
# Número de decisões com campo observação vazio (sem citações)
len(df_before[(df_before["observacao"] == "")])

43187

In [10]:
# Campo observação não vazio e sem citações
df_before[(df_before["observacao"] != "") & ~df_before["observacao"].str.contains("[Cc]it")][["acordaoId", "observacao"]]

# O motivo do resultado abaixo é que a extração dos dados foi feita usando padrões antigo que não eram mais válidos

,acordaoId,observacao
1,AGR AI 383828,"Votação: unânime. Resultado: desprovido. Número de páginas: (05). Análise:(ANA). Revisão:(RCO/AAF). Inclusão: 30/06/03, (MLR). Alteração: 02/07/03, (MLR)."
3,AGR AI 394182,"Votação: unânime. Resultado: desprovido. Número de páginas: (05). Análise:(JEN). Revisão:(VAS). Inclusão: 05/08/03, (SVF)."
5,AGR RE 351132,"Votação: unânime. Resultado: desprovido. Número de páginas: (05). Análise:(VAS). Revisão:(RCO). Inclusão: 26/11/03, (SVF). Alteração: 06/02/06, (MLR)."
6,AGR AI 394737,"Votação: unânime. Resultado: desprovido. -Obs: O AI-394737-AgR foi objeto de embargos, rejeitados em 09/09/2003. Número de páginas: (06). Análise:(VAS). Revisão:(RCO). Inclusão: 19/09/03, (MLR). Alteração: 13/05/04, (JVC)."
7,AGR AI 412808,"Votação: unânime. Resultado: desprovido. Número de páginas: (5). Análise:(JEN). Revisão:(VAS). Inclusão: 01/08/03, (SVF)."
...,...,...
76363,AGR RE 910617,ARE 917126 AgR PROCESSO ELETRÔNICO JULG-24-11-2015 UF-MA TURMA-02 MIN-DIAS TOFFOLI N.PÁG-009 DJe-254 DIVULG 16-12-2015 PUBLIC 17-12-2015 ARE 910870 AgR PROCESSO ELETRÔNICO JULG-17-11-2015 UF-RS TURMA-02 MIN-DIAS TOFFOLI N.PÁG-008 DJe-252 DIVULG 15-12-2015 PUBLIC 16-12-2015 ARE 864668 AgR ACÓRDÃO ELETRÔNICO JULG-27-10-2015 UF-PE TURMA-02 MIN-DIAS TOFFOLI N.PÁG-007 DJe-250 DIVULG 11-12-2015 PUBLIC 14-12-2015
76364,AGR ARE 744404,RE 898967 AgR PROCESSO ELETRÔNICO JULG-01-03-2016 UF-PE TURMA-02 MIN-DIAS TOFFOLI N.PÁG-011 DJe-051 DIVULG 17-03-2016 PUBLIC 18-03-2016 RE 906625 AgR PROCESSO ELETRÔNICO JULG-01-03-2016 UF-PB TURMA-02 MIN-DIAS TOFFOLI N.PÁG-012 DJe-051 DIVULG 17-03-2016 PUBLIC 18-03-2016
76366,AGR ARE 881868,ARE 918548 AgR PROCESSO ELETRÔNICO JULG-24-11-2015 UF-PR TURMA-02 MIN-DIAS TOFFOLI N.PÁG-018 DJe-018 DIVULG 29-01-2016 PUBLIC 01-02-2016
76373,AGR ARE 907036,ARE 907036 AgR-ED PROCESSO ELETRÔNICO JULG-15-12-2015 UF-PR TURMA-01 MIN-ROBERTO BARROSO N.PÁG-006 DJe-025 DIVULG 11-02-2016 PUBLIC 12-02-2016


In [11]:
# Decisões que possuem citações
df_before[(df_before["citacoesObs"].str.len() > 0)][["acordaoId", "citacoesObs"]]

,acordaoId,citacoesObs
0,MS 24305,"[ADI 1303, MS 21269, MS 21517, RE 235487]"
2,HC 82445,"[HC 69141, HC 76555]"
4,AGR RE 234954,"[RE 223796, RE 227106]"
8,AGR RE 223510,[RE 199366]
10,ED ED AGR RE 167522,"[MS 1957, MS 2924]"
...,...,...
72496,RE 598572,"[RE 138284, RE 177296, RE 351717, RE 377457, RE 583712, RE 406955, RE 627543, RE 370590, RE 335275, RE 490576, AR 2322, AC 1109]"
72497,RE 841526,"[RE 161422, RE 215981, RE 481110, AI 577908, AI 299125, RE 607771, AI 852237, RE 677283, ARE 754778, RE 215981, RE 592581, AI 603865, AI 799789, RE 594902, ARE 662563, ADPF 347, RE 272839, RE 580252, RE 121130, RE 272839, HC 84072, ARE 700927]"
72574,RE 655265,"[MS 25504, ADI 3460, MS 27609, MS 27608, ADC 12, RE 184425, RE 392976, MS 26668, AI 733252, AI 839058, AI 418727, RE 184425]"
73007,RE 641320,"[HC 82959, HC 75299, HC 93596, HC 94526, HC 94810, HC 94829, HC 110892, HC 72643, RE 592581, ADI 5240, ADPF 347, ADI 1105, ADI 1127, ADI 1946, ADI 2209, ADI 2596, ADI 2652, ADI 3046, ADI 3324, ADI 2405, ADI 1344, RP 1417, PET 3388, ADI 4425, ADI 4357, ADPF 347, HC 76930, HC 87985, HC 93596, RHC 65127, HC 67072, HC 66593, HC 74732, HC 68310, MI 283, ADI 1105, ADI 1127, RMS 22307, MI 543, MI 670, RE 405579, ADPF 54, MI 708, MI 712, PET 3388, MS 26603, MS 26602, MS 26604, ADI 2332, ADPF 54, ADI 2084, ADI 1797, ADI 1668, ADI 1344, ADI 2332]"


In [12]:
# Decisões que deveriam ter citações, mas por algum motivo nenhuma foi detectada
df_before[df_before["observacao"].str.contains("[Cc]it") &
          (df_before["citacoesObs"].str.len() == 0)][["acordaoId", "observacao"]]

# Alguns casos aqui possuem espaço para melhora. Outros sofreram com extração incorreta
# decorrente do uso de padrões antigos de extração extração das decisões

,acordaoId,observacao
55,AGR RE 354935,"Votação: unânime. Resultado: desprovido. Acórdaõs citados: RE-195894, AI-250852-AgR. Número de páginas: (05). Análise:(DMV). Revisão:(RCO). Inclusão: 22/09/03, (MLR). Alteração: 25/09/03, (MLR)."
97,HC 82142,"Votação: unânime. Resultado: concessão parcial da ordem, para assentar a incompetência da justiça comum quanto aos crimes militares, excetuado o de formação de quadrilha e estendendo a ordem ao demais co-réus militares. Acórdãos citados: RHC-56046 (RTJ-87/47), RHC-56275 (RTJ-87/460), RHC-59444, RHC-61367 (RTJ-109/105), HC-68928 (RTJ-138/569), HC-69571 (RTJ-144/283), RE-121124 (RTJ-132/917; STJ - CC-2686, CC-3532 Número de páginas: (18). Análise:(MML). Revisão:(RCO). Inclusão: 30/11/03, (MLR)."
117,AO 609,"Votação: unânime. Resultado: provido o recurso dos autores e parcialmente provido o recurso do Estado do Rio Grande do Sul. Acórdãos citados: AORQO-263; AO-303; AORQO-331; AO-517; (RTJ-172/738); RE-145018; (RTJ-149/928); Número de páginas: (13). Análise:(CMM). Revisão:(AAF). Inclusão: 25/04/01, (MLR). Alteração: 12/07/04, (NT)."
128,HC 80587,"Votação: Unânime. Resultado: Indeferido. Acórdãos citados do STJ : HC-6779, HC-8551, RHC-4569, REsp-130611. Veja: RE-77713, RE-18082. Número de páginas: (22). Análise:(CRP). Revisão:(RCO/AAF). Inclusão: 24/09/01, (SVF)."
167,RE 285047,"Votação: Por maioria, vencido o Min. Marco Aurélio. Resultado: Não conhecido. Acórdãos citados: HC-72131 (RTJ-186/980), HC-75306; STJ: REsp-2320 (RSTJ-23/378). Número de páginas: (16). Análise:(PCC). Revisão:(). Inclusão: 03/02/05, (PCC)."
...,...,...
71096,MS 33406,"AMARAL, Héctor A. La Doctrina de Los Propios Actos de La Administración Pública. Depalma, 1988. p. 133-138. ARAGÃO, Alexandre Santos de. Curso de Direito Administrativo. Forense, 2012. p. 65-69, item VI.6. BERTONCINI, Mateus Eduardo Siqueira Nunes. Princípios de Direito Administrativo Brasileiro. Malheiros, 2002. p. 178-180, item 4.5.7. BINENBOJM, Gustavo. Temas de Direito Administrativo e Constitucional. Renovar, 2008. p. 735-740, itens II.2.2 a II.2.2.2. BORDA, Alejandro. La Teoria de Los Actos Propios. 2. ed. Abeledo Perrot, 1992. p. 136-138. CARVALHO, Raquel Melo Urbano de. Curso de Direito Administrativo. JusPodivm, 2008. p. 78-94, itens 8 a 8.4. CANOTILHO, José Joaquim Gomes. Direito Constitucional e Teoria da Constituição. Almedina, 1998. p. 250. COSTA, Judith Martins. A Ilicitude Derivada do Exercício do Comportamento Contraditório de Um Direito: o Renascer do Venire Contra Factum Proprium. Revista Forense, 2004. v. 376. p. 109-129. FACCI, Lucio Picanço. A Proibição do Comportamento Contraditório no Âmbito da Administração Pública: A Tutela da Confiança nas Relações Jurídico-Administrativas. Revista da EMERJ, 2011. v. 14, n. 53, p. 197-229. FERRAZ, Sérgio. O princípio da segurança jurídica em face das reformas constitucionais. Revista Forense. v. 334. p. 191-210. FIGUEIREDO, Lúcia Valle. Curso de Direito Administrativo. 9. ed. Malheiros, 2008. p. 257-260, itens ns. 3.2 a 4. JUSTEN FILHO, Marçal. Curso de Direito Administrativo. 4. ed. Saraiva, 2009. p. 1097-1100, itens XVII.1 a XVII.3.1. MEIRELLES, Hely Lopes. Direito Administrativo Brasileiro. 34. ed. Atualização: Eurico de Andrade Azevedo, Délcio Balestero Aleixo e José Emmanuel Burle Filho. Malheiros, 2008. p. 99-101, item 2.3.7. MELLO, Celso Antônio Bandeira de. Curso de Direito Administrativo. 26. ed. Malheiros, 2009. p. 87, item 77 e 123-125, item 27. PIETRO, Maria Sylvia Zanella di. Direito Administrativo. 22. ed. Atlas, 2009. p. 87-88, item 3.3.15.4. SCHREIBER, Anderson. A Proibição de Comportame..."
71552,SEGUNDO AGR RCL 20132,"BADARÓ, Gustavo. Processo penal. Rio de Janeiro: Campus Elsevier, 2012. p. 89. MARQUES, José Frederico. Elementos de Direito Processual Penal. Campinas: Bookseller, 1997. v. 1. p. 128. NOGUEIRA, Carlos Frederico Coelho apud TOURINHO FILHO, Fernando da Costa. Comentários ao Código de Processo Penal. São Paulo: Edipro, 2002. v. 1 p. 409. (op. cit., p. 424). TOU

In [13]:
# Decisões que deveriam ter e têm citações
df_before[(df_before["citacoesObs"].str.len() > 0) &
          df_before["observacao"].str.contains("[Cc]it")][["observacao", "citacoesObs"]]

,observacao,citacoesObs
0,"Votação: Unânime. Resultado: Concedida a segurança, nos termos do voto do Relator. Acórdãos citados: ADI-1303-MC (RTJ-174/743), MS-21269 (RTJ-148/393), MS-21517 (RTJ-150/751), RE-235487 (RTJ-181/1141). Número de páginas: (29). Análise:(JBM). Inclusão: 14/06/2004, (SVF). Alteração: 13/02/06, (MLR).","[ADI 1303, MS 21269, MS 21517, RE 235487]"
2,"Votação: unânime. Resultado: indeferido. Acórdãos citados: HC-69141 (RTJ-142/865), HC-76555. Número de páginas: (04). Análise:(VAS). Revisão:(RCO). Inclusão: 12/12/03, (MLR).","[HC 69141, HC 76555]"
4,"Votação: unânime. Resultado: desprovido. Acórdãos citados: RE-223796, RE-227106. - Obs: O RE-234954-AgR foi objeto dos RE-AgR-ED rejeitados em 03/06/2003. Número de páginas: (07). Análise:(VAS). Revisão:(RCO). Inclusão: 08/10/03, (MLR). Alteração, 06/05/04, (JVC).","[RE 223796, RE 227106]"
8,"Votação: unânime. Resultado: desprovido. Acórdão citado: RE-199366. Número de páginas: (6). Análise:(VAS). Revisão:(RCO/AAF). Inclusão: 22/05/03, (SVF). Alteração: 01/09/03, (SVF).",[RE 199366]
10,"Votação: unânime. Resultado: rejeitados. Acórdãos citados:MS-1957, MS-2924. Número de páginas: (06). Análise:(VAS). Revisão:(RCO/AAF). Inclusão: 17/03/03, (MLR). Alteração: 10/05/04, (MLR).","[MS 1957, MS 2924]"
...,...,...
72496,"- Acórdão(s) citado(s): (CONTRIBUIÇÃO SOCIAL, LEI COMPLEMENTAR) RE 138284 (TP), RE 177296 (TP), RE 351717 (TP), RE 377457 (TP), RE 583712 (TP). (PRINCÍPIO DA CAPACIDADE CONTRIBUTIVA) RE 406955, RE 627543 (TP). (PODER JUDICIÁRIO, SUPRESSÃO, EQUIPARAÇÃO, ALÍQUOTA DIFERENCIADA) RE 370590 AgR (2ªT), RE 335275 AgR-segundo (1ªT), RE 490576 AgR (2ªT), AR 2322 AgR (TP). (INSTITUIÇÃO FINANCEIRA, CONTRIBUIÇÃO PREVIDENCIÁRIA PATRONAL, PRINCÍPIO DA IGUALDADE TRIBUTÁRIA) AC 1109. - Veja RE 599309 RG do STF. Número de páginas: 43. Análise: 23/08/2016, KBP.","[RE 138284, RE 177296, RE 351717, RE 377457, RE 583712, RE 406955, RE 627543, RE 370590, RE 335275, RE 490576, AR 2322, AC 1109]"
72497,". - Acórdão(s) citado(s): (RESPONSABILIDADE CIVIL DO ESTADO ) RE 607771 AgR (2ªT), AI 852237 AgR (2ªT), RE 677283 AgR (2ªT), ARE 754778 AgR (1ªT). (RESPONSABILIDADE CIVIL DO ESTADO, SISTEMA CARCERÁRIO, MORTE, PRESO) RE 215981 (2ªT), RE 592581 (TP), AI 603865 AgR (1ªT), AI 799789 AgR (1ªT), RE 594902 AgR (1ªT), ARE 662563 AgR (2ªT), ADPF 347 MC (TP). (RESPONSABILIDADE CIVIL DOS ESTADO, DANO MORAL, SISTEMA CARCERÁRIO) RE 272839 (2ªT), RE 580252 RG. (PRESO, SUÍCIDIO, INEXISTÊNCIA, OBRIGAÇÃO, AÇÃO DE INDENIZAÇÃO, RESPONSABILIDADE CIVIL DO ESTADO) RE 121130 (2ªT). (RESPONSABILIDADE OBJETIVA, AFERIÇÃO, REPARAÇÃO DE DANO, DOLO, CULPA, AGENTE) RE 272839 (2ªT), HC 84072 (1T). (RESPONSABILIDADE CIVIL DO ESTADO,SISTEMA CARCERÁRIO, SUICÍDIO, PRESO) ARE 700927 AgR (2ªT). - Decisões monocráticas citadas: (RESPONSABILIDADE CIVIL DO ESTADO,SISTEMA CARCERÁRIO, SUICÍDIO, PRESO) RE 161422. (RESPONSABILIDADE CIVIL DOS ESTADO, DANO MORAL, SISTEMA CARCERÁRIO) RE 215981, RE 481110, AI 577908. (RESPONSABILIDADE CIVIL DO ESTADO, ÔNUS DA PROVA) AI 299125. - Acórdão(s) citado(s) - outros tribunais: STJ: REsp 1374284. Número de páginas: 101. Análise: 18/08/2016, JRS.","[RE 161422, RE 215981, RE 481110, AI 577908, AI 299125, RE 607771, AI 852237, RE 677283, ARE 754778, RE 215981, RE 592581, AI 603865, AI 799789, RE 594902, ARE 662563, ADPF 347, RE 272839, RE 580252, RE 121130, RE 272839, HC 84072, ARE 700927]"
72574,"- Acórdão(s) citado(s): (MOMENTO, COMPROVAÇÃO, TEMPO DE ATIVIDADE JURÍDICA) ADI 3460 (TP), MS 27609 (TP), MS 27608 (TP). (PODER NORMATIVO, CNJ) ADC 12 (TP). (COMPROVAÇÃO, HABILITAÇÃO LEGAL, INSCRIÇÃO, CONCURSO PÚBLICO) RE 184425 (2ªT), RE 392976 (1ªT), MS 26668 (TP), AI 733252 AgR (2ªT), AI 839058 AgR (2ªT), AI 418727 ED (1ªT). (MOMENTO, CUMPRIMENTO, REQUISITO, CARGO PÚBLICO) RE 184425 (2ªT). - Decisão monocrática citada: (MOMENTO, COMPROVAÇÃO, TEMPO DE ATIVIDADE JURÍDICA) MS 25504 MC. Número de páginas: 88. Análise: 18/08/2016, IMC.","[MS 25504, ADI 3460, MS 27609, MS 27608, ADC 12, RE 184425,

In [14]:
# Soma do número de citações de todas as decisões
sum(df_before["citacoesObs"].apply(lambda x: len(x)))

19171

### Medidas descritivas da rede de acórdãos como é agora 

In [15]:
# Número de decisões
len(df_after)

103168

In [16]:
# Número de decisões com campo observação vazio (sem citações)
len(df_after[(df_after["observacao"] == "")])

4444

In [17]:
df_after[(df_after["observacao"] == "")]["acordaoId"]

4555              AGR RE 230739
6360                   HC 82420
7091               AGR RCL 2182
7676                QO PET 2935
11548                  HC 84802
                  ...          
103163    ED ED AGR ARE 1208076
103164    ED AGR ED ARE 1217415
103165            AGR HC 173987
103166       ED AGR ARE 1220362
103167           AGR RHC 176636
Name: acordaoId, Length: 4444, dtype: object

In [18]:
# Campo observação não vazio e sem citações
def remove_useless_observation_segment(txt):
    txt = re.sub(
        r"((Número\s*de\s*p[aá]ginas|Altera[aç][aã]o|Revis[aã]o|Inclus[aã]o|An[aá]lise|Obs\.|Vota[cç][aã]o|Resultado):)[^\.]*\.", "", txt, flags=re.IGNORECASE
    )
    # Além disso, uma decisão específica começa com a string “Veja” Será feita uma exceção para este caso.    
    veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
    while re.match(veja_pattern, txt, flags=re.IGNORECASE):
        txt = re.sub(
            veja_pattern, r"\2", txt, flags=re.IGNORECASE
        )
    return txt.strip(" -")

df_obs_nao_vazio = df_after[(df_after["observacao"].apply(lambda x: remove_useless_observation_segment(x)) != "") &
         ~df_after["observacao"].str.contains("[Cc]it")]

In [17]:
df_obs_nao_vazio.to_csv("observacao_sem_citacoes.csv", sep="@")
df_obs_nao_vazio[["acordaoId", "observacao", "partesTexto"]]
## no banco V5 tinham 1375 decisões assim
## no banco V6 tinham 949 decisões assim

,acordaoId,observacao,partesTexto
0,AGR RE 308114,"O RE 308114 AgR foi objeto de embargos de declaração rejeitados em 17/09/2002. Número de páginas: (06). Análise:(CRP). Revisão:(CTM). Inclusão: 01/07/02, (SVF). Alteração: 01/08/05, (MLR). Alteração: 25/05/2018, JRM.",AGTE. : GUIDO - COMÉRCIO E TRANSPORTE DE MATERIAIS PARA CONSTRUÇÕES LTDA \nADVDOS. : CLAUDIO ZANKOSKI E OUTROS \nAGDO. : ESTADO DO PARANÁ \nADVDA. : PGE-PR - MÁRCIA DIEGUEZ LEUZINGER E OUTRO
21,AGR AI 336079,"O AI 336079 AgR foi objeto dos embargos de declaração rejeitados em 28/09/2004. Número de páginas: (08). Análise:(ANA). Revisão:(VAS/RCO). Inclusão: 11/09/03, (MLR). Alteração: 02/12/04, (SVF). Alteração: 25/09/2018, JLS.",AGTE. : MUNICÍPIO DE MANAUS \nADV. : ANTÔNIO DIONYSIO CARVALHO PAIXÃO \nAGDA. : CONSTRUTORA ANDRADE GUTIERREZ S/A \nADVDOS. : JOSÉ MAURÍCIO BALBI SOLLERO E OUTROS
28,AGR RE 345871,"O AI 345871 AgR foi objeto de embargos de declaração rejeitados em 12/04/2005. O RE 345871 AgR foi objeto de embargos de declaração rejeitados em 12/04/2005. Número de páginas: (07). Análise:(VAS). Revisão:(FLO/RCO). Inclusão: 12/12/03, (SVF). Alteração: 01/08/05, (MLR). Alteração: 12/09/2018, ALS.",AGTE. : DISTRITO FEDERAL \nADV. : PGDF - RENATO GUANABARA LEAL DE ARAÚJO \nAGDOS. : SHIRLEY FERREIRA DE OLIVEIRA E OUTROS \nADVDOS. : MARCOS LUIS BORGES DE RESENDE E OUTROS
109,AGR AI 406993,"AI 406993 AgR foi objeto de embargos de declaração rejeitados em 27/09/2005. Número de páginas: (06). Análise:(DMV). Revisão:(FLO). Inclusão: 20/08/03, (MLR). Alteração: 10/03/06, (NAL). Alteração: 20/03/2019, HAC.",AGTES. : JOÃO CUCHARO E OUTROS \nADVDOS. : AGUINALDO DE BASTOS E OUTROS \nAGDO. : MUNICÍPIO DE JUNDIAI \nADVD. : ROLFF MILANI DE CARVALHO
115,ED RE 337680,"- O AI 429950 foi objeto de embargos de declaração rejeitados em 09/12/2003. Número de páginas: (5). Análise:(ANA). Revisão:(RCO/AAF). Inclusão: 23/06/03, (SVF). Alteração: 21/11/05, (MLR). Alteração: 29/08/2018, HAC.",EMBTE. : USINA SANTA RITA S/A AÇÚCAR E ÁLCOOL \nADV. : HAMILTON DIAS DE SOUZA \nEMBDA. : UNIÃO \nADVDA. : PFN - ELYADIR FERREIRA BORGES
...,...,...,...
77991,HC 129734,"- Caso ""OPERAÇÃO HERODES"". Número de páginas: 12. Análise: 22/01/2018, JRS. Revisão: 22/02/2018, KBP.",PACTE.(S) : FABIO FELIX DOS REIS \nIMPTE.(S) : RALPH HAGE NICOLAU RITTER VIANNA \nCOATOR(A/S)(ES) : RELATOR DO RHC Nº 61.038 DO SUPERIOR TRIBUNAL DE JUSTIÇA
79058,AGR ARE 980355,"- O ARE 980355 AgR foi objeto de embargos de declaração acolhidos com efeitos infringentes. Número de páginas: 10. Análise: 13/10/2017, MJC.",AGTE.(S) : PHENIX SEGURADORA S/A \nADV.(A/S) : PAULA CASSETTARI FLORES \nADV.(A/S) : THIAGO ZUCCHETTI CARRION \nAGDO.(A/S) : LEILA REGINA TECHIO \nADV.(A/S) : JORENILDA LUCIA STUKER SCHMIDT
82708,AGR RE 1094802,"- O ARE 1107360 AgR foi objeto de embargos de declaração acolhidos com efeitos infringentes. Número de páginas: 8. Análise: 01/06/2018, BMP.",AGTE.(S) : EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES - EBSERH \nADV.(A/S) : KARLA SOARES DE ARAUJO AMORIM \nADV.(A/S) : JOSEAM CATANHEDE DE OLIVEIRA \nADV.(A/S) : ANTONIO SABINO DA SILVA FILHO \nAGDO.(A/S) : ANA CLAUDIA DA SILVA MOURA \nADV.(A/S) : ANA AUGUSTA LIMA SOARES BARBOSA
84818,AGR ARE 1110171,"- O ARE 1110171 AgR foi objeto de embargos de declaração acolhidos com efeitos modificativos. Número de páginas: 3. Análise: 24/09/2018, ER.",AGTE.(S) : B.J.S.R. \nADV.(A/S) : EDUARDO XIBLE SALLES RAMOS \nADV.(A/S) : LETICIA SOBRINHO DE AGUIAR \nADV.(A/S) : RODRIGO MELO MESQUITA \nAGDO.(A/S) : UNIÃO \nPROC.(A/S)(ES) : PROCURADOR-GERAL DA FAZENDA NACIONAL


In [19]:
# Aprimoramento de detecção de corpo de texto sobre publicação ou divulgação
df_public_pattern = df_after[df_after["publicacao"].apply(lambda x: " PUBLIC " not in x) & df_after["publicacao"].apply(lambda x: "PUBLIC" in x)]
# df_public_pattern = df_after[df_after["publicacao"].apply(lambda x: " DIVULG " not in x) & df_after["publicacao"].apply(lambda x: "DIVULG" in x)]

df_public_pattern[["publicacao"]]

,publicacao
16,DJ 14-05-2001 PP-00189 EMENT VOL-02030-02 PP-00267\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00065
190,DJ 14-05-2001 PP-00189 EMENT VOL-02030-03 PP-00486\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00064
225,DJ 14-05-2001 PP-00189 EMENT VOL-02030-04 PP-00860\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00087
304,DJ 14-05-2001 PP-00189 EMENT VOL-02030-04 PP-00884\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00066
316,DJ 14-05-2001 PP-00189 EMENT VOL-02030-05 PP-01010\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00088
...,...
22856,DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00024\r\nEMENT VOL-02290-01 PP-00208
23085,"DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00043 EMENT VOL-02290-02 PP-00262\r\nRTJ VOL-00203-03 PP-01164\r\nRMDPPP v. 4, n. 20, 2007, p. 107-114"
23102,"DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00020 EMENT VOL-02290-01 PP-00019\r\nRT v. 97, n. 867, 2008, p. 93-97"
23182,DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00025\r\nEMENT VOL-02290-02 PP-00379


In [20]:
df_public_pattern[df_public_pattern["publicacao"].apply(lambda x: True if re.search("\sDIVULG\-?\s?|\sREPUBLICAÇÃO\s*|\sPUBLIC\-?\s+", x) else None)]

# Este problema foi resolvido e a função de detecção de publicação em revistas já foi escrita

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index
16,5e327362d0a3cf19aa5ba25b,MS 23193,DJ 14-05-2001 PP-00189 EMENT VOL-02030-02 PP-00267\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00065,[],IMPTE. : JOSÉ TEIXEIRA \nADVDOS. : LÉO EDUARDO RIBEIRO PRADO E OUTRO \nIMPDO. : PRESIDENTE DA REPÚBLICA,"Acórdãos citados : MS 20430, MS 20714, MD 20882, RMS 21300, (RTJ 142/782), RMS 21597, MS 22022, (RTJ 168/163), MS 22164, (RTJ 164/158), MS 22225, MS 22319, (RTJ 162/918), MS 22519, MS 23311, (RTJ 172/501, RTJ 124/948). Número de páginas: (63). Análise: (CMM). Revisão: (AAF). Inclusão: 19/06/01, (SVF). Alteração: 13/02/06, (MLR). Alteração: 17/01/2018, GIB.","[MS 20430, MS 20714, MD 20882, RMS 21300, RMS 21597, MS 22022, MS 22164, MS 22225, MS 22319, MS 22519, MS 23311]",164
190,5e3275cbd0a3cf19aa5ba309,HC 80566,DJ 14-05-2001 PP-00189 EMENT VOL-02030-03 PP-00486\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00064,[],PACTE. : LUIZ FELIPE GABRIEL GOMES \nIMPTE. : ADILSON RAMOS JÚNIOR \nCOATOR : SUPERIOR TRIBUNAL DE JUSTIÇA,"Veja: HC 42967. O HC 80566 foi objeto dos EDHC, rejeitados. Número de páginas: (19). Análise: (CTM). Revisão: (RCO/AAF). Inclusão: 28/08/01, (MLR). Alteração: 08/11/01, (MLR). Alteração: 16/01/2018, GIB.",[],240
225,5e3275f3d0a3cf19aa5ba32c,RE 232331,DJ 14-05-2001 PP-00189 EMENT VOL-02030-04 PP-00860\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00087,[],RECTE. : ESTADO DO CEARÁ \nADV. : PGE-CE - GERARDO RODRIGUES DE ALBUQUERQUE FILHO \nRECDOS. : RAIMUNDO EDUARDO DE OLIVEIRA E OUTROS \nADV. : EVANDRO FERREIRA MONTE,"Veja : RE 231234, RE 255408. Número de páginas: (06). Análise: (CRP). Revisão: (RCO/AAF). Inclusão: 04/06/01, (SVF). Alteração: 09/08/01, (SVF). Alteração: 17/01/2018, GIB.",[],193
304,5e32769cd0a3cf19aa5ba37b,AGR AI 238279,DJ 14-05-2001 PP-00189 EMENT VOL-02030-04 PP-00884\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00066,[],AGTES. : ANTÔNIO FRANCISCO BERNARDES DE ASSIS E OUTROS \nADVDOS. : MARCO ANTÔNIO BILIBIO CARVALHO E OUTROS \nAGDO. : BANCO CENTRAL DO BRASIL \nADVDOS. : DANIELA ZARZAR PEREIRA DE MELO E OUTRO,"Acórdão citado: RE 234388. Número de páginas: (06). Análise: (MML). Revisão: (RCO/AAF). Inclusão: 03/08/01, (SVF). Alteração: 16/12/01, (SVF). Alteração: 12/01/2018, GIB.",[RE 234388],340
316,5e3276a7d0a3cf19aa5ba387,RE 255639,DJ 14-05-2001 PP-00189 EMENT VOL-02030-05 PP-01010\r\nREPUBLICAÇÃO: DJ 18-05-2001 PP-00088,[],RECTE. : AROLDO GUILHERME SCHATTSCHNEIDER \nADVDOS. : SÍLVIA DOMINGUES SANTOS MANSUR E OUTROS \nRECDO. : MINISTÉRIO PÚBLICO FEDERAL,"Acórdão citado: HC 67759. Número de páginas: (09). Análise: (FCB). Revisão: (COF/AAF). Inclusão: 21/09/01, (MLR). Alteração: 17/01/2018, GIB.",[HC 67759],326
...,...,...,...,...,...,...,...,...
22856,5e332e56d0a3cf34f6749c1b,AGR RCL 2663,DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00024\r\nEMENT VOL-02290-01 PP-00208,[],AGTE.(S) : UNIÃO \nADV.(A/S) : ADVOGADO-GERAL DA UNIÃO \nAGDO.(A/S) : RELATOR DO AI Nº200304010361080 DO TRIBUNAL \n REGIONAL FEDERAL DA 4ª REGIÃO \nINTDO.(A/S) : LUCIANE SANTOS LIMA \nADV.(A/S) : WASHINGTON DIAS DA SILVA,"-Acórdãos citados: ADC 4, Rcl 1459, Rcl 1578, Rcl 2421 AgR, Rcl 2482 (RTJ 195/435), Rcl 2497, Rcl 4383. - Veja REsp 697752 do STJ. Número de páginas: 13 Análise: 08/10/2007, ACL.","[ADC 4, RCL 1459, RCL 1578, AGR RCL 2421, RCL 2482, RCL 2497, RCL 4383, STJ DO RESP VEJA 697752]",2353
23085,5e332fa7d0a3cf34f6749d00,HC 88978,"DJe-106 DIVULG-20-09-2007 PUBLIC-21-09-2007\r\nDJ 21-09-2007 PP-00043 EMENT VOL-02290-02 PP-00262\r\nRTJ VOL-00203-03 PP-01164\r\nRMDPPP v. 4, n. 20, 2007, p. 107-114","[RMDPPP 20/107, RTJ 203/1164]",PACTE.(S) : JAIME PEREIRA SARDINHA \nIMPTE.(S) : GILSOMAR SILVA BARBALHO \nCOATOR(A/S)(ES) : SUPERIOR TRIBUNAL DE JUSTIÇA,"-Acórdãos citados: HC 45553, HC 69958, HC 72992 (RTJ 168/863), HC 81260 (RTJ 181/680), HC 84768 (RTJ 194/298), RHC 88144, HC 89433. Número de páginas: 12 Análise: 04/10/2007, ACL. Revisão: 31/10/2007, RCO.","[HC 45553, HC 69958, HC 72992, HC 81260, HC 84768, RHC 88144, HC

## [A fazer] Citações que não são capturadas e precisam ser checadas ainda após nova extração

In [21]:
# Encontrar casos em que deveria haver decisões citadas, mas não há por alguma razão
df_after[df_after["observacao"].str.contains("órdão") & ~df_after["observacao"].str.contains("ecis") &
         ~df_after["observacao"].str.contains("[Tt]ribunais") &
         (df_after["citacoesObs"].apply(lambda x: len(x)) == 0)][["acordaoId", "observacao"]]

,acordaoId,observacao
69,AGR RE 222874,"Acórdão citado: 201462. Número de páginas: (05). Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 22/08/03, (MLR). Alteração: 04/04/2019, HAC."
317,HC 80587,"Acórdãos citados do STJ : HC 6779, HC 8551, RHC 4569, REsp 130611. Veja: RE 77713, RE 18082. Número de páginas: (22). Análise:(CRP). Revisão:(RCO/AAF). Inclusão: 24/09/01, (SVF). Alteração: 30/01/2018, CLS."
368,MC ADI 2254,"Acórdão citado: (RTJ 100/1003). Número de páginas: (23). Análise:(DMV). Revisão:(FLO). Inclusão: 02/04/04, (SVF). Alteração: 14/04/04, (NT). Alteração: 13/11/2018, CLS."
397,AGR RE 285167,"Acórdãos citados: RE230156. Número de páginas: (06). Análise:(MML). Revisão:(RCO/AAF). Inclusão: 29/05/01, (SVF). Alteração: 16/01/2018, ALS."
685,AGR AI 304365,"Acórdãos citados : RTJ 107/661; RTJ 120/912; RTJ 125/705. Número de páginas: (05). Análise: (MML). Revisão: (COF). Inclusão: 31/07/01, (SVF). Alteração: 29/01/2018, GIB."
...,...,...
86276,AGR ARE 1157575,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 6. Análise: 22/01/2019, MJC."
86304,AGR HC 141996,"- Acórdão(s) citado(s): (HC, AUSÊNCIA, PERTINÊNCIA, LOCOMOÇÃO FÍSICA, DESCABIMENTO) RTJ 116./523, RTJ 141/159, RTJ 135/593, RTJ 142/896, RTJ 197/587. Número de páginas: 10. Análise: 13/12/2018, BMP."
86375,AGR ARE 1154088,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. (RE, REEXAME DE FATO) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 8. Análise: 23/01/2019, MJC."
86672,AGR ARE 1146806,"- Acórdão(s) citado(s): (RE, REEXAME DE FATO E PROVA) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 6. Análise: 23/01/2019, MJC."


In [22]:
# def denormalize_id(txt):
#     parts = [s for s in re.split("(\w+\s+)(\w+\s+\d+)", txt) if s.strip() != '']
#     start = parts[-1].strip()
#     rest = " ".join([s.strip() for s in parts[:-1]])
    
#     return start + " " + rest


# df_after["acordaoId_"] = df_after.index.map(lambda x: denormalize_id(x))
# Olhar "HC 86878 AGR"

def remove_uninteresting_citations(txt):
    txt = re.sub(
        r"(Ac[oó]rd[aã]o\(?s?\)?\s*(citado\(?s?\)?)?\s*(de)?\s*\-?\s*outros?\s*tribuna(is|l)|Ac[oó]rd[aã]o\(?s?\)?\s*(citado\(?s?\)?)?\s*(do)?\s*:?\s*(STJ|TST|TSE)|Decis([aã]o|[oõ]es)\s*monocr[aá]ticas?\s*citad[ao]s?|Legisla[cç][aã]o\s*estrangeira\s*citada).*", "", txt, flags=re.IGNORECASE
    )
    return remove_useless_observation_segment(txt)

# Encontrar casos em que deveria haver decisões citadas, mas não há por alguma razão
df_citacoes_com_problemas = df_after[(df_after["observacao"].str.contains("[Cc][Ii][Tt]") | df_after["observacao"].str.contains("órdão")) &
         (df_after["observacao"].apply(lambda x: remove_uninteresting_citations(x) != "")) &
         (df_after["citacoesObs"].apply(lambda x: len(x)) == 0)]
# df_citacoes_com_problemas.to_csv("observacao_sem_citacoes.csv", sep="@")
df_citacoes_com_problemas[["acordaoId", "observacao"]]

# Enviar planilha ao André com algumas sugestões de correções
# mandar também problemas de casamento de parênteses
# df_after[df_after["observacao"].apply(lambda dec: True if re.search(r"\([^\(\)]+\(", remove_uninteresting_citations(dec)) is not None else False) &
#          ~df_after["observacao"].apply(lambda dec: True if re.search(r"\([^\(\)]*\([^\(\)]+\)[^\(\)]+\)|\([^\(\)]+\([^\(\)]+\)[^\(\)]*\)", remove_uninteresting_citations(dec)) is not None else False)
#         ].to_csv("descasamento_de_parenteses.csv", sep="@")
# txt = """Acórdãos citados: RE 101114 (RTJ 108/1369), RE 113149 (RJ 136/774), RE 117788, RE 117907,
#          RE 120117 (RTJ 147/286) RTJ 108/1369, 114/361. Número de páginas: (11). Análise: (MML).
#          Revisão: (CTM/AAF). Inclusão: 07/11/02, (MLR). Alteração: 15/06/2018, GIB."""


,acordaoId,observacao
19,AGR RE 356736,"Votação: unânime. Resultado: desprovimento. Acórdãos citados: RTJ-158/327, RTJ-158/934. Decisão monocrática: RE-233929, AI-269131, AI-271425, AI-272123, RE-338927, RE-360269. Número de páginas: (09). Análise:(CEL). Revisão:(RCO). Inclusão: 01/04/04, (SVF). Alteração: 25/10/2018, PDR."
69,AGR RE 222874,"Acórdão citado: 201462. Número de páginas: (05). Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 22/08/03, (MLR). Alteração: 04/04/2019, HAC."
368,MC ADI 2254,"Acórdão citado: (RTJ 100/1003). Número de páginas: (23). Análise:(DMV). Revisão:(FLO). Inclusão: 02/04/04, (SVF). Alteração: 14/04/04, (NT). Alteração: 13/11/2018, CLS."
397,AGR RE 285167,"Acórdãos citados: RE230156. Número de páginas: (06). Análise:(MML). Revisão:(RCO/AAF). Inclusão: 29/05/01, (SVF). Alteração: 16/01/2018, ALS."
685,AGR AI 304365,"Acórdãos citados : RTJ 107/661; RTJ 120/912; RTJ 125/705. Número de páginas: (05). Análise: (MML). Revisão: (COF). Inclusão: 31/07/01, (SVF). Alteração: 29/01/2018, GIB."
...,...,...
86276,AGR ARE 1157575,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 6. Análise: 22/01/2019, MJC."
86304,AGR HC 141996,"- Acórdão(s) citado(s): (HC, AUSÊNCIA, PERTINÊNCIA, LOCOMOÇÃO FÍSICA, DESCABIMENTO) RTJ 116./523, RTJ 141/159, RTJ 135/593, RTJ 142/896, RTJ 197/587. Número de páginas: 10. Análise: 13/12/2018, BMP."
86375,AGR ARE 1154088,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. (RE, REEXAME DE FATO) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 8. Análise: 23/01/2019, MJC."
86672,AGR ARE 1146806,"- Acórdão(s) citado(s): (RE, REEXAME DE FATO E PROVA) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 6. Análise: 23/01/2019, MJC."


In [23]:
txt = "Acórdãos citados de outros tribunais: HC 6918, (STJ). Número de páginas: (08). Análise:(CRP). Revisão:(CMM/AAF). Inclusão: 05/12/01, (SVF). Alteração: 06/12/01, (SVF). Alteração: 01/03/2018, JRM."
txt = "Votação: unânime. Resultado: indeferido. Acórdãos citados: STJ: RESP-4889, RESP-27028. Número de páginas: (05). Análise:(JOY). Revisão:(MSA/RCO). Inclusão: 28/10/04, (MLR). Alteração: 24/11/04, (NT). Alteração: 06/09/2019, (PDR)."
# re.search(r"(Acórdãos?\s*(de)?\s*\-?\s*outros\s*tribunais|Acórdão\(?s\)?\*citado\(s?\)?\s*\-?\s*outros\s*tribunais|Acórdãos? citados do STJ|Decisões monocráticas citad[ao]s?|Decisão monocrática citada).*", txt, flags=re.IGNORECASE)

re.sub(r"(Acórdão\(?s?\)?\s*(citado\(?s?\)?)?\s*(do)?\s*:?\s*(STJ|TST)).*", "", txt, flags=re.IGNORECASE)
# re.search(r"(Acórdão\(?s?\)?\s*(citado\(?s?\)?)?\s*(de)?\s*\-?\s*outros?\s*tribuna(is|l))", txt, flags=re.IGNORECASE)

'Votação: unânime. Resultado: indeferido. '

In [24]:
df_citacoes_com_problemas[df_citacoes_com_problemas["observacao"].str.contains("Acórdãos citados competência da Justiça do Trabalho:")]

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index
4411,5e329fccd0a3cf19aa5bb386,EDV RE 146942,DJ 09-08-2002 PP-00085 EMENT VOL-02077-01 PP-00090,[],EMBTE. : UNIVERSIDADE DE SÃO PAULO - USP \nADVDOS. : JOSÉ ALBERTO COUTO MACIEL E OUTROS \nEMBDOS. : LUIZ ROBERTO ALVES E OUTROS \nADVDO. : ANTÔNIO ROBERTO SANDOVAL FILHO,"Acórdãos citados competência da Justiça do Trabalho: RE 141051, RE 141862, RE 143876. Acórdãos citados competência da Justiça comum Estadual: RE 148753, RE 172209. Número de páginas: (11). Análise:(VAS). Revisão:(RCO/AAF). Inclusão: 20/02/03, (SVF). Alteração: 05/06/2018, JRM.",[],4590


In [25]:
# "observacao_lab" é a coluna onde são examinadas regex para melhorar a extração das decisões
def get_stj_tse_mention(txt):
    match = re.search(r"(STJ|TSE)\s*:[^;\.]+", txt)
    return match.group() if match is not None else ""

df_after["observacao_lab"] = df_after["observacao"].apply(lambda x: get_stj_tse_mention(x))

In [26]:
df_after[(df_after["observacao_lab"] == "") & 
         df_after["observacao"].apply(lambda x: True if re.search(r"\((STJ|TSE|TST)\)", remove_useless_observation_segment(x)) else False) #&
        ][["observacao", "observacao_lab"]]#.to_csv("stf_stj_citacoes_dois_pontos.csv", sep="@")

,observacao,observacao_lab
91,"Acórdãos citados: HC 17073 (STJ), HC 7731 (STJ). Número de páginas: (5). Análise:(MML). Revisão:(COF/AAF). Inclusão: 08/07/03, (SVF). Alteração: 02/02/06, (SVF). Alteração: 27/08/2018, HAC.",
155,"Acórdãos citados: RE 100397 ED, Rcl 217; RESP 29101 (STJ). Número de páginas: (05). Análise:(MML). Revisão:(AAF). Inclusão: 30/05/03, (SVF). Alteração: 02/06/03, (SVF). Alteração: 05/09/2018, ALS.",
276,"Acórdãos de outros Tribunais: HC 6918, (STJ). Número de páginas: (08). Análise:(CRP). Revisão:(CMM/AAF). Inclusão: 05/12/01, (SVF). Alteração: 06/12/01, (SVF). Alteração: 01/03/2018, JRM.",
814,"Acórdãos citados : RVC 4886, (RTJ-146/49), HC 13772 (STJ). Número de páginas: (13). Análise:(CRP). Revisão:(CMM/AAF). Inclusão: 06/12/01, (SVF). Alteração: 09/03/2018, JLS.",
1473,"Acórdão citado: EARMS 2331 (STJ). Número de páginas: (08). Análise:(FLO). Revisão:(CMM/AAF). Inclusão: 22/02/02, (MLR). Alteração: 04/03/02, (MLR). Alteração: 14/02/2018, ALS.",
1949,"Acórdãos citados: ADI 1751, HC 63373 (RHC 64373) (RTJ 121/96), HC 67039, HC 68348 (RTJ 135/1041), HC 74678, HC 75261 (RTJ 163/759), HC 75338 (RTJ 167/206), HC 75677, HC 78051 (RTJ 170/935), HC 80204 (RTJ 176/816), RHC 80478, HC 80810, RESP 88881, RE 143776; RMS 6735 (STJ). Número de páginas: (76). Análise:(MML). Revisão:(AAF). Inclusão: 07/07/03, (SVF). Alteração: 08/06/04, (NT). Alteração: 04/09/2018, ALS.",
2380,"Acórdão citado: RE 252245, (Reclamação nº 479 (STJ). Número de páginas: (10). Análise:(CRP). Revisão:(RCO/AAF). Inclusão: 07/06/02, (MLR). Alteração: 11/06/02, (MLR). Alteração: 11/04/2018, JRM.",
2450,"Acórdãos citados: AGRRCL 1067 (RTJ 171/751), RCL 1105, RE 201158, AGRAG 202645, AGRAG 213885, em sentido contrário: RE 179276 (RTJ 173/251), RE 196567, RE 223230 (RTJ 172/639), AGMC 1626 (STJ). Número de páginas: (13). Análise:(CMM). Inclusão: 17/04/02, (MLR). Alteração: 19/04/02, (MLR). Alteração: 17/04/2018, JRM.",
2456,"Acórdãos citados: HC 81162, HC 12487 (STJ), RHC 11225 (STJ). Número de páginas: (09). Análise:(MML). Revisão:(RCO). Inclusão: 15/04/02, (MLR). Alteração: 16/04/2018, JRM.",
2889,"Acórdão citado de outro tribunal: INQ-214/DF (STJ). Número de páginas: (07). Análise:(MML). Revisão:(CTM/AAF). Inclusão: 25/06/02, (SVF). Alteração: 27/06/02, (SVF). Alteração: 18/04/2018, ALS.",


In [26]:
txt = "Acórdão citado: RE 252245, (Reclamação nº 479 (STJ). Número de páginas: (10). Análise:(CRP). Revisão:(RCO/AAF). Inclusão: 07/06/02, (MLR). Alteração: 11/06/02, (MLR). Alteração: 11/04/2018, JRM."
get_stj_tse_mention(txt)
re.sub(r"[,;]?[\w\d\s]+\((STJ|TSE|TST)\)", "", txt)

'Acórdão citado: RE 252245, (. Número de páginas: (10). Análise:(CRP). Revisão:(RCO/AAF). Inclusão: 07/06/02, (MLR). Alteração: 11/06/02, (MLR). Alteração: 11/04/2018, JRM.'

In [27]:
df_after[(df_after["observacao_lab"] == "") & 
         df_after["observacao"].apply(lambda x: True if re.search(r"\((STJ|TSE|TST)\)", remove_useless_observation_segment(x)) else False)
        ][["observacao"]].apply(lambda x: re.sub(r"[,;]?[\w\d\s]+\((STJ|TSE|TST)\)", "", x["observacao"]), axis=1)

91                                                                                                                                                                                                                                                                                                     Acórdãos citados:. Número de páginas: (5). Análise:(MML). Revisão:(COF/AAF). Inclusão: 08/07/03, (SVF). Alteração: 02/02/06, (SVF). Alteração: 27/08/2018, HAC.
155                                                                                                                                                                                                                                                                                 Acórdãos citados: RE 100397 ED, Rcl 217. Número de páginas: (05). Análise:(MML). Revisão:(AAF). Inclusão: 30/05/03, (SVF). Alteração: 02/06/03, (SVF). Alteração: 05/09/2018, ALS.
276                                                                                       

In [29]:
txt = "- Veja HC 14411 do STJ. - Veja Processo-crime 656/2000 da Comarca de Belém do São Francisco - PE. Número de páginas: 7. Análise: 22/09/2010, KBP. Revisão: 23/09/2010, IMC."

txt = re.sub(
    r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*", r"\2", txt, flags=re.IGNORECASE
)

re.match(
    r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*", txt, flags=re.IGNORECASE
)
veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"

while re.match(veja_pattern, txt, flags=re.IGNORECASE):
    txt = re.sub(
        veja_pattern, r"\2", txt, flags=re.IGNORECASE
    )
txt

'- '

In [29]:
magazines_list = ["MAG-CD", "MAG-DVD", "RGJ-CD", "JTARS", "BIBJURID-DVD",
                  "JCSTF", "JPSTF", "JPO-DVD", "JBCC", "JBC", "JBT",
                  "JC", "JTJ", "JTJ-CD", "JURISonline-INT",
                  "JurisSíntese-DVD", "JurisSíntese-INT", "LEXJTACSP",
                  "LEXSTF", "LEXSTF-CD", "MAG-INT", "NRDF",
                  "PLENUMonline-INT", "REPIOB", "RADCOAST", "RB", "RCJ",
                  "RDA", "RDC", "RDP", "RDTAPET", "RDJTJDFT", "RDJ", "RET",
                  "RJDTACSP", "RJADCOAS", "RJTJRS", "RJTJRS-INT",
                  "RDECTRAB", "RDDP", "RDDT", "RMP", "RT", "RF", "RIP",
                  "RIOBTP", "RJTS", "RJSP", "RJMG", "RJP", "RJP-CD",
                  "REVJMG", "REVJMG-INT", "RLTR", "RMDPPP", "RNDJ",
                  "RPTGJ", "RSJADV", "RST", "RSTP", "RTFP", "RTJ",
                  "RTJE", "SINTESE-INT", "COAD-INT"
                 ]
max_mag_size = max([len(mag) for mag in magazines_list])
for mag in magazines_list:
    dec_cit_size = sum(df_after["observacao"].apply(lambda x: 0 if re.search("(\s+|\()" + mag + "[^\w]+", x) is None else 1))
    dec_cit_count = sum(df_after["observacao"].apply(lambda x: len(re.findall("[^\w]+(" + mag + "\-?[\s\d]+\/\d+)", x))))
    if (dec_cit_size > 0):
        print(mag + " " * (max_mag_size - len(mag)), dec_cit_size, " " * (4 - len(str(dec_cit_size))), dec_cit_count)
        
# + Identificar citações de revistas
# + Ver como é feita a citação da revista na 'observação' comparando em como ela é citada em 'publicação'
# padrão de citação: "[^\w]+(\w+\-?[\s\d]+\/\d+)"

# Buscar citações em publicações para traduzir para acordaoId
# Usar casos exemplo para testar extração de decisões


# Pegar uma amostra de citações de revista que não encontraram decisões e descobrir
# se realmente as decisões não foram extraídas.

JTJ              13    9
LEXSTF           18    28
RDA              121   209
RDC              511   0
RDP              4     3
RDJ              1     1
RJDTACSP         1     1
RDDT             1     2
RT               675   2292
RF               181   335
RJSP             3     3
RMDPPP           1     1
RTJ              10573  32265


In [30]:
# REMOÇÃO DE CONTEÚDO ENTRE PARÊNTESES
txt = """- Acórdão(s) citado(s): (EMBARGOS INFRINGENTES, AÇÃO PENAL ORIGINÁRIA) HC 72465 (1ªT).
         (DUPLO GRAU DE JURISDIÇÃO, EXCEÇÕES) AP 470 (TP).
         (EMBARGOS INFRINGENTES, AÇÃO PENAL ORIGINÁRIA, TRIBUNAL INFERIOR) HC 71124 (1ªT), HC 71949 (1ªT),
         HC 71951 (1ªT), HC 72465 (1ªT), RHC 53947 (2ªT).
         (EMBARGOS INFRINGENTES, AÇÃO DE CONTROLE CONCENTRADO DE CONSTITUCIONALIDADE) ADI 1591 (1ªT),
         ADI 1289 EI (TP). (EMBARGOS INFRINGENTES, AÇÃO RESCISÓRIA, ÂMBITO, STF) AR 1472 EI-AgR (TP),
         AR 1178 EI-QO (TP). (EMBARGOS INFRINGENTES, AUSÊNCIA, QUÓRUM MÍNIMO) AO 1046 ED (TP).
         (EMBARGOS INFRINGENTES, CABIMENTO, HIPÓTESE) ADI 171 EI, ADI 1591 EI (TP), HC 88247 AgR-AgR (TP),
         HC 72664 EI-AgR (2ªT), RE 172004 EI-AgR (2ªT), AI 617763 AgR-ED-AgR-ED (2ªT), AI 828792 AgR-EI (1ªT),
         ADI 29 EI (TP), RCL 377 EI-AGR (TP). (DUPLO GRAU DE JURISDIÇÃO, CONVENÇÃO AMERICANA DE DIREITOS HUMANOS)
         RHC 79785 (TP). (REGIMENTO INTERNO, ALCANCE, PODER NORMATIVO) ADI 1105 (TP), ADI 2580 (TP), ADI 2970 (TP),
         MS 28447 (TP), ADI 1503 (TP), ADI 1152(TP).
         (RECEPÇÃO, CONSTITUIÇÃO FEDERAL DE 1988, REGIMENTO INTERNO DO STF, FORÇA DE LEI) AI 148475 AgR (1ªT),
         SL 32 AgR (TP), RE 146747 EDv-AgR (TP), Pet 2961 QO (2ªT), RE 433592 AgR-ED-EDv-AgR (TP), RCL 377 EI-AGR,
         SS 260 QO (TP), RTJ 190/1084, RTJ 147/1010, RTJ 151/278. (REGIMENTO INTERNO DO STF, REVOGAÇÃO TÁCITA)
         RE 220286 EDv-AgR (TP), RE 146747 EDv-AgR (TP), MS 1637 EI (TP) .
         (APLICAÇÃO, NORMA MAIS FAVORÁVEL, TRATADO INTERNACIONAL, DIREITOS HUMANOS) HC 96772 (2ªT), HC 90450 (2ªT).
         (EMBARGOS INFRINGENTES, VALIDADE, NORMA, REGIMENTO INTERNO) HC 71124 (1ªT), ADI 171 EI, ADI 29 EI-AGR (TP).
         (AGRAVO REGIMENTAL, INDEFERIMENTO LIMINAR, MS) Rp 1299 - RTJ 119/980, MS 23466 AgR (2ªT).
         (RECEPÇÃO, CONSTITUIÇÃO FEDERAL DE 1988, EMBARGOS INFRINGENTES, REGIMENTO INTERNO DO STF) SS 327,
         SL 32 AgR (TP), RE 140829 EDv-ED (TP), AP 409 EI, RCL 377 EI-AGR, SS 260 QO (TP).
         (COMPETÊNCIA, STF, DISPOSIÇÃO, MATÉRIA PROCESSUAL, ÂMBITO, CONSTITUIÇÃO FEDERAL DE 1967) AP 409 EI,
         RP 700 EI-AGR (TP), RTJ 54/183, RTJ 69/138. (SUPRALEGALIDADE, TRATADO INTERNACIONAL, DIREITOS HUMANOS)
         RE 349703 (TP). (PRISÃO PREVENTIVA, CLAMOR PÚBLICO) RTJ 112/1115, RTJ 172/159, RTJ 180/262, RTJ 187/933,
         RTJ 193/1050. (RECLAMAÇÃO, PRINCÍPIO DA RESERVA ABSOLUTA DE LEI FORMAL) RP 1092 - RTJ 112/540.
         (RECEPÇÃO, CONSTITUIÇÃO FEDERAL, EFEITO, LEIS ANTERIORES) RTJ 71/289, RTJ 77/657.
         - Decisões monocráticas citadas: (EMBARGOS INFRINGENTES, CABIMENTO, HIPÓTESE) HC 83678 AgR, HC 92303 EI,
         RHC 97473 EI, HC 104075 ED. (EMBARGOS INFRINGENTES, NORMA, VALIDADE) AP 409 EI.
         - Acórdão(s) citado(s) - outros tribunais: STJ: Agravo Regimental nos Embargos de Divergência em Agravo
         1.321.228. (EMBARGOS INFRINGENTES, AÇÃO PENAL ORIGINÁRIA, TRIBUNAL INFERIOR) RT 571/295.
         Sentença n. 51 de 1985 do Reinado da Espanha.
         - Legislação estrangeira citada: art. 130 da Constituição de Portugal de 1822, art. 24 e art. 102,
         n. 1, da Constituição da Espanha de 1812, arts. 68 e 68.1 da Constituição da França, art. 134 da Constituição
         da Italia, arts. 103 e 125 da Constituição da Bélgica, art. 32, 3 da Constituição da Suíça, art.
         200 da Constituição da Venezuela, art. 99 e 100 da Constituição do Perú, arts. art. 29, 174, 175, 186,
         199 e art 234 da Constituição da Colômbia de 1991, art. 57, 2º da Lei Orgânica do Poder Judiciário
         (Lei n. 6, de 1º de julho de 1985) da Espanha. - Decisões estrangeiras citadas: Sentença 51 de 1985 e
         Sentença n. 66 de 2001 do Tribunal Constitucional espanhol, Sentença C-142 de 1993, Sentença C-650 de 2001,
         Sentença C-934 de 2006 e C-545 de 2008 da Corte Constitucional Colômbiana, § 74 e § 90 do trecho da Sentença
         da Corte Interamericana de Direitos Humanos no Caso Barreto Leiva contra Venezuela. - Veja AP 470 do STF.
         Número de páginas: 277. Análise: 09/04/2014, RAF. Revisão: 17/07/2014, JOS."""

# Ver casos em que parênteses não casam
# \([^\(]+\) # casa
# \([^\(\)]+\( # não casa

In [58]:

# dec = "Acórdãos citados: AI-199935-AgR, AI-214562-AgR; RTJ-144/948, RTJ-165/681- 682. Número de páginas: (07). Análise:(NAL). Inclusão: 31/08/05, (SVF)."
# df_after[df_after["observacao"].apply(lambda dec: True if re.search(r"\([^\(\)]+\(", remove_uninteresting_citations(dec)) is not None else False) &
#          ~df_after["observacao"].apply(lambda dec: True if re.search(r"\([^\(\)]*\([^\(\)]+\)[^\(\)]+\)|\([^\(\)]+\([^\(\)]+\)[^\(\)]*\)", remove_uninteresting_citations(dec)) is not None else False)
#         ]#.to_csv("descasamento_de_parenteses.csv", sep="@")
# Análise realizada e foram feitas leves modificações para lidar com isso

# check decisions ids between parenthesis
# df_after[df_after["observacao"].apply(lambda dec:
#                                       True if re.search(r"\([^\(\w]*(\w{2,}[\s-]+\d+\s*\w*[^\)\/\d]*)\)",
#                                                         remove_uninteresting_citations(dec)) is not None else False) &
#          ~df_after["observacao"].apply(lambda dec:
#                                       True if re.search(r"\((\w{2,}[\s-]+\d+\s*\w*)\)",
#                                                         remove_uninteresting_citations(dec)) is not None else False)
#         ].to_csv("decision_ids_between_parenthesis.csv", sep="@")
# INFORMAR RESULTADO DAS OBSERVAÇÕES
# Nada a aproveitar. Pode descartar tudo.

In [30]:
txt = """- Acórdão(s) citado(s): (RECLAMAÇÃO, CONTRUÇÃO PRETORIANA) STF: RTJ 112/504. (RECLAMAÇÃO, GARANTIA DA
         AUTORIDADE, STF) STF: Rcl 1722 (TP), Rcl 2143 AgR (TP), Rcl 2256 (TP), Rcl 935 (TP); RTJ 149/354. (RECLAMAÇÃO,
         GARANTIA DA AUTORIDADE, MEDIDA CAUTELAR, CONTROLE ABSTRATO STF: Rcl 1723 AgR-QO (TP), Rcl 2256 (TP);
         RTJ 157/433 (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX NUNC) STF: RTJ 124/80.
         (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX TUNC, EXCEPCIONALIDADE) STF:
         ADI 2381 MC-ED; RTJ 138/86. - Veja ADI 2381 MC e ADI 2737 do STF e Agravo Regimental em Suspensão de Liminar 70006089866 do TJRS e Mandado de Segurança 005/1.03.0001168-6 da 3ª Vara Cível Comarca de Bento Gonçalves.
         Número de páginas: 32. Análise: 26/02/2013, SEV. Revisão: 03/04/2013, AAT."""
veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
# re.search(veja_pattern, txt, flags=re.IGNORECASE)
# re.sub(
#         veja_pattern, r"\2", txt, flags=re.IGNORECASE
#     )
while re.search(veja_pattern, txt, flags=re.IGNORECASE):
    txt = re.sub(
        veja_pattern, r"\2", txt, flags=re.IGNORECASE
    )
txt

'- Acórdão(s) citado(s): (RECLAMAÇÃO, CONTRUÇÃO PRETORIANA) STF: RTJ 112/504. (RECLAMAÇÃO, GARANTIA DA\n         AUTORIDADE, STF) STF: Rcl 1722 (TP), Rcl 2143 AgR (TP), Rcl 2256 (TP), Rcl 935 (TP); RTJ 149/354. (RECLAMAÇÃO,\n         GARANTIA DA AUTORIDADE, MEDIDA CAUTELAR, CONTROLE ABSTRATO STF: Rcl 1723 AgR-QO (TP), Rcl 2256 (TP);\n         RTJ 157/433 (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX NUNC) STF: RTJ 124/80.\n         (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX TUNC, EXCEPCIONALIDADE) STF:\n         ADI 2381 MC-ED; RTJ 138/86. - \n         Número de páginas: 32. Análise: 26/02/2013, SEV. Revisão: 03/04/2013, AAT.'

In [331]:
# Buscar citações em publicações para traduzir para acordaoId

# ENCONTRAR CITAÇÃO QUE EXISTE EM OBSERVACAO E CITACOES_REVISTAS
# cit_magazine = "RTJ 168/863"
found_acordao_id = lambda cit_magazine_lst, cit_magazine: True if cit_magazine in cit_magazine_lst else False
# for index, row in df_after.iterrows():
# COMMON CASE of decision ids
txt = """- Acórdão(s) citado(s): (CONTRADITÓRIO, AMPLA DEFESA, COISA JULGADA, DEVIDO PROCESSO LEGAL)
         ARE 748371 RG . (RECEITAS TRIBUTÁRIAS, PARTICIPAÇÃO DOS MUNICÍPIOS, EXCLUSÃO, REDUÇÃO, SUPRESSÃO)
         RE 572762 RG , RTJ 82/200, RTJ 85/712, RTJ 86/722, RTJ 89/233, RTJ 516/223.
         (HONORÁRIOS ADVOCATÍCIOS, MAJORAÇÃO) AO 2063 AgR (TP). Número de páginas: 13. Análise: 09/01/2020, BMP."""
# CASE WHICH MAGAZINE CODES ARE REMOVED between parenthesis
txt = """Acórdãos citados: MI 211 (RTJ 157/411, JUA-1771/1918), MI 263 (RTJ 157/22), MI 274 AgR (RTJ 152/357),
         MS 21521 (RTJ 150/119), RE 140863, RE 161224, RE 179646, AI 221703 AgR.
         Número de páginas: (4). Análise:(CEL). Revisão:(FLO). Inclusão: 01/08/03, (SVF).
         Alteração: 05/12/03, (SVF). Alteração: 03/09/2018, JLS."""
# MAGAZINE CODES THAT SHOUD BE REMOVED
txt = """- Acórdãos citados: ADI 596 MC - Tribunal Pleno, ADI 939, ADI 1610 MC - Tribunal Pleno,
         ADI 2105 MC - Tribunal Pleno, ADI 2661 MC - Tribunal Pleno, RE 18331, RE 204769,
         AI 725227 AgR; TRF 4ª Região: 2007.71.08.012143-2; RTJ 87/374, RTJ 144/435,
         RTJ 167/661; RF 145/164; RDA 34/132. - Decisão monocrática citada: RE 428354.
         - Veja Informativo 125 do STF (Pet 1466) e Exposição Interministerial 84 do Ministério da Fazenda
         e da Casa Civil. - Decisões estrangeiras citadas: Caso McCulloch v. Maryland, de 1819 e
         Caso Panhandle Oil Co. v. State of Mississippi Ex Rel. Knox (277 U.S. 218), julgado em 1928,
         ambos da Suprema Corte do Estados Unidos da América Número de páginas: 44.
         Análise: 28/03/2012, MMR. Revisão: 20/04/2012, SEV."""
# DECISÕES SEGUIDAS DE CITAÇÕES DE REVISTA APÓS '-' (EQUIVALENTE A PARÊNTESES)
txt = """- Acórdão(s) citado(s): (HC, REEXAME, FATO, PROVA) HC 65887 , HC 66381 , HC 68610 (1ªT) - RTJ 136/1221,
         HC 69780 (1ªT) - RTJ 195/486, HC 70763 (1ªT) - RTJ 165/877, HC 72496 (1ªT) - RTJ 186/237,
         HC 72992 (1ªT) - RTJ 168/863, HC 73449 (1ªT) - RTJ 163/650, HC 96820 (1ªT), RHC 99388 (2ªT),
         HC 100067 (1ªT), HC 112507 (2ªT), RHC 131136 AgR (2ªT), HC 60555 (1ªT) - RTJ 110/555,
         HC 67349 (1ªT) - RTJ 129/1199, RHC 63806 (1ªT) - RTJ 137/198. (FUNDAMENTAÇÃO, DECISÃO JUDICIAL)
         RE 140370 (1ªT) - RTJ 150/269, AI 152586 AgR , RE 327143 AgR (2ªT), AI 529105 AgR (2ªT),
         AI 637301 AgR (1ªT), AI 731527 AgR (2ªT), AI 791292 QO-RG , AI 838209 AgR (2ªT), AI 842316 AgR (1ªT),
         AI 840788 AgR (1ªT), RTJ 170/627. (HC, SUPRESSÃO DE INSTÂNCIA) HC 73390 (2ªT), HC 79551 (1ªT),
         HC 80747 (1ªT) - RTJ 182/243, HC 81115 (1ªT), HC 83842 (2ªT) - RTJ 192/233, HC 97761 (2ªT).
         (ALEGAÇÃO, NULIDADE, PRECLUSÃO) HC 53508 (1ªT) - RTJ 76/435, HC 72454 (1ªT), HC 74339 (1ªT),
         HC 58994 (1ªT) - RTJ 104/540, RT 451/490. (HC, SUCEDÂNEO, REVISÃO CRIMINAL) HC 70457 ,
         HC 98206 (1ªT), HC 98412 (2ªT), RHC 116674 (2ªT), HC 122753 AgR (2ªT), RHC 130270 (1ªT),
         RHC 128723 AgR (1ªT), RHC 133200 (2ªT), HC 134974 (2ªT), HC 123182 AgR (1ªT), HC 134976 AgR (1ªT),
         HC 137059 AgR (2ªT). - Acórdão(s) citado(s) - outros tribunais: STJ: HC 130146, RHC 68917 EDcl-AgRg.
         Número de páginas: 20. Análise: 11/09/2019, JRS."""
# sequência mal formada de decisões citadas
txt = """Acórdãos citados: RE 101114 (RTJ 108/1369), RE 113149 (RJ 136/774), RE 117788, RE 117907,
         RE 120117 (RTJ 147/286) RTJ 108/1369, 114/361. Número de páginas: (11). Análise: (MML).
         Revisão: (CTM/AAF). Inclusão: 07/11/02, (MLR). Alteração: 15/06/2018, GIB."""
# manter acordao Id entre parênteses e remover citação à revista
txt = """- Acórdão(s) citado(s): (PRINCÍPIO DO JUIZ NATURAL) TJ 169/557 (HC 73801), RTJ 179/378 (HC 79865).
         (EXTRADIÇÃO, DUPLA TIPICIDADE) Ext 977 (TP). (PRINCÍPIO DA CONTENCIOSIDADE LIMITADA)
         Ext 669 (TP) - RTJ 161/409. (PRISÃO PREVENTIVA PARA EXTRADIÇÃO) HC 81127 (TP), Ext 1313 (1ªT).
         - Legislação estrangeira citada: Código Penal Uruguaio. Número de páginas: 10. Análise: 13/06/2017, JSF."""
# Parênteses descasados
txt = """- Acórdão(s) citado(s): (MOTIVAÇÃO "PER RELATIONEM") (HC 69438 (1ªT) , HC 69987 (TP), HC 54513 (2ªT),
         RE 37879 (TP), RE 49074 (1ªT). Número de páginas: 9. Análise: 20/02/2013, MAC."""
# Dois níveis de parênteses
txt = """- Acórdão(s) citado(s): (SUPREMO TRIBUNAL FEDERAL (STF), COMPETÊNCIA ORIGINÁRIA, HABEAS CORPUS,
         TURMA RECURSAL DOS JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS) HC 71713 (TP), HC 77647 (1ªT), HC 81340 (2ªT);
         RTJ 168/222, RTJ 175/279, RTJ 177/784. (PENA PRIVATIVA DE LIBERDADE, SUBSTITUIÇÃO, PENA DE MULTA)
         HC 66887 (1ªT); RTJ 125/548, RTJ 143/199. (PENA PRIVATIVA DE LIBERDADE, SUBSTITUIÇÃO,
         PENA RESTRITIVA DE DIREITO) RTJ 159/132. - Decisão monocrática citada: (SUPREMO TRIBUNAL FEDERAL (STF),
         COMPETÊNCIA ORIGINÁRIA, HABEAS CORPUS, TURMA RECURSAL DOS JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS)
         HC 81228 MC. - Veja Recurso 2001.700.007575-4 do Conselho Recursal e Proceso crime 17461-2/00 do
         Juizado Especial Criminal da Comarca do Rio de Janeiro/RJ. Número de páginas: 15. Análise: 12/03/2013,
         IMC. Revisão: 11/04/2013, MMR."""
# Decisão do STF após sequência de texto mal cercada de parênteses
txt = """- Acórdão(s) citado(s): (RECLAMAÇÃO, CONTRUÇÃO PRETORIANA) STF: RTJ 112/504. (RECLAMAÇÃO, GARANTIA DA
         AUTORIDADE, STF) STF: Rcl 1722 (TP), Rcl 2143 AgR (TP), Rcl 2256 (TP), Rcl 935 (TP); RTJ 149/354. (RECLAMAÇÃO,
         GARANTIA DA AUTORIDADE, MEDIDA CAUTELAR, CONTROLE ABSTRATO STF: Rcl 1723 AgR-QO (TP), Rcl 2256 (TP);
         RTJ 157/433 (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX NUNC) STF: RTJ 124/80.
         (MEDIDA CAUTELAR, AÇÃO DIRETA DE INCONSTITUCIONALIDADE, EFICÁCIA EX TUNC, EXCEPCIONALIDADE) STF:
         ADI 2381 MC-ED; RTJ 138/86. - Veja ADI 2381 MC e ADI 2737 do STF e Agravo Regimental em Suspensão de Liminar 70006089866 do TJRS e Mandado de Segurança 005/1.03.0001168-6 da 3ª Vara Cível Comarca de Bento Gonçalves.
         Número de páginas: 32. Análise: 26/02/2013, SEV. Revisão: 03/04/2013, AAT."""
# Outro exemplo de sequência mal cercada de parênteses
txt = """- Acórdão(s) citado(s): (EDITAL, "LEI DO CONCURSO PÚBLICO") RE 526600 AgR (1ªT), RE 440335 AgR (2ªT).
         (CNJ, ATUAÇÃO ADMINISTRATIVA, CONCURSO PÚBLICO) MS 26284 (TP), MS 27160 (TP), MS 26163 (TP).
         (CONCURSO PÚBLICO, QUESTÃO IMPUGNADA, CLASSIFICAÇÃO, DIREITO ADQUIRIDO) RMS 21255 (TP).
         (ANULAÇÃO, CONCURSO PÚBLICO ENCERRADO, INTIMAÇÃO, INTERESSADO MS 25962 (TP). (ANULAÇÃO, APOSENTADORIA,
         INTIMAÇÃO, INTERESSADO) MS 24268 (TP), MS 25116 (TP). (SERVIDOR, NOMEAÇÃO, FORA DO PRAZO DE VALIDADE,
         CONCURSO PÚBLICO, INTIMAÇÃO, INTERESSADO) MS 27154 (TP). (PROCESSO JUDICIAL, ESGOTAMENTO, VIA ADMINISTRATIVA)
         RE 233582 (TP). (PRESUNÇÃO, CIÊNCIA, ATO REVOGATÓRIO) MS 22938 (TP), MS 24927 (TP), MS 24742 (TP).
         - Veja MS 28594, MS 28651 e MS 28666 do STF. Número de páginas: 76. Análise: 02/08/2013, SEV."""
# Outro caso de parênteses descasados
txt = """Acórdãos citados: ADI 766 MC (157/460), ADI 864, ADI 151/416), ADI 139 (RTJ 138/14), ADI 182 (RTJ 133/1037),
         ADI 234 (RTJ 167/363), ADI 430 (RTJ 159/735). Número de páginas: (09). Análise:(MML). Revisão:(CTM/AAF).
         Inclusão: 07/11/02, (MLR). Alteração: 11/11/02, (MLR). Alteração: 28/05/2018, ALS."""
# outro caso de parênteses descasados (2)
txt = """- Acórdão(s) citado(s): (SÚMULA 343) RE 328812 (TP), RE 564781 AgR (2ªT), RE 500043 AgR 12ªT);
         - Decisões monocráticas citadas: (SÚMULA 343) RE 569140, RE 579740, RE 262589. Número de páginas: 23.
         Análise: 21/02/2013, MJC."""
# Padrão de citação de decisão monocrática mal formado
txt = """Votação: unânime. Resultado: desprovido. Acórdão citado: AI-345003-AgR-ED Decisão monocrática citada: AI-145153.
         Número de páginas: (08). Análise:(RDC). Revisão:(). Inclusão: 25/01/05, (CSM)."""
# problema com extração de padrão após 'Obs.'
txt = """Acórdãos citados: RHC 53192, HC 62786, RHC 63601 (RTJ 118/484), HC 70607, RHC 71354 (RTJ 156/562),
         HC 72277, HC 73545, HC 74106, HC 75385, HC 77583, HC 79750, HC 80096, HC 80379, RHC 81522, HC 81534;
         RTJ 63/410, RTJ 138/829; RTJ 141/816, RTJ 142/855.
         Obs.: - Art. 7, nº 5 da Convenção Americana sobre Direitos Humanos. Número de páginas: (23).
         Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 19/02/04, (MLR). Alteração: 03/01/06, (MLR).
         Alteração: 06/11/2018, CLS."""
# interrupção de extração de decisões após começarem a aparecer decisões do STJ
txt = """Acórdão citado: STJ - REsp 431760. Número de páginas: 20 Análise: 05/09/2007, ACL. Revisão: 28/09/2007, RCO."""
# Citações de revistas
txt = """- Acórdãos citados: AC 471 AgR (RTJ 192/778), AC 491 AgR (RTJ 192/411), AC 510 AgR, AC 865 AgR, AC 1317 MC-ED,
         Pet 1859 AgR (RTJ 174/437), Pet 2835 QO (RTJ 191/483), Pet 2961 QO (RTJ 191/123), RE 407099; RTJ 174/437,
         RTJ 191/123, RTJ 191/483. - Decisão monocrática citada: AC 1449.
         - Veja Apelação Cível n° 100.015.2005.001342-9. Número de páginas: 22. Análise: 04/06/2007, AAC."""

# Pegar uma amostra de citações de revista que não encontraram decisões e descobrir
# se realmente as decisões não foram extraídas.

# NOVO PADRÃO DE CITAÇÃO: RTJ-170/627-628 (o código depois do '-' no final significa o quê? Faz diferença?)
# > aparentemente não faz diferença

# súmulas não são decisões de fato (devem ser ignoradas)
# Olhar conclusões sobre padrões problemáticos e avaliar se vale a pena cuidar deles

quotes = []
txt = re.sub(r"art\.", "art", txt, flags=re.IGNORECASE)
txt = re.sub("(STF:|\-?\s*(t|p)\w*\s*pleno|CASO\s+LÍDER)", "", txt, flags=re.IGNORECASE)
# Verificar se é usado o padrão que remove o máximo possível de caracteres após modificar o padrão aqui
# Em alguns espelhos o padrão que aparece no final acaba aparecendo no começo.
# Então não é possível remover todo o texto que vem depois.
txt = re.sub(
    r"((Número\s*de\s*p[aá]ginas|Altera[aç][aã]o|Revis[aã]o|Inclus[aã]o|An[aá]lise|Obs\.|Vota[cç][aã]o|Resultado):)[^\.]*\.", "", txt, flags=re.IGNORECASE
)
# Além disso, uma decisão específica começa com a string “Veja” Será feita uma exceção para este caso.    
veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
while re.search(veja_pattern, txt, flags=re.IGNORECASE):
    txt = re.sub(
        veja_pattern, r"\2", txt, flags=re.IGNORECASE
    )
# Remoção de decisões pertencentes ao STJ ou ao TSE - este padrão pode ser usado para remover tudo que vem depois,
# TESTAR ISSO DEPOIS

# ver posteriormente que tipo de informação é inserida entre '()' nas citações
txt = re.sub(r"(STJ|TSE|TRE|TST|TRF\s*\-?[\s\w]*)\s*:.+[;\.]\s*", "", txt)

search_pattern = (
    "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
)
#     Acórdão seguido eventualmente de espaços, seguido eventualmente de (s) com eventuais espaços dentro,
#     seguido de espaços, seguido de Citado, seguido eventualmente de espaços, seguido eventualmente de (s)
#     com eventuais espaços dentro seguido de eventuais espaços e eventualmente ':', seguido de caracteres
#     que não sejam ':'. A última sequência (caracteres que não sejam ':') é que o que seve ser capturado.
dec = re.search((search_pattern), txt, flags=re.IGNORECASE)

if dec:
    dec = dec.group(1)
    dec = re.sub(r"[Dd]ecis(?:ão|ões)\s*monocráticas?\s*citada\s*\(?\s*s?\s*\)?\s*", "", dec, flags=re.IGNORECASE)
    print(dec) #######################################
    if (len(dec) > 2) and (dec[-2] == "."):
        dec = dec[:-2]

    dec = re.sub(r"(\d+)\.(\d+)", r"\1\2", dec)
    dec = re.sub(r"[,;]?[\w\d\s\.\/\-]+\((STJ|TSE|TST|TRE|TRF\s*\-?[\s\w]*)\)", "", dec)
    # remover menções a revistas entre parênteses porque isso significa que o próprio
    # id do acórdão citado já é referenciado
    dec = re.sub("\(([A-Z]+\-?[\s\d]+\/\d+\-?\d*,?\s*)+\)", ",", dec)
    dec = re.sub("\-\s*[A-Z]+\-?[\s\d]+\/\d+\-?\d*", ",", dec)
    # manter citação à decisão e não à revista como no exemplo: 'TJ 169/557 (HC 73801)'
    dec = re.sub(r"(\w+\-?[\s\d]+\/\d+\-?\d*)\s+\(([^\d]{2,}[\s-]+\d+[^\d\)]*)\)", r"\2,", dec)

    print(dec) #######################################
    # REMOVER CONTEÚDO ENTRE PARÊNTESES AQUI
    while re.search(r"\([^\(\)]+\)", dec):
        dec = re.sub(r"\([^\(\)]+\)", ",", dec)
    print(dec) #######################################

    # VER O EFEITO DO SPLIT NO PARÊNTESES PARA ENTENDER QUE PADRÃO É EXTRAÍDO          
    dec = re.split("[;,.()]", dec)
    for q in dec:
        q = q.strip()
        print(q) #######################################
        if re.search(r"embargos?\sde|(de|em)\sembargos?|RISTF|ESTRANGEIRA|(\-\s*STJ|\s+STJ|STJ\s+\-|TRF)", q, flags=re.IGNORECASE):
            break

        acordaoId_pattern = "([^\d\s]{2,}[\s-]+\d+[^\d\/]*)$"
        m = re.search(acordaoId_pattern, q)
        print(q, m) #######################################
        if m is None:
            n = re.search("([A-Z]+)\-?\s*(\d+\/\d+)", q)
            print("m is none", n) #######################################
            if (n is not None) and (not q.startswith("-")):
                n = " ".join(n.groups())
                print(n, n in revistas_to_acordao_id_dict) #######################################
                if n in revistas_to_acordao_id_dict:
                    quotes.append(revistas_to_acordao_id_dict[n])
                else:
                    quotes.append(n)
        while m:
            m = m.group()
            q = q.replace(m, "")
            m = m.replace("-", " ")
            m = m.strip().upper()
            m = " ".join(m.split())
            m = normalizeId(m)
            quotes.append(m)
            m = re.search(acordaoId_pattern, q)
sorted(set(quotes))

AC 471 AgR (RTJ 192/778), AC 491 AgR (RTJ 192/411), AC 510 AgR, AC 865 AgR, AC 1317 MC-ED,
         Pet 1859 AgR (RTJ 174/437), Pet 2835 QO (RTJ 191/483), Pet 2961 QO (RTJ 191/123), RE 407099; RTJ 174/437,
         RTJ 191/123, RTJ 191/483. - 
AC 471 AgR ,, AC 491 AgR ,, AC 510 AgR, AC 865 AgR, AC 1317 MC-ED,
         Pet 1859 AgR ,, Pet 2835 QO ,, Pet 2961 QO ,, RE 407099; RTJ 174/437,
         RTJ 191/123, RTJ 191/483. - 
AC 471 AgR ,, AC 491 AgR ,, AC 510 AgR, AC 865 AgR, AC 1317 MC-ED,
         Pet 1859 AgR ,, Pet 2835 QO ,, Pet 2961 QO ,, RE 407099; RTJ 174/437,
         RTJ 191/123, RTJ 191/483. - 
AC 471 AgR
AC 471 AgR <_sre.SRE_Match object; span=(0, 10), match='AC 471 AgR'>

 None
m is none None
AC 491 AgR
AC 491 AgR <_sre.SRE_Match object; span=(0, 10), match='AC 491 AgR'>

 None
m is none None
AC 510 AgR
AC 510 AgR <_sre.SRE_Match object; span=(0, 10), match='AC 510 AgR'>
AC 865 AgR
AC 865 AgR <_sre.SRE_Match object; span=(0, 10), match='AC 865 AgR'>
AC 1317 MC-ED
AC 1317 MC

['AGR AC 471',
 'AGR AC 491',
 'AGR AC 510',
 'AGR AC 865',
 'AGR PET 1859',
 'ED MC AC 1317',
 'QO PET 2835',
 'QO PET 2961',
 'RE 407099',
 'RTJ 174/437']

In [94]:
def traslations_magazine_citations(txt):
    txt = re.sub("STF:|\-\s*tribunal\s+pleno", "", txt, flags=re.IGNORECASE)
    # Verificar se é usado o padrão que remove o máximo possível de caracteres após modificar o padrão aqui
    # Em alguns espelhos o padrão que aparece no final acaba aparecendo no começo.
    # Então não é possível remover todo o texto que vem depois.
    txt = re.sub(
        r"((Número\s*de\s*p[aá]ginas|Altera[aç][aã]o|Revis[aã]o|Inclus[aã]o|An[aá]lise|Obs\.|Vota[cç][aã]o|Resultado):)[^\.]*\.", "", txt, flags=re.IGNORECASE
    )
    # Além disso, uma decisão específica começa com a string “Veja” Será feita uma exceção para este caso.    
    veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
    while re.search(veja_pattern, txt, flags=re.IGNORECASE):
        txt = re.sub(
            veja_pattern, r"\2", txt, flags=re.IGNORECASE
        )
    # Remoção de decisões pertencentes ao STJ ou ao TSE - este padrão pode ser usado para remover tudo que vem depois,
    # TESTAR ISSO DEPOIS

    # ver posteriormente que tipo de informação é inserida entre '()' nas citações
    txt = re.sub(r"(STJ|TSE|TRE|TST|TRF\s*\-?[\s\w]*)\s*:.+[;\.]\s*", "", txt)
    search_pattern = "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
    dec = re.search((search_pattern), txt, flags=re.IGNORECASE)

    quotes = []
    if dec:
        dec = dec.group(1)
        if (len(dec) > 2) and (dec[-2] == "."):
            dec = dec[:-2]

        dec = re.sub(r"(\d+)\.(\d+)", r"\1\2", dec)
        dec = re.sub(r"[,;]?[\w\d\s\.\/\-]+\((STJ|TSE|TST|TRE|TRF\s*\-?[\s\w]*)\)", "", dec)

        # manter citação à decisão e não à revista como no exemplo: 'TJ 169/557 (HC 73801)'    
        dec = re.sub(r"(\w+\-?[\s\d]+\/\d+\-?\d*)\s+\(([^\d]{2,}[\s-]+\d+[^\d\)]*)\)", r"\2", dec)
        # remover menções a revistas entre parênteses ou após um '-' porque isso significa que o próprio
        # id do acórdão citado já é referenciado
        dec = re.sub("\(([A-Z]+\-?[\s\d]+\/\d+\-?\d*,?\s*)+\)", ",", dec)
        dec = re.sub("\-\s*[A-Z]+\-?[\s\d]+\/\d+\-?\d*", "", dec)
        # VER O EFEITO DO SPLIT NO PARÊNTESES PARA ENTENDER QUE PADRÃO É EXTRAÍDO          
        dec = re.split("[;,.()]", dec)
        for q in dec:
            q = q.strip()

            acordaoId_pattern = "([^\d\s]{2,}[\s-]+\d+[^\d]*)$"
            m = re.search(acordaoId_pattern, q)
            if m is None:
                n = re.search("([A-Z]+)\-?\s*(\d+\/\d+)", q)
                if (n is not None) and (not q.startswith("-")):
                    n = " ".join(n.groups())
                    if n in revistas_to_acordao_id_dict:
                        quotes.append(revistas_to_acordao_id_dict[n])

    return quotes

In [96]:
s_citacoes_encontradas = df_after.loc[:, "observacao"].progress_apply(lambda txt: traslations_magazine_citations(txt))
df_after.loc[s_citacoes_encontradas[s_citacoes_encontradas.apply(lambda x: True if len(x) > 0 else False)].index, ["publicacao", "observacao"]].tail(60)

,publicacao,observacao
acordaoId,,
AGR HC 120393,PROCESSO ELETRÔNICO \r\nDJe-188 DIVULG 21-09-2015 PUBLIC 22-09-2015,"- Acórdão(s) citado(s): (MOTIVAÇÃO ""PER RELATIONEM"") HC 69438 (1ªT), HC 69987 (TP), AI 734689 AgR (2ªT), AI 825520 AgR-ED (2ªT), ARE 657355 AgR (1ªT), RE 585932 AgR (2ªT), HC 54513 (2ªT). (PRINCÍPIO DO ""PAS DE NULLITÉ SANS GRIEF"") HC 107769 (1ªT), HC 112558 (2ªT), RHC 116390 (2ªT). (NULIDADE, COMPROVAÇÃO DE PREJUÍZO) HC 85155 (2ªT), HC 100329 (2ªT), HC 104648 (2ªT), HC 110160 (2ªT), HC 112191 (2ªT), HC 116132 (2ªT), HC 117102 (2ªT), RTJ 182/662-663, RTJ 220/385, RT 567/398, RT 570/388, RT 603/311. Número de páginas: 13. Análise: 19/10/2015, MAD."
AGR RE 892938,PROCESSO ELETRÔNICO \r\nDJe-160 DIVULG 14-08-2015 PUBLIC 17-08-2015,"- Acórdão(s) citado(s): (PRESUNÇÃO DE INOCÊNCIA) RE 482006 (TP), HC 95886 (2ªT), RE 814792 AgR (2ªT), RTJ 183/327. (MAUS ANTECEDENTES, INQUÉRITO) RE 464947. Número de páginas: 13. Análise: 21/08/2015, BMP."
AGR ARE 847535,ACÓRDÃO ELETRÔNICO\r\nDJe-154 DIVULG 05-08-2015 PUBLIC 06-08-2015,"- Acórdão(s) citado(s): (CONCURSO PÚBLICO, INVESTIGAÇÃO SOCIAL) RE 450971 AgR (1ªT), RTJ 183/327. (PRINCÍPIO DA PRESUNÇÃO DE INOCÊNCIA) RE 482006 (TP), HC 95886 (2ªT), RTJ 139/885, RT 418/286, RT 422/307, RT 572/391, RT 586/338. Número de páginas: 12. Análise: 14/08/2015, MJC."
EXT 1361,ACÓRDÃO ELETRÔNICO\r\nDJe-156 DIVULG 07-08-2015 PUBLIC 10-08-2015,"- Acórdão(s) citado(s): (INCORPORAÇÃO, TRATADO INTERNACIONAL, ORDENAMENTO JURÍDICO BRASILEIRO) RTJ 174/463, RTJ 179/493. (CONCORDÂNCIA, EXTRADITANDO, PEDIDO, EXTRADIÇÃO) Ext 583 (TP), RTJ 64/22, RTJ 85/7, RTJ 132/137, EXT 352 (TP). (MANDADO DE PRISÃO, EXPEDIÇÃO, AUTORIDADE COMPETENTE, ESTADO REQUERENTE) EXT 478 (TP), EXT 744 (TP), RTJ 89/716, EXT 633 (TP). (PRINCÍPIO DO JUIZ NATURAL) RTJ 169/557, RTJ 179/378, RTJ 193/357. (FURTO DE USO) RTJ 34/655, RTJ 37/96, RTJ 86/42, HC 31892 (TP), RE 63884 (1ªT). (EXTRADIÇÃO, COMUTAÇÃO DA PENA, PRISÃO PERPÉTUA) Ext 426 (TP), Ext 486 (TP), Ext 811 (TP), Ext 838 (TP), Ext 855 (TP), Ext 944 (TP), Ext 985 (TP), Ext 1051 (TP), Ext 1103 (TP), Ext 1104 (TP), Ext 703 ED (2ªT), Ext 1343 (2ªT), EXT 654 (TP), EXT 711 (TP), EXT 773 (TP). (EXTRADITANDO, FILHO BRASILEIRO) Ext 510 (TP), Ext 803 (TP), Ext 822 (TP), Ext 833 (TP), RTJ 191/17, RTJ 177/1250, RTJ 165/472,RTJ 148/110, RTJ 172/751, RTJ 155/34, RTJ 183/42. - Acórdão(s) citado(s) - outros tribunais: (FURTO DE USO) RT 486/320, RT 491/336, RT 523/417, RT 553/370, RT 553/387. Número de páginas: 27. Análise: 31/08/2015, IMC. Revisão: 30/11/2015, KBP."
SEGUNDOS ED ARE 919464,PROCESSO ELETRÔNICO \r\nDJe-018 DIVULG 29-01-2016 PUBLIC 01-02-2016,"- Acórdão(s) citado(s): (INTEMPESTIVIDADE, RECURSO INCABÍVEL) AI 515208 AgR (2ªT), AI 521217 AgR (2ªT), AI 528553 AgR (2ªT), AI 588190 AgR (1ªT), AI 600672 ED (2ªT), AI 600452 AgR (1ªT), AI 602116 AgR (2ªT), AI 718826 AgR (2ªT), AI 578079 AgR (1ªT), AI 733719 AgR (2ªT), AI 746533 ED (TP), AI 766488 AgR (1ªT), ARE 663031 AgR (2ªT), RTJ 105/792, RTJ 105/1275, RTJ 120/458, RTJ 132/1374, RF 148/176, RF 148/179, RF 163/215, RT 489/105, REVISTA DE PROCESSO, VOLS. 1/196 – 1/210 – 4/393. (PRAZO RECURSAL) RT 473/200, RT 504/217, RT 611/155, RT 698/209, RF 251/244, RTJ 203/416. Número de páginas: 8. Análise: 03/02/2016, MJC."
AGR RHC 125242,PROCESSO ELETRÔNICO \r\nDJe-049 DIVULG 14-03-2017 PUBLIC 15-03-2017,"- Acórdão(s) citado(s): (PRINCÍPIO PAS DE NULLITÉ SANS GRIEF) HC 81510 (1ªT) - RTJ 182/662, HC 82899 (2ªT), HC 85155 (2ªT), HC 100329 (2ªT), HC 104648 (2ªT), HC 110160 (2ªT), HC 112191 (2ªT), HC 116132 (2ªT), HC 117102 (2ªT), RHC 126885 (2ªT), RTJ 220/385, RT 567/398, RT 570/388, RT 603/311. (PREVISÃO, LEI, PROCESSO, SUBSTITUIÇÃO, JUIZ) HC 68210 (TP) - RTJ 142/832, HC 68905 , HC 69601 (1ªT) - RTJ 143/962, HC 70103 (1ªT) - RTJ 148/773, HC 74109 (1ªT), ADI 1481 (TP) - RTJ 194/50. (CONVOCAÇÃO, JUIZ, TRIBUNAL) HC 86889 (1ªT) - RTJ 209/1135, RE 597133 (TP), HC 99240 (1ªT), HC 101952 (1ªT), HC 111507 (2ªT), HC 113874 (1ªT), RHC 122002 (2ªT). (HABEAS C

In [58]:
# Creating translation of magazine citations to decision_ids
revistas_to_acordao_id_dict = {}

def translate_revistas_citacoes_to_acordao_id(row, revistas_to_acordao_id_dict): 
    for cit_mag in row["citacoes_revistas"]:
        revistas_to_acordao_id_dict[cit_mag] = row.name


df_after.apply(translate_revistas_citacoes_to_acordao_id,
                                    args=(revistas_to_acordao_id_dict,), axis=1)
print("Número de traduções", len(revistas_to_acordao_id_dict))

Número de traduções 9414


In [59]:
revistas_to_acordao_id_dict

{'RTJ 191/483': 'QO PET 2835',
 'RT 836/442': 'QO INQ 1024',
 'RTJ 193/459': 'QO INQ 1024',
 'RTJ 177/1136': 'MC ADI 2155',
 'RTJ 191/797': 'AGR RCL 1108',
 'RTJ 183/774': 'RE 282644',
 'LEXSTF 313/291': 'RE 285047',
 'LEXSTF 314/205': 'RE 274280',
 'RTJ 177/470': 'RE 285571',
 'RJTJRS 282/84': 'EXT 747',
 'RTJ 177/969': 'RE 217194',
 'RTJ 191/271': 'RE 272872',
 'RTJ 192/222': 'HC 80405',
 'RTJ 192/78': 'ADI 2343',
 'RTJ 179/587': 'ADI 2061',
 'RTJ 177/965': 'RE 215267',
 'RTJ 191/457': 'MC ADI 2357',
 'RTJ 192/272': 'RE 265151',
 'RTJ 77/565': 'RHC 80969',
 'RTJ 183/100': 'MC ADI 1933',
 'RTJ 184/187': 'QO RE 217988',
 'RTJ 196/192': 'HC 80801',
 'LEXSTF 318/480': 'AGR RHC 80756',
 'RTJ 191/722': 'RE 297211',
 'RTJ 178/832': 'RHC 80919',
 'LEXSTF 324/344': 'HC 80240',
 'RTJ 191/399': 'ADI 909',
 'RTJ 192/131': 'MC ADI 2427',
 'RTJ 180/535': 'MC ADI 2381',
 'RTJ 184/148': 'AGR CR 9191',
 'RTJ 179/1009': 'ADI 2084',
 'RTJ 185/88': 'QO MI 635',
 'RTJ 183/327': 'RE 212198',
 'LEXSTF 340/

In [32]:
n = "RTJ-226/439"
n = " ".join(re.search("(\w+)\-?\s*(\d+\/\d+)", n).groups())
df_after[df_after["citacoes_revistas"].apply(found_acordao_id, args=(n,))]

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index,observacao_lab
43595,5e33bebdd0a3cf3007023aa2,AGR MS 30717,"PROCESSO ELETRÔNICO \r\nDJe-195 DIVULG 10-10-2011 PUBLIC 11-10-2011\r\nRTJ VOL-00226-01 PP-00439\r\nRJTJRS v. 47, n. 283, 2012, p. 48-52","[RJTJRS 283/48, RTJ 226/439]",RELATOR : MIN. RICARDO LEWANDOWSKI \nAGTE.(S) : MINISTÉRIO PÚBLICO DO ESTADO DO RIO GRANDE DO SUL \nPROC.(A/S)(ES) : PROCURADOR-GERAL DE JUSTIÇA DO ESTADO DO RIO GRANDE DO SUL \nAGDO.(A/S) : CONSELHO NACIONAL DO MINISTÉRIO PÚBLICO \nADV.(A/S) : ADVOGADO-GERAL DA UNIÃO \nINTDO.(A/S) : UNIÃO \nADV.(A/S) : ADVOGADO-GERAL DA UNIÃO,"- Acórdão citado: MS 21239. Número de páginas: 11. Análise: 10/11/2011, IMC. Revisão: 11/11/2011, KBP.",[MS 21239],4597,


In [33]:
mag = "RDC"
df_after[(df_after["observacao"].apply(lambda x: 0 if re.search("(\s+|\()" + mag + "[^\w]+", x) is None else 1) == 1) &
         (df_after["observacao"].apply(lambda x: len(re.findall("[^\w]+(" + mag + "\-?[\s\d]+\/\d+)", x))) == 0) &
         ~df_after["observacao"].apply(lambda x: True if re.search("\(" + mag + "\)", x) is not None else False)
        ]

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index,observacao_lab
9009,5e32c893d0a3cf555530cde5,AGR RE 158649,DJ 17-12-2004 PP-00066 EMENT VOL-02177-02 PP-00349,[],AGTE.(S) : RENATO DE SOUZA MARTINS \nADVDO.(A/S) : SID H. RIEDEL DE FIGUEIREDO E OUTRO (A/S) \nAGDO.(A/S) : ESTADO DO PARÁ \nADVDO.(A/S) : PGE-PA - LOANA LIA GENTIL ULIANA,"- Acórdãos citados: RMS 24458, RE 247013 AgR, RE 310388 AgR; RTJ 76/276. - Decisões monocráticas citadas: AI 303088, AI 310467, AI 338691. Número de páginas: 6. Análise: 28/01/2005, RDC.","[RMS 24458, AGR RE 247013, AGR RE 310388]",2467,
11434,5e32d37cd0a3cf555530d75e,AGR AI 482852,DJ 28-10-2004 PP-00050 EMENT VOL-02170-05 PP-00877,[],AGTE.(S) : MIRACY PIRES LUCAS E OUTRO (A/S) \nADVDO.(A/S) : MARCELISE DE MIRANDA AZEVEDO E OUTRO (A/S) \nAGDO.(A/S) : UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL \nADVDO.(A/S) : ADVOGADO-GERAL DA UNIÃO,"Número de páginas: (5). Análise: RDC. Inclusão: 24/11/2004, SVF. Alteração: 09/12/2019, (PDR).",[],5061,
11697,5e32d4e9d0a3cf555530d865,AGR AI 410501,DJ 12-11-2004 PP-00024 EMENT VOL-02172-04 PP-00710,[],AGTE. : BANCO ABN AMRO REAL S/A \nADVDOS. : CARLOS JOSÉ ELIAS JÚNIOR E OUTROS \nAGDA. : MARCIA REGINA MARQUES ANTUNES \nADVDOS. : LUIS EDUARDO RODRIGUES ALVES DIAS E OUTROS,"Número de páginas: (5). Análise: RDC. Inclusão: 06/12/2004, CFC. Alteração: 12/01/2006, AGS. Alteração: 19/12/2019, (PDR).",[],5196,


In [34]:
df_after[df_after["citacoes_revistas"].apply(lambda x: True if "JTJ" in " ".join(x) else False)]

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index,observacao_lab
525,5e3279b3d0a3cf19aa5ba458,EXT 747,"DJ 04-05-2001 PP-00033 EMENT VOL-02029-01 PP-00022\r\nRJTJRS v. 46, n. 282, 2011, p. 84-89",[RJTJRS 282/84],REQTE. : GOVERNO DA REPÚBLICA ORIENTAL DO URUGUAI \nEXTDO. : RONALD MARIO NEYRA BARREIRO \nEXTDO. : RICARDO ANACLETO RUIZ MENDIETA,"Acórdãos citados: EXT 272; EXT 524; (RTJ 134/56); HC 62438; (RTJ 115/144). Número de páginas: (11). Análise:(COF). Revisão:(AAF). Inclusão: 07/06/01, (MLR). Alteração: 15/06/07, (MLR). Alteração: 26/01/2018, CLS.","[EXT 272, EXT 524, HC 62438]",722,
8589,5e32c611d0a3cf555530cc41,QO QO ADI 807,"DJ 13-02-2004 PP-00011 EMENT VOL-02139-01 PP-00001\r\nRJTJRS v. 47, n. 284, 2012, p. 27-30",[RJTJRS 284/27],REQTE. : PROCURADOR-GERAL DA REPÚBLICA \nREQDO. : ASSEMBLÉIA LEGISLATIVA DO ESTADO DO RIO GRANDE DO SUL \nREQDO. : GOVERNADOR DO ESTADO DO RIO GRANDE DO SUL \nADV.(A/S) : PGE-RS - JOSÉ GUILHERME KLIEMANN,"Acórdãos citados: ADI 163 QO, ADI 218 (RTJ 132/1038), ADI 272 MC QO, ADI 273 MC QO, ADI 807 QO (RTJ 150/54). Número de páginas: (08). Análise:(JOY). Revisão:(RCO). Inclusão: 27/05/2004, (SVF). Alteração: 27/05/2005, (RCO). Alteração: 07/01/2019, ALS.","[QO ADI 163, ADI 218, QO MC ADI 272, QO MC ADI 273, QO ADI 807]",2100,
8873,5e32c7ecd0a3cf555530cd5d,QO AC 116,"DJ 06-02-2004 PP-00034 EMENT VOL-02138-01 PP-00016\r\nRTJ VOL-00189-03 PP-00874\r\nRJTJRS v. 41, n. 250/251, 2006, p. 31-33","[RJTJRS 250/31, RTJ 189/874]",REQTE.(S) : JOSÉ MOREIRA BENTO \nADV.(A/S) : ANTONIO CARLOS LAFOURCADE ESTRELLA \nREQDO.(A/S) : ESTADO DO RIO GRANDE DO SUL \nADV.(A/S) : PGE-RS - YASSODARA CAMOZZATO,"Acórdãos citados: AC 104, Pet 2890 (Informativo 301 do STF), Pet 2903 QO (RTJ 185/213), RE 321958, RE 375686. Número de páginas: (06). Análise:(JOY). Revisão:(RCO). Inclusão: 14/06/04, (JVC). Alteração: 14/09/05, (JOY). Alteração: 09/04/2019, HAC.","[AC 104, PET 2890, QO PET 2903, RE 321958, RE 375686]",2256,
13468,5e32e39dd0a3cf753014f40b,AGR RE 430514,"DJ 13-05-2005 PP-00019 EMENT VOL-02191-04 PP-00709\r\nRJTJRS v. 41, n. 258, 2006, p. 31-33",[RJTJRS 258/31],AGTE.(S) : INSTITUTO DE PREVIDÊNCIA DO ESTADO DO RIO \n GRANDE DO SUL - IPERGS \nADVDO.(A/S) : PGE-RS - KARINA DA SILVA BRUM E OUTRO (A/S) \nAGDO.(A/S) : NEUSA HAUSSEM \nADVDO.(A/S) : TELMO RICARDO SCHORR E OUTRO (A/S),"Acórdãos citados: ADI-2010-MC (RTJ-181/73), ADI-2189-MC (RTJ-173/786), RE-367094-AgR (RTJ-186/353), RE-372356-AgR. Número de páginas: (07). Análise:(ANA). Revisão:(). Inclusão: 17/05/05, (SVF). Alteração: 27/05/05, (AAS).","[MC ADI 2010, MC ADI 2189, AGR RE 367094, AGR RE 372356]",1187,
17909,5e330033d0a3cf7530150564,AI 527990,"DJ 05-05-2006 PP-00019 EMENT VOL-02231-07 PP-01242\r\nLEXSTF v. 28, n. 330, 2006, p. 319-324\r\nRJTJRS v. 41, n. 257, 2006, p. 31-34","[LEXSTF 330/319, RJTJRS 257/31]",AGTE.(S) : GABRIEL LIMA DA SILVA \nADV.(A/S) : DEFENSORIA PÚBLICA DA UNIÃO \nAGDO.(A/S) : MINISTÉRIO PÚBLICO DO ESTADO DO RIO GRANDE DO \n SUL,"- Acórdõas citados: HC 82959, RE 273363 (RTJ-175/1220). Número de páginas: 8. Análise: 18/05/2006, NAL.",[],5665,
23117,5e332fded0a3cf34f6749d20,RE 229096,"DJe-065 DIVULG 10-04-2008 PUBLIC 11-04-2008\r\nEMENT VOL-02314-05 PP-00985\r\nRTJ VOL-00204-02 PP-00858\r\nRJTJRS v. 45, n. 275, 2010, p. 29-42","[RJTJRS 275/29, RTJ 204/858]",RECTE.: CENTRAL RIOGRANDENSE DE AGROINSUMOS LTDA \nADVDOS.: GUSTAVO NYGAARD E OUTROS \nRECDO.: ESTADO DO RIO GRANDE DO SUL \nADVDOS.: PGE-RS - CARLOS HENRIQUE KAIPPER E OUTROS,"- Acórdãos citados: ADI 1600 (RTJ 186/855), RE 80004 (RTJ 83/809), RE 113150, RE 113701 (RTJ 125/367), RE 114950 (RTJ 125/1285). Número de páginas: 35 Análise: 11/06/2008, FMN.","[ADI 1600, RE 80004, RE 113150, RE 113701, RE 114950]",2597,
26151,5e3348cad0a3cf34f674a8fa,AGR RE 569476,"DJe-074 DIVULG 24-04-2008 PUBLIC 25-04-2008\r\nEMENT VOL-02316-09 PP-01926\r\nRTJ VOL-00205-01 PP-00468\r\nRJTJRS v. 47, n. 283, 2012, p. 45-48","[RJTJRS 283/45, RTJ 205/468]",AGTE.(S): JOÃO RIBEIRO DE J

In [35]:
txt = "Acórdãos citados: MI 211 (RTJ 157/411, JUA-1771/1918), MI 263 (RTJ 157/22), MI 274 AgR (RTJ 152/357), MS 21521 (RTJ 150/119), RE 140863, RE 161224, RE 179646, AI 221703 AgR. Número de páginas: (4). Análise:(CEL). Revisão:(FLO). Inclusão: 01/08/03, (SVF). Alteração: 05/12/03, (SVF). Alteração: 03/09/2018, JLS."

re.sub("\(((\w+)\-?\s*(\d+\/\d+),?\s*)+\)", "", txt)

'Acórdãos citados: MI 211 , MI 263 , MI 274 AgR , MS 21521 , RE 140863, RE 161224, RE 179646, AI 221703 AgR. Número de páginas: (4). Análise:(CEL). Revisão:(FLO). Inclusão: 01/08/03, (SVF). Alteração: 05/12/03, (SVF). Alteração: 03/09/2018, JLS.'

In [36]:
mag = "RTJ"
df_after[df_after["observacao"].apply(lambda x: False if re.search("(\s+|\()" + mag + "[^\w]+", x) is None else True)
        ][["observacao", "citacoesObs"]]

,observacao,citacoesObs
1,"Acórdãos citados: CC 7025 (RTJ 175/908), RE 182089. Número de páginas: (12). Análise:(MML). Revisão:(AAF). Inclusão: 12/06/03, (SVF). Alteração: 04/09/2018, HAC.","[CC 7025, RE 182089]"
3,"Acórdão citado: RE 220397 (RTJ 173/662). Número de páginas: (04). Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 22/08/03, (SVF). Alteração: 31/03/04, (MLR). Alteração: 04/04/2019, HAC.",[RE 220397]
12,"Acórdãos citados: ADC 4 (RTJ 169/383), Rcl 899, Rcl 1575, Rcl 1723, Rcl 2043, Pet 1404. Número de páginas: (09). Análise:(MML). Revisão:(AAF). Inclusão: 01/04/03, (MLR). Alteração: 02/04/03, (MLR). Alteração: 27/08/2018, HAC.","[ADC 4, RCL 899, RCL 1575, RCL 1723, RCL 2043, PET 1404]"
14,"Acórdãos citados: MI 211 (RTJ 157/411), MI 263 (RTJ 157/22), MI 274 AgR (RTJ 152/357), MS 21521 (RTJ 150/119), RE 140863, RE 161224, RE 179646, AI 221703 AgR. Número de páginas: (4). Análise:(CEL). Revisão:(FLO). Inclusão: 01/08/03, (SVF). Alteração: 05/12/03, (SVF). Alteração: 03/09/2018, JLS.","[MI 211, MI 263, AGR MI 274, MS 21521, RE 140863, RE 161224, RE 179646, AGR AI 221703]"
16,"Acórdãos citados : MS 20430, MS 20714, MD 20882, RMS 21300, (RTJ 142/782), RMS 21597, MS 22022, (RTJ 168/163), MS 22164, (RTJ 164/158), MS 22225, MS 22319, (RTJ 162/918), MS 22519, MS 23311, (RTJ 172/501, RTJ 124/948). Número de páginas: (63). Análise: (CMM). Revisão: (AAF). Inclusão: 19/06/01, (SVF). Alteração: 13/02/06, (MLR). Alteração: 17/01/2018, GIB.","[MS 20430, MS 20714, MD 20882, RMS 21300, RMS 21597, MS 22022, MS 22164, MS 22225, MS 22319, MS 22519, MS 23311]"
...,...,...
100753,"- Acórdão(s) citado(s): (DIREITO TRIBUTÁRIO, PRINCÍPIO DA ANTERIORIDADE) RE 1081041 AgR (2ªT), RE 1040084 AgR (1ªT). (CLÁUSULA DA RESERVA DE PLENÁRIO) RE 88160 (2ªT) - RTJ 96/1188, HC 70249 - RTJ 150/223, RE 91057 - RTJ 95/859. - Decisões monocráticas citadas: (DIREITO TRIBUTÁRIO, PRINCÍPIO DA ANTERIORIDADE) RE 1026463, RE 1053254, RE 1065092, RE 1131223, RE 1147498, RE 1160814, RE 1190379. Número de páginas: 10. Análise: 20/11/2019, BMP.","[AGR RE 1081041, AGR RE 1040084, RE 88160, HC 70249, RE 91057]"
100773,"- Acórdão(s) citado(s): (FUNDAMENTAÇÃO, DECISÃO JUDICIAL) RE 140370 (1ªT) - RTJ 150/269. (CONTRADITÓRIO, AMPLA DEFESA, COISA JULGADA, DEVIDO PROCESSO LEGAL) ARE 748371 RG . Número de páginas: 6. Análise: 04/12/2019, MJC.","[RE 140370, RG ARE 748371]"
100801,"- Acórdão(s) citado(s): (FUNDAMENTAÇÃO, DECISÃO JUDICIAL) RE 140370 (1ªT) - RTJ 150/269. (PREQUESTIONAMENTO, MATÉRIA CONSTITUCIONAL) AI 733063 AgR (2ªT), ARE 1072565 AgR (2ªT). (ATO ADMINISTRATIVO, CONTROLE JUDICIAL, PRINCÍPIO DA SEPARAÇÃO DOS PODERES) ARE 744080 AgR (2ªT), ARE 1163985 AgR (2ªT). Número de páginas: 8. Análise: 06/12/2019, MJC.","[RE 140370, AGR AI 733063, AGR ARE 1072565, AGR ARE 744080, AGR ARE 1163985]"
100959,"- Acórdão(s) citado(s): (CONTRADITÓRIO, AMPLA DEFESA, COISA JULGADA, DEVIDO PROCESSO LEGAL) ARE 748371 RG . (RECEITAS TRIBUTÁRIAS, PARTICIPAÇÃO DOS MUNICÍPIOS, EXCLUSÃO, REDUÇÃO, SUPRESSÃO) RE 572762 RG , RTJ 82/200, RTJ 85/712, RTJ 86/722, RTJ 89/233, RTJ 516/223. (HONORÁRIOS ADVOCATÍCIOS, MAJORAÇÃO) AO 2063 AgR (TP). Número de páginas: 13. Análise: 09/01/2020, BMP.","[RG ARE 748371, RG RE 572762, AGR AO 2063]"


In [37]:
df_after[df_after["citacoes_revistas"].str.len() > 0]

,_id,acordaoId,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index,observacao_lab
104,5e3274c1d0a3cf19aa5ba2b3,QO PET 2835,DJ 11-04-2003 PP-00038 EMENT VOL-02106-02 PP-00227\r\nRTJ VOL-00191-02 PP-00483,[RTJ 191/483],REQTE. (S) : UNIBANCO AIG SEGUROS S/A \nADV. (A/S) : MAURÍCIO PERNAMBUCO SALIN E OUTROS (A/S) \nREQDO. (A/S) : UNIÃO \nADV. (A/S) : PFN - ELYADIR FERREIRA BORGES E OUTRO (A/S),"Acórdãos citados: Pet 1162 AgR, Pet 1981 (RTJ 174/437), Pet 2772 QO. Número de páginas: (12). Análise:(PCC). Revisão:(JBM). Inclusão: 08/01/04, (SVF). Alteração: 29/03/05, (MLR). Alteração: 28/09/2018, JLS.","[AGR PET 1162, PET 1981, QO PET 2772]",66,
147,5e327564d0a3cf19aa5ba2de,QO INQ 1024,"DJ 04-03-2005 PP-00011 EMENT VOL-02182-01 PP-00049\r\n RT v. 94, n. 836, 2005, p. 442-448\r\n RTJ VOL-00193-02 PP-00459","[RT 836/442, RTJ 193/459]",AUTOR : MINISTÉRIO PÚBLICO FEDERAL \nASSIST. : SAID FELICIO FERREIRA \nDNDO. : RICARDO JOSE MAGALHAES BARROS,"Acórdãos citados: Inq 681 (RTJ 155/396), Inq 810 QO, Inq 874 AgR (RTJ 166/844), Pet 1113, Inq 1710 (RTJ 181/882), Inq 1775 AgR; RTJ 99/477, RTJ 99/487, RTJ 104/441, RTJ 112/481, RTJ 129/970, RTJ 131/1039, RTJ 133/90, RTJ 135/509, RTJ 141/406, RTJ 166/133, RTJ 167/29, RTJ 167/180, RTJ 169/969. - Veja o Informativo 291 do STF. Número de páginas: (18). Análise:(MSA). Revisão: (RCO). Inclusão: 06/06/05, (MSA). Alteração: 07/08/08, (JBM).","[INQ 681, QO INQ 810, AGR INQ 874, PET 1113, INQ 1710, AGR INQ 1775, STF DO INFORMATIVO O VEJA 291]",18,
301,5e327699d0a3cf19aa5ba378,MC ADI 2155,DJ 01-06-2001 PP-00076 EMENT VOL-02033-02 PP-00249\r\nRTJ VOL-00177-03 PP-01136,[RTJ 177/1136],REQTE. : GOVERNADOR DO ESTADO DE SÃO PAULO \nADVDA. : PGE-SP - MARCIA JUNQUEIRA S ZANOTTI \nREQDO. : GOVERNADOR DO ESTADO DO PARANÁ \nADVDA. : PGE-PR - MÁRCIA DIEGUEZ LEUZINGUER,"Acórdãos citados: ADI 84, ADI 902 MC, (RTJ 151/444), ADI 1179 MC, (RTJ 164/881), ADI 1247 MC, (RTJ 168/754), ADI 1467 MC, (RTJ 163/69), ADI 1522 MC, ADI 1587 MC, ADI 1999, ADI 2021 MC, ADI 2157 MC. Também foram citados os seguintes textos legais, todos do Dec-2736/1996 PR: item 78, do anexo I, item 6, letra b, ns. 1 e 2, item7, ns. 7, 10 e 11, item 17-A e item 22, letra a, todos da tabela I do anexo II, e item 10 da tabela II do anexo II. Número de páginas: (59). Análise:(CMM). Revisão:(AAF). Inclusão: 19/07/01, (SVF). Alteração: 06/01/05, (MLR). Alteração: 25/01/2018, ALS.","[ADI 84, MC ADI 902, MC ADI 1179, MC ADI 1247, MC ADI 1467, MC ADI 1522, MC ADI 1587, ADI 1999, MC ADI 2021, MC ADI 2157]",343,
302,5e32769ad0a3cf19aa5ba379,AGR RCL 1108,DJ 07-05-2004 PP-00020 EMENT VOL-02150-01 PP-00005\r\nRTJ VOL-00191-03 PP-00797,[RTJ 191/797],AGTES. : CIA AGRÍCOLA E INDUSTRIAL SÃO JORGE E OUTROS \nADVDOS. : CARLOS MÁRIO DA SILVA VELLOSO FILHO E OUTROS \nAGDO. : PRESIDENTE DO PRIMEIRO TRIBUNAL DE ALÇADA \n CIVIL DO ESTADO DE SÃO PAULO \nAGDO. : SUPERIOR TRIBUNAL DE JUSTIÇA,"Votação: por maioria, vencido o Min. Marco Aurélio. Resultado: acolhida a preliminar suscitada, de descabimento da reclamação, ante a preclusão. Acórdãos citados: Rcl-252 (RTJ-126/494), Rcl-509 (RTJ-174/353), AI-151063-AgR, RE-174158; RTJ-142/385. - Impedido o Min. Carlos Velloso. Número de páginas: (23). Análise:(PCC). Revisão:(JOY). Inclusão: 01/12/04, (SVF). Alteração: 04/01/05, (JVC) Alteração: 03/05/2019 , (HRS)","[RCL 252, RCL 509, AGR AI 151063, RE 174158]",342,
355,5e32775ad0a3cf19aa5ba3ae,RE 282644,DJ 20-09-2002 PP-00116 EMENT VOL-02083-05 PP-00821\r\n RTJ VOL-00183-02 PP-00774,[RTJ 183/774],RECTE. : LIBRA- ADMINISTRADORA DE CONSÓRCIOS LTDA \nADVDOS. : VILMAR JOSÉ ARRABAL DE CARVALHO E OUTROS \nRECDA. : VENDA DIRETA COMERCIAL LTDA,"Acórdãos citados: HC 72131, RE 206482, RE 331159; STJ: REsp 2320. Número de páginas: (16). Análise:(ANA). Revisão:(VAS/RCO). Inclusão: 27/11/03, (MLR). Alteração: 24/08/05, (AAS). Alteração: 04/07/2018, CLS.","[HC 72131, RE 206482, RE 331159]",283,STJ: REsp 2320
...,...,...,...,...,...,...,...,...,...
53352,5e33ee88d0a3cf5a8587227e,ED ED AG

In [38]:
def get_other_non_two_collon_chars(txt):
    search_pattern = "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
    #     Acórdão seguido eventualmente de espaços, seguido eventualmente de (s) com eventuais espaços dentro,
    #     seguido de espaços, seguido de Citado, seguido eventualmente de espaços, seguido eventualmente de (s)
    #     com eventuais espaços dentro seguido de eventuais espaços e eventualmente ':', seguido de caracteres
    #     que não sejam ':'. A última sequência (caracteres que não sejam ':') é que o que seve ser capturado.
    dec = re.search((search_pattern), txt, flags=re.IGNORECASE)
    if dec:
        return True
    return False

In [39]:
# padrões indesejados:    
# Lidar com o seguinte padrão: -Acórdãos citados: CORREÇÃO DO IMPOSTO DE RENDA:

df_after[df_after["observacao"].str.contains("-Acórdãos citados: CORREÇÃO DO IMPOSTO DE RENDA:")]

txt = "-Acórdãos citados: CORREÇÃO DO IMPOSTO DE RENDA: AI 178585 AgR, RE 195599, RE 203486. Número de páginas: 32 Análise: 15/03/2007, CEL."
search_pattern = "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
#     Acórdão seguido eventualmente de espaços, seguido eventualmente de (s) com eventuais espaços dentro,
#     seguido de espaços, seguido de Citado, seguido eventualmente de espaços, seguido eventualmente de (s)
#     com eventuais espaços dentro seguido de eventuais espaços e eventualmente ':', seguido de caracteres
#     que não sejam ':'. A última sequência (caracteres que não sejam ':') é que o que seve ser capturado.
dec = re.search((search_pattern), txt, flags=re.IGNORECASE)
df_after[df_after["observacao"].apply(lambda txt: get_other_non_two_collon_chars(txt)) &
         (df_after["citacoesObs"].str.len() == 0)][["observacao"]]
# parseAcordaosQuotes(txt, "acordaos")

,observacao
19,"Votação: unânime. Resultado: desprovimento. Acórdãos citados: RTJ-158/327, RTJ-158/934. Decisão monocrática: RE-233929, AI-269131, AI-271425, AI-272123, RE-338927, RE-360269. Número de páginas: (09). Análise:(CEL). Revisão:(RCO). Inclusão: 01/04/04, (SVF). Alteração: 25/10/2018, PDR."
69,"Acórdão citado: 201462. Número de páginas: (05). Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 22/08/03, (MLR). Alteração: 04/04/2019, HAC."
368,"Acórdão citado: (RTJ 100/1003). Número de páginas: (23). Análise:(DMV). Revisão:(FLO). Inclusão: 02/04/04, (SVF). Alteração: 14/04/04, (NT). Alteração: 13/11/2018, CLS."
397,"Acórdãos citados: RE230156. Número de páginas: (06). Análise:(MML). Revisão:(RCO/AAF). Inclusão: 29/05/01, (SVF). Alteração: 16/01/2018, ALS."
685,"Acórdãos citados : RTJ 107/661; RTJ 120/912; RTJ 125/705. Número de páginas: (05). Análise: (MML). Revisão: (COF). Inclusão: 31/07/01, (SVF). Alteração: 29/01/2018, GIB."
...,...
86276,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 6. Análise: 22/01/2019, MJC."
86304,"- Acórdão(s) citado(s): (HC, AUSÊNCIA, PERTINÊNCIA, LOCOMOÇÃO FÍSICA, DESCABIMENTO) RTJ 116./523, RTJ 141/159, RTJ 135/593, RTJ 142/896, RTJ 197/587. Número de páginas: 10. Análise: 13/12/2018, BMP."
86375,"- Acórdão(s) citado(s): (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. (RE, REEXAME DE FATO) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 8. Análise: 23/01/2019, MJC."
86672,"- Acórdão(s) citado(s): (RE, REEXAME DE FATO E PROVA) RTJ 161/992, RTJ 186/703, RTJ 152/612, RTJ 153/1019, RTJ 158/693. Número de páginas: 6. Análise: 23/01/2019, MJC."


In [40]:
# Decisões que possuem mais de uma citação real
df_after[(df_after["citacoesObs"].str.len() > 0)][["acordaoId", "observacao", "citacoesObs"]]

,acordaoId,observacao,citacoesObs
1,CC 7127,"Acórdãos citados: CC 7025 (RTJ 175/908), RE 182089. Número de páginas: (12). Análise:(MML). Revisão:(AAF). Inclusão: 12/06/03, (SVF). Alteração: 04/09/2018, HAC.","[CC 7025, RE 182089]"
3,AGR RE 199722,"Acórdão citado: RE 220397 (RTJ 173/662). Número de páginas: (04). Análise:(DMV). Revisão:(FLO/RCO). Inclusão: 22/08/03, (SVF). Alteração: 31/03/04, (MLR). Alteração: 04/04/2019, HAC.",[RE 220397]
4,AGR AI 294374,"Acórdãos citados: AI 162245 AgR, AI 181489 AgR, RE 209140, RE 249319. Número de páginas: (07). Análise:(MML). Revisão:(AAF). Inclusão: 25/03/03, (MLR). Alteração: 23/08/2018, HAC.","[AGR AI 162245, AGR AI 181489, RE 209140, RE 249319]"
5,AGR AI 401300,"Acórdão citado: AI 260787 AgR. Número de páginas: (06). Análise:(CEL). Revisão:(VAS/RCO). Inclusão: 13/08/03, (MLR). Alteração: 16/10/03, (MLR). Alteração: 23/08/2018, HAC.",[AGR AI 260787]
6,AGR AI 336011,"Acórdão citado: AI 355302 AgR. Número de páginas: (6). Análise:(ANA). Revisão:(COF/AAF). Inclusão: 31/03/03, (SVF). Alteração: 10/12/03, (SVF). Alteração: 15/03/2019, HAC.",[AGR AI 355302]
...,...,...,...
102455,AGR ARE 1154353,"- Acórdão(s) citado(s): (DANO AMBIENTAL, REEXAME, FATO, PROVA) AI 856568 AgR (2ªT), RE 629502 ED (1ªT). Número de páginas: 9. Análise: 20/01/2020, MJC.","[AGR AI 856568, ED RE 629502]"
102456,AGR ARE 1214326,"- Acórdão(s) citado(s): (UNICIDADE, PETIÇÃO, INTERPOSIÇÃO, RECURSO EXTRAORDINÁRIO, RECURSO ESPECIAL) AI 624851 AgR (2ªT). - Decisões monocráticas citadas: (UNICIDADE, PETIÇÃO, INTERPOSIÇÃO, RECURSO EXTRAORDINÁRIO, RECURSO ESPECIAL) ARE 1140555, ARE 1151861, ARE 1210111, ARE 1216337. Número de páginas: 6. Análise: 19/01/2020, MJC.",[AGR AI 624851]
102458,AGR ARE 1230956,"- Acórdão(s) citado(s): (PRESSUPOSTO DE ADMISSIBILIDADE, RECURSO, COMPETÊNCIA, TRIBUNAL DE ORIGEM, APRECIAÇÃO, MATÉRIA INFRACONSTITUCIONAL) RE 598365 RG . (CARÊNCIA, INTERESSE RECURSAL ) RE 612920 AgR (1ªT). Número de páginas: 8. Análise: 19/01/2020, MJC.","[RG RE 598365, AGR RE 612920]"
102465,AGR ED ARE 1152510,"- Acórdão(s) citado(s): (SERVIDOR PÚBLICO, PARCELAMENTO, REMUNERAÇÃO) ADI 657 (1ªT), ARE 1129152 AgR (2ªT), ARE 1130766 AgR (1ªT). (FUNDAMENTAÇÃO, DECISÃO JUDICIAL) AI 791292 QO-RG . Número de páginas: 11. Análise: 19/01/2020, MJC.","[ADI 657, AGR ARE 1129152, AGR ARE 1130766, RG QO AI 791292]"


In [41]:
# Soma do número de citações de todas as decisões
sum(df_after["citacoesObs"].apply(lambda x: len(x)))

330163

### Preparação dos datasets para comparações

In [111]:
df_after = df_after.set_index("acordaoId")
df_before = df_before.set_index("acordaoId")

df_after_i = df_after[df_after.index.isin(df_before.index)]
df_after_i.sort_index(inplace=True)
df_after_i.reset_index(inplace=True)

df_before_i = df_before[df_before.index.isin(df_after.index)]
df_before_i.sort_index(inplace=True)
df_before_i.reset_index(inplace=True)

df_before_i["citacoesObs"] = df_before_i["citacoesObs"].apply(lambda x: sorted(x))
df_after_i["citacoesObs"] = df_after_i["citacoesObs"].apply(lambda x: sorted(x))

# Cópia de decisões citadas no dataset coletada anteriormente
df_after_i["citacoesObs_before_i"] = df_before_i["citacoesObs"]

In [45]:
df_after_i.loc[:, "citacoesObs"] = df_after_i.loc[:, "observacao"].apply(lambda txt: sorted(parseAcordaosQuotes(txt, "acordaos")))

/home/jackson/.local/share/virtualenvs/analise-juridica-SICVn6Yg/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [32]:
# Creating translation of magazine citations to decision_ids
revistas_to_acordao_id_dict = {}

def translate_revistas_citacoes_to_acordao_id(row, revistas_to_acordao_id_dict): 
    for cit_mag in row["citacoes_revistas"]:
        revistas_to_acordao_id_dict[cit_mag] = row.name


df_after.apply(translate_revistas_citacoes_to_acordao_id,
                                    args=(revistas_to_acordao_id_dict,), axis=1)

acordaoId
AGR RE 308114            None
CC 7127                  None
HC 82016                 None
AGR RE 199722            None
AGR AI 294374            None
                         ... 
ED ED AGR ARE 1208076    None
ED AGR ED ARE 1217415    None
AGR HC 173987            None
ED AGR ARE 1220362       None
AGR RHC 176636           None
Length: 103168, dtype: object

## Analysis of loop citations

In [122]:
class Acordao:

    def __init__( self, idAcordao, tribunal, relator, virtual, similares=[], citacoes=[]):
        self.idAcordao = idAcordao
        self.tribunal = tribunal
        self.relator = relator
        self.citacoes = citacoes
        self.similares = similares
        self.virtual = virtual

    def getId( self):
        return self.idAcordao

    def getTribunal( self):
        return self.tribunal

    def getRelator( self):
        return self.relator

#     def getCitacoes( self):
#         return self.citacoes

#     def getSimilares( self):
#         return self.similares

    def getVirtual( self):
        return self.virtual


class GraphMaker:
    def __init__(
        self, mongo_uri, dbName, collections_in, collectionOutName,
    ):
        client = MongoClient(mongo_uri)
        self.db = client[dbName]
        self.collectionsIn = collections_in
        self.collectionOut = self.db[collectionOutName]
        self.collectionOut.drop()
        self.onePercent = sum([coll.count() for coll in self.collectionsIn]) / 100
        self.count = 0
        self.progress = 0

    def set_collections_out(self, collection_out_name):
        self.collectionOut = self.db[collection_out_name]
        self.collectionOut.drop()

    def save_removed_decisions(self, i, removed_decisions, collection_out_name):
        removed_coll = self.db[collection_out_name + "_removed_%d" % i]
        removed_coll.drop()

        removed_coll.insert_one(
            {"iteration": i, "removed_decisions": removed_decisions}
        )

    def __addElemSetToDict(self, aDict, elemKey, elemValue):
        if elemKey not in aDict:
            aDict[elemKey] = set()

        aDict[elemKey].add(elemValue)
        return aDict

    def removeInvalidAcordaosFromDicts(self, validAcordaos, quotes, quotedBy):
        """
            Remove do 'quotedBy' acórdãos que não estão presentes no BD ou nos similares apontados
            por decisões do BD. 'quotes' fica apenas com decisões citadas presentes no BD ou nos
            similares de uma determinada decisão.
        """
        for docId, quotesId in quotes.items():
            newQuotesId = set()
            for q in quotesId:
                if q in validAcordaos:
                    newQuotesId.add(q)
                else:
                    quotedBy.pop(q, 0)

            quotes[docId] = newQuotesId

        return [quotes, quotedBy]

    def buildDicts(self, query, removed_decisions, compute_similars):
        acordaos = {}
        quotes = {}
        quotedBy = {}
        similars = {}

        print("building map")

        self.count = self.progress = 0

        for coll in self.collectionsIn:
            decisions_set = list(coll.find({}, no_cursor_timeout=True))
            dec_relator_trib = {
                dec["acordaoId"]: [
                    re.sub(r"\s*\(.+", "", dec["relator"]),
                    dec["tribunal"],
                ]
                for dec in decisions_set
            }
            docsFound = coll.find(query, no_cursor_timeout=True)
            for doc in docsFound:
                if doc["acordaoId"] in removed_decisions:
                    continue

                docId = doc["acordaoId"]
                for quotedId in doc["citacoesObs"]:
                    if (quotedId in removed_decisions) or (quotedId == docId):
                        continue
                    if quotedId not in acordaos:
                        relator, tribunal = (
                            dec_relator_trib[quotedId]
                            if quotedId in dec_relator_trib
                            else ["", ""]
                        )
                        acordaos[quotedId] = Acordao(quotedId, tribunal, relator, False)

                    quotes = self.__addElemSetToDict(quotes, docId, quotedId)
                    quotedBy = self.__addElemSetToDict(quotedBy, quotedId, docId)

                # similares são decisões (nós) virtuais que apontam para citacoes de 'docId'
                if compute_similars == "S":
                    for similar in doc["similares"]:
                        similarId = similar["acordaoId"]
                        if similarId not in removed_decisions:
                            for quotedId in doc["citacoesObs"]:
                                if quotedId == similarId:
                                    continue
                                quotes = self.__addElemSetToDict(
                                    quotes, similarId, quotedId
                                )
                                quotedBy = self.__addElemSetToDict(
                                    quotedBy, quotedId, similarId
                                )

                                similars = self.__addElemSetToDict(
                                    similars, similarId, docId
                                )
                                similars = self.__addElemSetToDict(
                                    similars, docId, similarId
                                )

                            if similarId not in acordaos:
                                acordaos[similarId] = Acordao(
                                    similarId, doc["tribunal"], similar["relator"], True
                                )

                acordaos[docId] = Acordao(docId, doc["tribunal"], doc["relator"], False)
                self.__printProgress()

            print("")

        return [acordaos, quotes, quotedBy, similars]

    def insertNodes(self, acordaos, quotes, quotedBy, similars, pageRanks):
        nDocs = len(acordaos)
        self.onePercent = nDocs / 100
        self.count = self.progress = 0
        insertStep = nDocs
        if nDocs > 10000:
            insertStep = 10000

        print("n acordaos %s to be inserted" % nDocs)

        i = 0
        docs2Insert = []
        for docId, doc in acordaos.items():
            docQuotedBy = list(quotedBy.get(docId, set()))
            docQuotes = list(quotes.get(docId, set()))
            docSimilars = list(similars.get(docId, set()))
            docPageRank = float(pageRanks.get(docId, 0.0))
            docs2Insert.append(
                {
                    "acordaoId": docId,
                    "citacoes": docQuotes,
                    "citadoPor": docQuotedBy,
                    "similares": docSimilars,
                    "indegree": len(docQuotedBy),
                    "outdegree": len(docQuotes),
                    "relator": doc.getRelator(),
                    "tribunal": doc.getTribunal(),
                    "pageRank": docPageRank,
                    "virtual": doc.getVirtual(),
                }
            )
            i += 1
            self.__printProgress()
            if i >= insertStep:
                self.collectionOut.insert_one(docs2Insert)
                docs2Insert = []
                i = 0

        print("")
        if i > 0:
            self.collectionOut.insert_one(docs2Insert)

    def __printProgress(self):
        self.count += 1
        if self.count >= self.onePercent:
            self.count = 0
            self.progress += 1
            sys.stdout.write("\r%d%%" % self.progress)
            sys.stdout.flush()


In [10]:
def get_decisions_ids(collections, query):
    MONGO_URI = os.getenv("MONGO_URI")
    MONGO_DATABASE = os.getenv("MONGO_DATABASE")

    client = MongoClient(MONGO_URI)
    db = client[MONGO_DATABASE]

    decisions_ids = []
    colls = []
    if collections == "acordaos":
        colls.append(db["acordaos"])
    elif collections == "decisoes_monocraticas":
        colls.append(db["decisoes_monocraticas"])
    elif collections == "decisoes":
        colls.append(db["acordaos"])
        colls.append(db["decisoes_monocraticas"])

    for coll in colls:
        docs = coll.find(query, no_cursor_timeout=True)
        for doc in docs:
            decisions_ids.append(doc["acordaoId"])

    return decisions_ids, colls


def get_removed_decisions(decisions_ids, percentage):
    removed_decisons_len = ceil(len(decisions_ids) * (percentage / 100.0))
    decisions_ids_len = len(decisions_ids)
    removed_decisions = []
    i = 0
    while i < removed_decisons_len:
        x = randint(0, decisions_ids_len - 1)
        if decisions_ids[x] not in removed_decisions:
            removed_decisions.append(decisions_ids[x])
            i += 1

    return removed_decisions

In [125]:
# query = {}
# collections_name = "acordaos"
# collection_out_iter_name = "complete_graph_20200426"
# compute_similars = "S"

# decisions_ids, collections = get_decisions_ids(
#     collections_name, query
# )

# MONGO_URI = os.getenv("MONGO_URI")
# MONGO_DATABASE = os.getenv("MONGO_DATABASE")
# graph = GraphMaker(
#     MONGO_URI,
#     MONGO_DATABASE,
#     collections,
#     collection_out_iter_name,
# )
# removed_decisions = []
# # Construct graph
[acordaos, quotes, quotedBy, similars] = graph.buildDicts(query, removed_decisions, compute_similars)

# [quotes, quotedBy] = graph.removeInvalidAcordaosFromDicts(
#     acordaos, quotes, quotedBy
# )

building map
99%


In [ ]:
citations = set()
decisions = set()
loops = set()
for k, decs in quotes.items():
    decisions = decisions.union([k])
    for v in decs:
        citations = citations.union(["{} -> {}".format(k, v)])
        if "{} -> {}".format(v, k) in citations:
            loops = loops.union([(k, v)])

In [107]:
# before   533952
# after    525351
# after 2  525252
# after 3  780189
len(citations)

780189

In [126]:
sum([len(v) for v in quotes.values()])

780269

In [108]:
# before  2450
# after    506
# after 2  407
# after 3  327
len(loops)

327

In [109]:
loops_unnormalized = [c[0] for c in loops if c[0] == c[1]]
# before  458
# after   179
# after 2  80
# after 3  0
len(loops_unnormalized)

0

In [73]:
loops_unnormalized

['HC 81869',
 'ADI 1704',
 'RE 396266',
 'RE 354897',
 'HC 107476',
 'AGR ARE 1049532',
 'HC 81162',
 'ADI 3741',
 'EXT 1103',
 'RE 576155',
 'ED AI 588877',
 'AGR RE 585058',
 'AGR ARE 637359',
 'AGR RE 1097569',
 'MS 24268',
 'AGR EDV RE 220286',
 'MS 30822',
 'AGR AI 438772',
 'AGR AO 2016',
 'RE 272872',
 'AGR ADI 4389',
 'ADI 4167',
 'ADI 1351',
 'AGR EDV AGR RE 1079683',
 'RE 1054110',
 'MS 25181',
 'AGR RE 274115',
 'AGR ARE 804232',
 'AGR AR 1749',
 'ED AGR RE 275300',
 'AGR RCL 22612',
 'ADPF 387',
 'RE 527602',
 'ADI 2189',
 'AGR RCL 17512',
 'AGR ARE 836295',
 'RE 279872',
 'AGR ARE 808236',
 'HC 108554',
 'ADI 2724',
 'ADI 2632',
 'ADI 1976',
 'SEGUNDO AGR ARE 1166921',
 'RE 317903',
 'ADI 2364',
 'AGR RE 913486',
 'ADI 1359',
 'AGR ED RMS 26973',
 'ADI 2797',
 'ADI 990',
 'MC ADI 2356',
 'AGR INQ 3574',
 'ADI 3189',
 'RE 346084',
 'MC ADI 5104',
 'AGR RE 653736',
 'INQ 2052',
 'MS 25124',
 'RE 381367',
 'AGR RE 994739',
 'ADI 134',
 'ADI 4277',
 'AGR RE 481058',
 'AGR ACO 

In [27]:
# txt = "Acórdãos citados: RE 347825, RE 369930, RE 370092 AgR. Obs.: Os RE 383562 AgR, RE 387115 AgR foram objeto dos RE AgR ED rejeitados em 09/12/2003. Os RE 356595 AgR, RE 364219 AgR, RE 383344 AgR, RE 387240 AgR, RE 390698 AgR foram objeto dos RE AgR ED rejeitados em 16/12/2003. Os RE 367542 AgR, RE 369269 AgR, RE 369462 AgR, RE 369816 AgR, RE 376543 AgR, RE 376790 AgR, RE 377601 AgR, RE 382930 AgR foram objeto dos RE AgR ED rejeitados em 16/12/2003. Os RE 369429 AgR, RE 375032 AgR, RE 376916 AgR, RE 377147 AgR, RE 377593 AgR, RE 377636 AgR, RE 381929 AgR, RE 383033 AgR e o RE 383343-AgR foram objeto dos RE AgR ED. Os RE 386504 AgR, RE 390924 AgR, RE 390925 AgR, RE 390931 AgR, RE 394032 AgR, RE 394694 AgR foram objeto dos RE AgR ED rejeitados em 10/02/2004. Número de páginas: (06). Análise:(MSA). Revisão:(RCO). Inclusão: 10/02/04, (SVF). Alteração: 20/04/04, (SVF). Alteração: 04/12/2018, HAC."
# re.sub(r"[AO]s?[^:]+(foram|foi).+objeto.+(rejeitado|conhecido|recebido|acolhido)s[^\.]+\.", "", txt, flags=re.IGNORECASE)


# txt = "Acórdãos citados: Rcl 337 (RTJ 164/832), ADI 347 MC (RTJ 135/12), Rcl 383 (RTJ 147/404), RE 161390 MC (RTJ 150/329). - ADI 699 apensada aos autos da ADI 508. Número de páginas: (12). Análise:(DMV). Revisão:(FLO). Inclusão: 16/03/04, (MLR). Alteração: 13/09/05, (SVF). Alteração: 04/10/2018, HAC."
# re.sub(r"\..+(apensado|a).+aos.+autos[^\.]+\.", "", txt, flags=re.IGNORECASE)



' em 03/05/2006. Acórdãos citados: SS 514 AgR (RTJ 150/402), MS 23550, MS 24268, AI 120893 AgR, RE 158543 (RTJ 156/1042), RE 185255, RE 199733 (RTJ 169/1061), RE 199800 (RTJ 178/409), RE 211242, RE 213513; RTJ 183/371. Veja informativos 336 e 343 do STF. Número de páginas: (66). Análise:(JBM). Inclusão: 07/03/05, (CSM). Alteração: 16/11/05, (AAS). Alteração: 04/11/2019, (CLN).'

In [20]:
df_after[df_after["acordaoId"].isin([c[0] for c in loops if c[0] == c[1]]) #& 
#          df_after["observacao"].apply(lambda x: re.search(r"Os.+(foram|foi).+objeto.+(rejeitado|conhecido)s", x))
        ][["acordaoId", "observacao", "citacoesObs"]].head(100)

,acordaoId,observacao,citacoesObs
315,RE 279872,"Acórdãos citados : RE 125556, (RTJ 141299), RE 279872. Número de páginas: (13). Análise: (FLO). Revisão: (RCO/AAF). Inclusão: 19/11/01, (SVF). Alteração: 27/02/2018, GIB.","[RE 125556, RE 279872]"
750,RE 272872,"Acórdãos citados: ADC 3, RE 82483 (RTJ-76/960), RE 83622, RE 102553 (RTJ-120/725), RE 168277 QO (RTJ-168/1005), RE 232896 (RTJ-170/993), RE 191044, RE 214206 (RTJ-167/705), RE 272872. Número de páginas: (72). Análise:(VAS). Revisão:(RCO). Inclusão: 05/12/03, (MLR). Alteração: 10/10/05, (AAS). Alteração: 22/11/2018, PDR.","[ADC 3, QO RE 168277, RE 102553, RE 191044, RE 214206, RE 232896, RE 272872, RE 82483, RE 83622]"
2456,HC 81162,"Acórdãos citados: HC 81162, HC 12487 (STJ), RHC 11225 (STJ). Número de páginas: (09). Análise:(MML). Revisão:(RCO). Inclusão: 15/04/02, (MLR). Alteração: 16/04/2018, JRM.",[HC 81162]
2540,HC 81260,"- Acórdãos citados: HC 70290 (RTJ 162/559), HC 80032 QO, HC 80197 (RTJ 175/1068), HC 81260 (RTJ 75/311), HC 81261, HC 70272 ED EDv AgR (RTJ 157/106). - Os HC-81260 e HC-81261 foram objeto de HC-ED não conhecidos em 22/05/2002. Número de páginas: (36). Análise: (FLO). Revisão: (AAF). Inclusão: 12/07/02, (MLR). Alteração: 19/04/05, (CSM). Alteração: 26/04/2010, TBS. Alteração: 08/05/2018, PDR.","[AGR EDV ED HC 70272, HC 70290, HC 80197, HC 81260, HC 81261, QO HC 80032]"
2967,RE 317903,"Acórdãos citados: RE 161712 (RTJ 155/635), RE 317903. Número de páginas: (10). Análise:(FLO). Revisão:(RCO/AAF). Inclusão: 10/09/02, (SVF). Alteração: 24/04/2018, PDR.","[RE 161712, RE 317903]"
4015,AGR RE 304785,"Acórdãos citados: SS 665 AgR (RTJ 153/750), RE 311771 AgR. Os RE 304785 AgR, RE 305569 AgR, RE 305606 AgR, RE 307283 AgR, RE 311771 AgR, RE 314401 AgR e RE 318013 AgR foram objeto dos RE AgR ED rejeitados em 17/09/2002. Número de páginas: (08). Análise:(DMV). Revisão:(RCO/AAF). Inclusão: 13/11/02, (SVF). Alteração: 20/02/06, (MLR). Alteração: 18/06/2018, ALS.","[AGR RE 304785, AGR RE 305569, AGR RE 305606, AGR RE 307283, AGR RE 311771, AGR SS 665]"
4749,AGR RE 300743,"- Acórdãos citados: RE 311771 AgR, RE 323642 AgR. - Os RE 301322 AgR, RE 304764 AgR, RE 304774 AgR, RE 304784 AgR, RE 316309 AgR, RE 317069 AgR, RE-318016 AgR, RE 318418 AgR, RE 320406 AgR, RE 323627 AgR, RE-323844 AgR, RE 335601 AgR, RE 336566 AgR, RE 338621 AgR, RE 338948 AgR foram objeto dos RE-AgR-ED rejeitados em 12/11/2002. - Os RE 300743-AgR, RE 301077 AgR, RE 316047 AgR, RE 324877 AgR e RE 332369 AgR foram objeto dos RE-AgR-ED rejeitados em 05/11/2002. - O RE 301077-AgR foi objeto dos Embargos Declaratórios rejeitados em 05/11/2002. Número de páginas: (04). Análise:(FLO). Revisão:(RCO/AAF). Inclusão: 02/09/02, (MLR). Alteração: 08/03/05, (SVF). Alteração: 19/06/2018, JRM.","[AGR RE 300743, AGR RE 301077, AGR RE 301322, AGR RE 304764, AGR RE 304774, AGR RE 304784, AGR RE 311771, AGR RE 316047, AGR RE 316309, AGR RE 317069, AGR RE 318016, AGR RE 318418, AGR RE 320406, AGR RE 323627, AGR RE 323642, AGR RE 323844, AGR RE 335601, AGR RE 336566, AGR RE 338621]"
4805,ADI 1704,"Acórdãos citados: ADI 474 (RTJ 159/37) ADI 476 (RTJ 136/41), ADI 532 (RTJ 168/376), ADI 1592, ADI 1991 (RTJ 170/133), ADI 2101 (RTJ 173/91), ADI 2137 (RTJ 173/490), ADI 2432 MC, ADI 1704. Número de páginas: (15). Análise:(MML). Revisão:(COF/AAF). Inclusão: 10/02/03, (SVF). Alteração: 23/02/06, (MLR). Alteração: 26/06/2018, CLS.","[ADI 1592, ADI 1704, ADI 1991, ADI 2101, ADI 2137, ADI 474, ADI 476, ADI 532, MC ADI 2432]"
4903,HC 81869,"Acórdãos citados: HC 81841, HC 81869. Número de páginas: (06). Análise:(MML). Revisão:(RCO). Inclusão: 13/01/03, (MLR). Alteração: 30/01/03, (SVF). Alteração: 12/07/2018, ALS.","[HC 81841, HC 81869]"
5168,ADI 1359,"Acórdãos citados: SS 846 AgR, ADI 1045 MC (RTJ 153/116), ADI 1359, ADI 1475 (RTJ 177/616), ADI 2102 (RTJ 173/97), RE 198799 (RTJ 175/1149), RE 241494. Número de páginas: (11). Análise: (VAS). Revisão: (RCO/AAF). Inclusão: 18/03/03, (MLR). Alteração: 03/03/05, (CFC). Alteração: 04/

In [ ]:
# identificar loops que NÃO vem de similares que fazem citações provenientes de regex 'capengas'
df_after[df_after["acordaoId"].isin([c[0] for c in loops if c[0] == c[1]]) & 
         df_after["observacao"].apply(lambda x: re.search(r"Os.+foram objetos.+rejeitados", x))]
# citacoesObs

# identificar loops que vem de similares que fazem citações provenientes de regex 'capengas'
df_after[df_after["acordaoId"].isin([c[0] for c in loops if c[0] == c[1]])]


In [47]:
def parseAcordaosQuotes(txt, dec_type):
    quotes = []
    # Quando decisões do STF são prefixadas por pela string "STF:" a expressão regular abaixo não funciona.
    # Então remove-se a string sem prejuízo para a detecção das decisões citadas em txt
    txt = re.sub(r"art\.", "art", txt, flags=re.IGNORECASE)
    txt = re.sub("(STF:|\-?\s*(t|p)\w*\s*pleno|CASO\s+LÍDER)", "", txt, flags=re.IGNORECASE)
    # Verificar se é usado o padrão que remove o máximo possível de caracteres após modificar o padrão aqui
    # Em alguns espelhos o padrão que aparece no final acaba aparecendo no começo.
    # Então não é possível remover todo o texto que vem depois.
    txt = re.sub(
        r"((Número\s*de\s*p[aá]ginas|Altera[aç][aã]o|Revis[aã]o|Inclus[aã]o|An[aá]lise|Obs\.|Vota[cç][aã]o|Resultado):)[^\.]*\.", "", txt, flags=re.IGNORECASE
    )
    # Além disso, uma decisão específica começa com a string “Veja” Será feita uma exceção para este caso.    
    veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
    while re.search(veja_pattern, txt, flags=re.IGNORECASE):
        txt = re.sub(
            veja_pattern, r"\2", txt, flags=re.IGNORECASE
        )

    # ver posteriormente que tipo de informação é inserida entre '()' nas citações
    txt = re.sub(r"(STJ|TSE|TRE|TST|TRF\s*\-?[\s\w]*)\s*:.+[;\.]\s*", "", txt)

    search_pattern = (
        "[Dd]ecis(?:ão|ões)\s*monocráticas?\s*citada(?:\s*\(?s\)?)?\s*:\s*([^:]*)(?=\.[^:])"
        if dec_type == "decisoes_monocraticas"
        else "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
    )
    #     Acórdão seguido eventualmente de espaços, seguido eventualmente de (s) com eventuais espaços dentro,
    #     seguido de espaços, seguido de Citado, seguido eventualmente de espaços, seguido eventualmente de (s)
    #     com eventuais espaços dentro seguido de eventuais espaços e eventualmente ':', seguido de caracteres
    #     que não sejam ':'. A última sequência (caracteres que não sejam ':') é que o que seve ser capturado.
    dec = re.search((search_pattern), txt, flags=re.IGNORECASE)

    if dec:
        dec = dec.group(1)
        dec = re.sub(r"[Dd]ecis(?:ão|ões)\s*monocráticas?\s*citada\s*\(?\s*s?\s*\)?\s*", "", dec, flags=re.IGNORECASE)
        if (len(dec) > 2) and (dec[-2] == "."):
            dec = dec[:-2]

        dec = re.sub(r"\..+(apensado|a).+aos.+autos[^\.]+\.", "", dec, flags=re.IGNORECASE)
        dec = re.sub(r"\.[^\.]+[AO]s?[^:]+(foram|foi).+objeto.+(rejeitado|conhecido|recebido|acolhido)s[^\.]+\.", "",
                     dec, flags=re.IGNORECASE)
        
        dec = re.sub(r"(\d+)\.(\d+)", r"\1\2", dec)
        dec = re.sub(r"[,;]?[\w\d\s\.\/\-]+\((STJ|TSE|TST|TRE|TRF\s*\-?[\s\w]*)\)", "", dec)
        # remover menções a revistas entre parênteses porque isso significa que o próprio
        # id do acórdão citado já é referenciado
        dec = re.sub("\(([A-Z]+\-?[\s\d]+\/\d+\-?\d*,?\s*)+\)", ",", dec)
        dec = re.sub("\-\s*[A-Z]+\-?[\s\d]+\/\d+\-?\d*", ",", dec)
        # manter citação à decisão e não à revista como no exemplo: 'TJ 169/557 (HC 73801)'
        dec = re.sub(r"(\w+\-?[\s\d]+\/\d+\-?\d*)\s+\(([^\d]{2,}[\s-]+\d+[^\d\)]*)\)", r"\2,", dec)

        # REMOVER CONTEÚDO ENTRE PARÊNTESES AQUI
        while re.search(r"\([^\(\)]+\)", dec):
            dec = re.sub(r"\([^\(\)]+\)", ",", dec)
        
        # VER O EFEITO DO SPLIT NO PARÊNTESES PARA ENTENDER QUE PADRÃO É EXTRAÍDO          
        dec = re.split("[;,.()]", dec)
        for q in dec:
            # SIGNIFICA QUE ACABOU O PADRÃO DE DECISÕES
            if re.search(r"embargos?\sde|(de|em)\sembargos?|RISTF|ESTRANGEIRA|(\-\s*STJ|\s+STJ|STJ\s+\-|TRF)", q, flags=re.IGNORECASE):
                return quotes
            q = q.strip()

            acordaoId_pattern = "([^\d\s]{2,}[\s-]+\d+[^\d\/]*)$"
            m = re.search(acordaoId_pattern, q)
            if m is None:
                n = re.search("([A-Z]+)\-?\s*(\d+\/\d+)", q)
                if (n is not None) and (not q.startswith("-")):
                    n = " ".join(n.groups())
                    if n in revistas_to_acordao_id_dict:
                        quotes.append(revistas_to_acordao_id_dict[n])
                    else:
                        quotes.append(n)

            while m:
                m = m.group()
                q = q.replace(m, "")
                m = m.replace("-", " ")
                m = m.strip().upper()
                m = " ".join(m.split())
                m = normalizeId(m)
                quotes.append(m)
                m = re.search(acordaoId_pattern, q)
    return sorted(set(quotes))

# https://pt.stackoverflow.com/questions/13598/significado-de-em-uma-regex

In [338]:
# Criar metodologia comparativa
# Ver forma de descobrir bons padrões de aprimoramento da extração

# Decisão SS 697-9 não está sendo capturada

In [122]:
# txt = "- Acórdão(s) citado(s): (COMPETÊNCIA, JUSTIÇA DO TRABALHO) AI 670715 AgR-ED (2ªT), RE 716896 AgR (2ªT). Número de páginas: 7. Análise: 11/07/2013, AUR."
txt = "- Acórdão citado: RE 92264. Número de páginas: 6. Análise: 11/12/2009, SEV."
print(txt)
txt = txt.replace("STF:", "")
print(txt)
txt = re.sub(
    r"(Número de páginas)?(Alteração)?(Revisão)?(Inclusão)?(Análise)?", "", txt
)
print(txt)
txt = re.sub(r"(STJ):[^;\.]+", "", txt)
print(txt)
txt = re.sub(r"(TSE):[^;\.]+", "", txt)
print(txt)
search_pattern = "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*:\s*(\.(?!\s)*|[^:]*)?"
re.search((search_pattern), txt).group(1)

- Acórdão citado: RE 92264. Número de páginas: 6. Análise: 11/12/2009, SEV.
- Acórdão citado: RE 92264. Número de páginas: 6. Análise: 11/12/2009, SEV.
- Acórdão citado: RE 92264. : 6. : 11/12/2009, SEV.
- Acórdão citado: RE 92264. : 6. : 11/12/2009, SEV.
- Acórdão citado: RE 92264. : 6. : 11/12/2009, SEV.


'HC 46187 RTJ 114/408 (104722).'

### Benchmark da melhora da extração de citações

In [48]:
# comparar com última extração para checar se está tudo certo
df_after_i.loc[:, "citacoesObs_"] = df_after_i.loc[:, "NEWcitacoesObs"] if "NEWcitacoesObs" in df_after_i.columns else df_after_i.loc[:, "citacoesObs"]
df_after_i.loc[:, "NEWcitacoesObs"] = df_after_i.loc[:, "observacao"].progress_apply(lambda txt:
                                                                            sorted(parseAcordaosQuotes(txt, "acordaos"))
                                                                           )
print("Antes:", sum(df_after_i["citacoesObs"].apply(lambda x: len(x))),
      "\nDepois anterior:", sum(df_after_i["citacoesObs_"].apply(lambda x: len(x))),
      "\nDepois:", sum(df_after_i["NEWcitacoesObs"].apply(lambda x: len(x)))
      )


Antes: 341395 
Depois anterior: 338857 
Depois: 340163


In [51]:
df_after_i[df_after_i.progress_apply(lambda x: len(x["NEWcitacoesObs"]) != len(x["citacoesObs_"]),
                            axis=1
                           )][["observacao", "citacoesObs_", "NEWcitacoesObs"]].head(60)
# df_after_i[df_after_i.progress_apply(lambda x: len(x["NEWcitacoesObs"]) != len(x["citacoesObs_"]),
#                             axis=1
#                            ) &
#            ~(~df_after_i["NEWcitacoesObs"].progress_apply(lambda x: True if re.search(r"S\wMULA|VINCULANTE|SUV|INFORMATIVO|STF|DE\s+|RTJ", ",".join(x)) else False) &
#            df_after_i["citacoesObs_"].progress_apply(lambda x: True if re.search(r"S\wMULA|VINCULANTE|SUV|INFORMATIVO|STF|DE\s+|RTJ", ",".join(x)) else False))
#           ][["observacao", "citacoesObs_", "NEWcitacoesObs"]]

,observacao,citacoesObs,NEWcitacoesObs
43,"- Acórdão(s) citado(s): (AÇÃO CAUTELAR, JULGAMENTO, AÇÃO PRINCIPAL, PERDA DO OBJETO) AC 2008 AgR (1ªT), AC 1947 MC-REF (TP), AC 2910 MC-AgR (TP). - Veja RMS 34203 e MS 27008 do STF. Número de páginas: 8. Análise: 17/07/2018, JRS.","[AGR AC 2008, AGR MC AC 2910, REF MC AC 1947, RMS VEJA 34203, STF DO MS E 27008]","[AGR AC 2008, AGR MC AC 2910, REF MC AC 1947]"
65,"- Acórdão(s) citado(s): (COMPETÊNCIA ORIGINÁRIA, STF, RISCO, PACTO FEDERATIVO) ACO 2101 AgR (TP). - Veja AC 3136 MC do STF. Número de páginas: 17. Análise: 30/05/2017, AMA.","[AGR ACO 2101, STF DO MC AC VEJA 3136]",[AGR ACO 2101]
79,"- Acórdão(s) citado(s): (POSSE, ÁREA INDÍGENA) RE 44585 (TP). (TERRA DEVOLUTA, ÁREA INDÍGENA, EXTINÇÃO, ALDEAMENTO INDÍGENA) ADI 255 (TP). (DIREITO INDÍGENA, TERRA INDÍGENA) Pet 3388 (TP). (POSSE PERMANENTE, COMUNIDADE INDÍGENA, TERRA DEVOLUTA) ACO 469 AgR (2ªT). (NULIDADE, TÍTULO, PROPRIEDADE, TERRA INDÍGENA) ACO 312 (TP), ACO 323 (TP). (CONFLITO FEDERATIVO, CONFIGURAÇÃO) ACO 644 AgR (TP). - Veja ACO 79, ACO 362 e ACO 366 do STF. - Veja Carta do Estado do Mato Grosso e regiões circunvizinhas, publicada em 1952 sob a coordenação do marechal Cândido Rondon. - Veja Alvará de 1º de abril de 1680 e Lei Pombalina de 1755. Número de páginas: 99. Análise: 06/03/2018, JRS.","[ACO 312, ACO 323, ACO 362, ACO VEJA 79, ADI 255, AGR ACO 469, AGR ACO 644, DE ABRIL DE º 1680, DE ALVARÁ VEJA 1, DE POMBALINA LEI E 1755, PET 3388, RE 44585, RONDON CÂNDIDO MARECHAL DO COORDENAÇÃO A SOB EM PUBLICADA 1952, STF DO ACO E 366]","[ACO 312, ACO 323, ADI 255, AGR ACO 469, AGR ACO 644, PET 3388, RE 44585]"
103,"- Acórdão(s) citado(s): (DEVIDO PROCESSO LEGAL, CONTRADITÓRIO, AMPLA DEFESA, INSCRIÇÃO, ENTE FEDERADO, CADASTRO DE INADIMPLENTES) AC 259 MC (TP), AC 2032 QO (TP). - Veja RE 607420. Número de páginas: 12. Análise: 06/07/2017, KBP.","[MC AC 259, QO AC 2032, RE VEJA 607420]","[MC AC 259, QO AC 2032]"
144,"- Acórdão(s) citado(s): (EMENDA PARLAMENTAR) Rp 611 (TP) - RTJ 33/107, RMS 15015 (TP) - RTJ 36/382, RMS 9315 (3ªT) - RTJ 37/113, RE 55718 (TP) - RTJ 32/143, RMS 14405 (TP) - RTJ 34/6, RE 57713 (TP) - RTJ 40/348, RTJ 36/385, RDA 102/261, RDA 97/213, RF 165/155. (ATUAÇÃO, ADVOGADO-GERAL DA UNIÃO, AÇÃO DE CONTROLE CONCENTRADO DE CONSTITUCIONALIDADE) ADI 3916 (TP), ADI 2681 MC (TP), ADI 97 QO (TP) - RTJ 131/470, ADI 1254 AgR (TP) - RTJ 170/801, ADI 72 QO (TP) - RTJ 131/958, ADI 1616 (TP), ADI 2101 (TP). (AGU, PARECER FAVORÁVEL, PROCEDÊNCIA, ADI) ADI 341 (TP), ADI 1440 (TP), ADI 4190 MC-REF (TP) - RTJ 213/436. (PROCESSO LEGISLATIVO, SIMETRIA) ADI 1254 MC (TP) - RTJ 170/792. (PROCESSO LEGISLATIVO, INICIATIVA PRIVATIVA, PODER JUDICIÁRIO) ADI 89 (TP) - RTJ 150/341, ADI 872 (TP) - RTJ 185/408, ADI 980 (TP) - RTJ 156/777, ADI 1935 (TP), ADI 2569 (TP), ADI 2731 (TP), ADI 2867 (TP), ADI 3773 (TP), ADI 1060 MC (TP), ADI 822 MC (TP) - RTJ 150/482, ADI 1064 MC (TP) - RTJ 156/788, ADI 2079 MC (TP) - RTJ 174/75, ADI 1070 MC (TP) - RTJ 168/87, ADI 348 (TP) - RTJ 155/22, ADI 700 - RTJ 180/91, ADI 2192 MC (TP), ADI 2400 MC (TP), ADI 1730 MC (TP), ADI 1729 MC (TP), ADI 2115 MC (TP), ADI 2336 MC (TP), ADI 2417 MC (TP), ADI 1391 MC (TP) - RTJ 178/621. (PODER DE EMENDA PARLAMENTAR, PERTINÊNCIA TEMÁTICA, PROPOSIÇÃO LEGISLATIVA) Rp 1009 (TP) - RTJ 93/500, Rp 1043 (TP) - RTJ 102/908, Rp 1046 (TP) - RTJ 97/986, Rp 1062 (TP) - RTJ 100/41, ADI 574 (TP), ADI 1333 (TP), ADI 1682 (TP), ADI 1834 (TP), ADI 2350 (TP), ADI 3655 (TP), ADI 865 MC (TP), ADI 973 MC (TP). (CONVALIDAÇÃO, VÍCIO DE INCONSTITUCIONALIDADE, SANÇÃO DO CHEFE DO PODER EXECUTIVO) Rp 890 (TP) - RTJ 69/625, Rp 1051 (TP) - RTJ 103/36, ADI 2867 (TP), ADI 2840 ED (TP), RE 78781 (1ªT) - RTJ 72/226, ADI 2192 MC (TP), ADI 1070 MC (TP) - RTJ 168/87, ADI 2079 MC (TP) - RTJ 174/75, ADI 700 (TP) - RTJ 180/91. Número de páginas: 28. Análise: 28/11/2018, JSF.","[ADI 1333, ADI 1440, ADI 1616, ADI 1682, ADI 1834, ADI 1935, ADI 2101, ADI 2350, ADI 2569, ADI 2731, ADI 2867, ADI 2867, ADI 341, ADI 348, AD

In [46]:
txt = "Acórdãos citados: ADI 301 (RTJ 186/7), ADI 304 MC (RTJ 132/1078), ADI 305 MC (RTJ 153/428), ADI 463 MC (RTJ 137/559), ADI 465 EI (RTJ 179/848), ADI 546 (RTJ 173/710), ADI 700 (RTJ 180/91), ADI 774 (RTJ 171/397), ADI 805 (RTJ 168/391), ADI 821, ADI 955 MC (RTJ 186/399), ADI 1070 MC (RTJ 168/87), ADI 1274, ADI 2079 MC (RTJ 174/75), ADI 2186 MC, ADI 2192 MC, ADI 2646, ADI 2863, ADI 2888. A ADI 2840 QO foi objeto dos embargos de declaração rejeitados em 17/11/2005. Número de páginas: (30). Análise:(PCC/JOY/JBM). Revisão:(). Inclusão: 02/02/05, (SVF). Alteração: 02/01/06, (LMS). Alteração: 20/08/2019, (CLN)."
dec_type = "acordaos"

quotes = []
# Quando decisões do STF são prefixadas por pela string "STF:" a expressão regular abaixo não funciona.
# Então remove-se a string sem prejuízo para a detecção das decisões citadas em txt
txt = re.sub(r"art\.", "art", txt, flags=re.IGNORECASE)
txt = re.sub("(STF:|\-?\s*(t|p)\w*\s*pleno|CASO\s+LÍDER)", "", txt, flags=re.IGNORECASE)
# Verificar se é usado o padrão que remove o máximo possível de caracteres após modificar o padrão aqui
# Em alguns espelhos o padrão que aparece no final acaba aparecendo no começo.
# Então não é possível remover todo o texto que vem depois.
txt = re.sub(
    r"((Número\s*de\s*p[aá]ginas|Altera[aç][aã]o|Revis[aã]o|Inclus[aã]o|An[aá]lise|Obs\.|Vota[cç][aã]o|Resultado):)[^\.]*\.", "", txt, flags=re.IGNORECASE
)
# Além disso, uma decisão específica começa com a string “Veja” Será feita uma exceção para este caso.    
veja_pattern = r"(^Veja[^\.]*\.)?(.*)\-?\s*Veja.*"
while re.search(veja_pattern, txt, flags=re.IGNORECASE):
    txt = re.sub(
        veja_pattern, r"\2", txt, flags=re.IGNORECASE
    )

# ver posteriormente que tipo de informação é inserida entre '()' nas citações
txt = re.sub(r"(STJ|TSE|TRE|TST|TRF\s*\-?[\s\w]*)\s*:.+[;\.]\s*", "", txt)

search_pattern = (
    "[Dd]ecis(?:ão|ões)\s*monocráticas?\s*citada(?:\s*\(?s\)?)?\s*:\s*([^:]*)(?=\.[^:])"
    if dec_type == "decisoes_monocraticas"
    else "[Aa]c[óo]rd[ãa]o\s*\(?\s*s?\s*\)?\s+[Cc]itado\s*\(?\s*s?\s*\)?\s*[:;,]\s*(\.(?!\s)*|[^:]*)?"
)
#     Acórdão seguido eventualmente de espaços, seguido eventualmente de (s) com eventuais espaços dentro,
#     seguido de espaços, seguido de Citado, seguido eventualmente de espaços, seguido eventualmente de (s)
#     com eventuais espaços dentro seguido de eventuais espaços e eventualmente ':', seguido de caracteres
#     que não sejam ':'. A última sequência (caracteres que não sejam ':') é que o que seve ser capturado.
dec = re.search((search_pattern), txt, flags=re.IGNORECASE)

if dec:
    dec = dec.group(1)
    dec = re.sub(r"[Dd]ecis(?:ão|ões)\s*monocráticas?\s*citada\s*\(?\s*s?\s*\)?\s*", "", dec, flags=re.IGNORECASE)
    if (len(dec) > 2) and (dec[-2] == "."):
        dec = dec[:-2]
    dec = re.sub(r"\..+(apensado|a).+aos.+autos[^\.]+\.", "", dec, flags=re.IGNORECASE)
    dec = re.sub(r"\.[^\.]+[AO]s?[^:]+(foram|foi).+objeto.+(rejeitado|conhecido|recebido|acolhido)s[^\.]+\.", "",
                 dec, flags=re.IGNORECASE)
dec

'ADI 301 (RTJ 186/7), ADI 304 MC (RTJ 132/1078), ADI 305 MC (RTJ 153/428), ADI 463 MC (RTJ 137/559), ADI 465 EI (RTJ 179/848), ADI 546 (RTJ 173/710), ADI 700 (RTJ 180/91), ADI 774 (RTJ 171/397), ADI 805 (RTJ 168/391), ADI 821, ADI 955 MC (RTJ 186/399), ADI 1070 MC (RTJ 168/87), ADI 1274, ADI 2079 MC (RTJ 174/75), ADI 2186 MC, ADI 2192 MC, ADI 2646, ADI 2863, ADI 2888      '

In [341]:
# df_after_i.loc[:, "citacoesObs_"] = df_after_i.apply(lambda x: [c for c in x["citacoesObs"] if "VEJA" not in c], axis=1)
# df_after_i.loc[df_after_i["citacoesObs_"].str.len() != df_after_i["NEWcitacoesObs"].str.len(),
#                ["citacoesObs_", "NEWcitacoesObs", "observacao"]
#               ]

txt = "- Acórdão(s) citado(s): (HC, REEXAME, FATO, PROVA) HC 65887 , HC 66381 , HC 68610 (1ªT) - RTJ 136/1221, HC 69780 (1ªT) - RTJ 195/486, HC 70763 (1ªT) - RTJ 165/877, HC 72496 (1ªT) - RTJ 186/237, HC 72992 (1ªT) - RTJ 168/863, HC 73449 (1ªT) - RTJ 163/650, HC 96820 (1ªT), RHC 99388 (2ªT), HC 100067 (1ªT), HC 112507 (2ªT), RHC 131136 AgR (2ªT), HC 60555 (1ªT) - RTJ 110/555, HC 67349 (1ªT) - RTJ 129/1199, RHC 63806 (1ªT) - RTJ 137/198. (FUNDAMENTAÇÃO, DECISÃO JUDICIAL) RE 140370 (1ªT) - RTJ 150/269, AI 152586 AgR , RE 327143 AgR (2ªT), AI 529105 AgR (2ªT), AI 637301 AgR (1ªT), AI 731527 AgR (2ªT), AI 791292 QO-RG , AI 838209 AgR (2ªT), AI 842316 AgR (1ªT), AI 840788 AgR (1ªT), RTJ 170/627. (HC, SUPRESSÃO DE INSTÂNCIA) HC 73390 (2ªT), HC 79551 (1ªT), HC 80747 (1ªT) - RTJ 182/243, HC 81115 (1ªT), HC 83842 (2ªT) - RTJ 192/233, HC 97761 (2ªT). (ALEGAÇÃO, NULIDADE, PRECLUSÃO) HC 53508 (1ªT) - RTJ 76/435, HC 72454 (1ªT), HC 74339 (1ªT), HC 58994 (1ªT) - RTJ 104/540, RT 451/490. (HC, SUCEDÂNEO, REVISÃO CRIMINAL) HC 70457 , HC 98206 (1ªT), HC 98412 (2ªT), RHC 116674 (2ªT), HC 122753 AgR (2ªT), RHC 130270 (1ªT), RHC 128723 AgR (1ªT), RHC 133200 (2ªT), HC 134974 (2ªT), HC 123182 AgR (1ªT), HC 134976 AgR (1ªT), HC 137059 AgR (2ªT). - Acórdão(s) citado(s) - outros tribunais: STJ: HC 130146, RHC 68917 EDcl-AgRg. Número de páginas: 20. Análise: 11/09/2019, JRS."
parseAcordaosQuotes(txt, "acordaos")
# df_after_i.loc[df_after_i["citacoesObs"].str.len() != df_after_i["NEWcitacoesObs"].str.len(),
#                ["citacoesObs_", "NEWcitacoesObs", "observacao"]
#               ]

['HC 65887',
 'HC 66381',
 'HC 68610',
 'HC 69780',
 'HC 70763',
 'HC 72496',
 'HC 72992',
 'HC 73449',
 'HC 96820',
 'RHC 99388',
 'HC 100067',
 'HC 112507',
 'AGR RHC 131136',
 'HC 60555',
 'HC 67349',
 'RHC 63806',
 'RE 140370',
 'AGR AI 152586',
 'AGR RE 327143',
 'AGR AI 529105',
 'AGR AI 637301',
 'AGR AI 731527',
 'RG QO AI 791292',
 'AGR AI 838209',
 'AGR AI 842316',
 'AGR AI 840788',
 'HC 73390',
 'HC 79551',
 'HC 80747',
 'HC 81115',
 'HC 83842',
 'HC 97761',
 'HC 53508',
 'HC 72454',
 'HC 74339',
 'HC 58994',
 'HC 70457',
 'HC 98206',
 'HC 98412',
 'RHC 116674',
 'AGR HC 122753',
 'RHC 130270',
 'AGR RHC 128723',
 'RHC 133200',
 'HC 134974',
 'AGR HC 123182',
 'AGR HC 134976',
 'AGR HC 137059']

In [188]:
# Número de acórdãos com MENOS extrações que coleta anterior ANTES
df_after_i[df_after_i.apply(lambda x: len(x["citacoesObs"]) < len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["citacoesObs", "citacoesObs_before_i", "observacao"]]

,citacoesObs,citacoesObs_before_i,observacao
129,"[ADI 139, ADI 182, ADI 234, ADI 430, ADI 864, MC ADI 766]","[ADI 139, ADI 151, ADI 182, ADI 234, ADI 430, ADI 766, ADI 864]","Acórdãos citados: ADI 766 MC (157/460), ADI 864, ADI 151/416), ADI 139 (RTJ 138/14), ADI 182 (RTJ 133/1037), ADI 234 (RTJ 167/363), ADI 430 (RTJ 159/735). Número de páginas: (09). Análise:(MML). Revisão:(CTM/AAF). Inclusão: 07/11/02, (MLR). Alteração: 11/11/02, (MLR). Alteração: 28/05/2018, ALS."
167,"[ADI 425, ADI 805, MC ADI 1070, MC ADI 2079, MC ADI 2417]","[ADI 1070, ADI 2079, ADI 2417, ADI 425, ADI 805, RJT 168]","Acórdãos citados: ADI 425, ADI 805 (RTJ 168/391), ADI 1070 MC (RJT 168/87), ADI 2079 MC (RTJ 174/75), ADI 2417 MC. Veja: Informativo do STF 339. Número de páginas: (11). Análise:(JOY). Revisão:(RCO). Inclusão: 02/12/04, (CFC). Alteração: 07/12/04, (JVC). Alteração 21/02/2019 VRL"
222,"[ADI 1571, HC 77002, HC 81611]","[ADI 1571, HC 77002, HC 81611, RT 796]","Votação: por maioria, vencidos os Mins. Carlos Britto e Ellen Gracie. Resultado: julgada improcedente a Ação e firmada a constitucionalidade do artigo 83, ""caput"", da Lei-9430/1996. Acórdãos citados: ADI-1571 (RTJ-167/53), HC-77002, HC-81611; RTJ-167/53), RT-796/492. Veja: Informativos do STF-286, 326 e 333. Número de páginas: (22). Análise:(PCC). Revisão:(JOY). Inclusão: 25/01/05, (MLR). Alteração: 01/02/05, (MLR)."
250,"[ADI 1523, ADI 1775, AGR ADI 203, MC ADI 2137, MC ADI 647, MC ADI 709, MC ADI 842]","[ADI 1523, ADI 1775, ADI 203, ADI 2137, ADI 528, ADI 647, ADI 709, ADI 842]","Acórdãos citados: ADI 203 AgR (RTJ 131/1001), ADI 647 MC (RTJ 140/36), ADI 709 MC (RTJ 154/401), ADI 842 MC (RTJ 147/545), ADI 1523, ADI 1775 (RTJ 177/669), ADI 2137 MC (RTJ 173/490). Decisão monocrática citada: ADI 528. Veja: informativo do STF 338. Número de páginas: (10). Análise:(JOY). Revisão:(RCO). Inclusão: 07/10/04, (SVF). Alteração: 09/10/04, (JVC), Alteração: 18/02/2019, RAM."
370,[],"[ADI 1727, ADI 2093, ADI 2107, ADI 2195, ADI 740, AO 679, AO 707, AO 724]","Número de páginas: (08). Análise:(PCC). Revisão:(RCO). Inclusão: 18/11/04, (MLR). Alteração: 29/11/04, (NT). Alteração: 19/11/2019, (PDR)."
...,...,...,...
73668,[],"[SE 5778, SEC 4694]","Número de páginas: (9). Análise:(PCC). Inclusão: 09/05/05, (PCC). Alteração: 10/01/06, (SVF). Alteração: 10/12/2019, (PDR)."
73671,"[INFORMATIVO 292, SEC 4694, SEC 6729, SEC 6971]","[SE 3495, SE 3534, SE 4248, SE 4307, SEC 2912, SEC 5378, SEC 5418, SEC 6122]","Acórdãos citados: SEC 4694, SEC 6729, SEC 6971 (Informativo-292). Número de páginas: (10). Análise:(JOY). Revisão:(RCO). Inclusão: 06/07/04, (MLR). Alteração: 07/07/04, (NT). Alteração: 16/04/2019, HAC."
73675,[],"[SE 2582, SE 3495, SE 4248, SE 4605, SEC 6304, SEC 6729]","Votação: unânime. Resultado: indeferida Número de páginas: (07). Análise:(MSA). Revisão:(RCO). Alteração: 19/05/05, (SVF). Alteração: 30/04/2019, NSB."
73680,[],"[SEC 6304, SEC 6729, SEC 7218]","Votação: unânime. Resultado: homologada a sentença estrangeira. - Veja Informativo 370 do STF. Número de páginas: (05). Análise:(RDC). Revisão:(ANA). Inclusão: 04/02/05, (SVF). Alteração: 29/11/05, (CSV)."


In [189]:
# Número de acórdãos com MENOS extrações que coleta anterior DEPOIS
df_after_i[df_after_i.apply(lambda x: len(x["NEWcitacoesObs"]) < len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["NEWcitacoesObs", "citacoesObs_before_i", "observacao"]]

,NEWcitacoesObs,citacoesObs_before_i,observacao
129,"[ADI 139, ADI 182, ADI 234, ADI 430, ADI 864, MC ADI 766]","[ADI 139, ADI 151, ADI 182, ADI 234, ADI 430, ADI 766, ADI 864]","Acórdãos citados: ADI 766 MC (157/460), ADI 864, ADI 151/416), ADI 139 (RTJ 138/14), ADI 182 (RTJ 133/1037), ADI 234 (RTJ 167/363), ADI 430 (RTJ 159/735). Número de páginas: (09). Análise:(MML). Revisão:(CTM/AAF). Inclusão: 07/11/02, (MLR). Alteração: 11/11/02, (MLR). Alteração: 28/05/2018, ALS."
167,"[ADI 425, ADI 805, MC ADI 1070, MC ADI 2079, MC ADI 2417]","[ADI 1070, ADI 2079, ADI 2417, ADI 425, ADI 805, RJT 168]","Acórdãos citados: ADI 425, ADI 805 (RTJ 168/391), ADI 1070 MC (RJT 168/87), ADI 2079 MC (RTJ 174/75), ADI 2417 MC. Veja: Informativo do STF 339. Número de páginas: (11). Análise:(JOY). Revisão:(RCO). Inclusão: 02/12/04, (CFC). Alteração: 07/12/04, (JVC). Alteração 21/02/2019 VRL"
222,"[ADI 1571, HC 77002, HC 81611]","[ADI 1571, HC 77002, HC 81611, RT 796]","Votação: por maioria, vencidos os Mins. Carlos Britto e Ellen Gracie. Resultado: julgada improcedente a Ação e firmada a constitucionalidade do artigo 83, ""caput"", da Lei-9430/1996. Acórdãos citados: ADI-1571 (RTJ-167/53), HC-77002, HC-81611; RTJ-167/53), RT-796/492. Veja: Informativos do STF-286, 326 e 333. Número de páginas: (22). Análise:(PCC). Revisão:(JOY). Inclusão: 25/01/05, (MLR). Alteração: 01/02/05, (MLR)."
250,"[ADI 1523, ADI 1775, AGR ADI 203, MC ADI 2137, MC ADI 647, MC ADI 709, MC ADI 842]","[ADI 1523, ADI 1775, ADI 203, ADI 2137, ADI 528, ADI 647, ADI 709, ADI 842]","Acórdãos citados: ADI 203 AgR (RTJ 131/1001), ADI 647 MC (RTJ 140/36), ADI 709 MC (RTJ 154/401), ADI 842 MC (RTJ 147/545), ADI 1523, ADI 1775 (RTJ 177/669), ADI 2137 MC (RTJ 173/490). Decisão monocrática citada: ADI 528. Veja: informativo do STF 338. Número de páginas: (10). Análise:(JOY). Revisão:(RCO). Inclusão: 07/10/04, (SVF). Alteração: 09/10/04, (JVC), Alteração: 18/02/2019, RAM."
370,[],"[ADI 1727, ADI 2093, ADI 2107, ADI 2195, ADI 740, AO 679, AO 707, AO 724]","Número de páginas: (08). Análise:(PCC). Revisão:(RCO). Inclusão: 18/11/04, (MLR). Alteração: 29/11/04, (NT). Alteração: 19/11/2019, (PDR)."
...,...,...,...
73668,[],"[SE 5778, SEC 4694]","Número de páginas: (9). Análise:(PCC). Inclusão: 09/05/05, (PCC). Alteração: 10/01/06, (SVF). Alteração: 10/12/2019, (PDR)."
73671,"[SEC 4694, SEC 6729, SEC 6971]","[SE 3495, SE 3534, SE 4248, SE 4307, SEC 2912, SEC 5378, SEC 5418, SEC 6122]","Acórdãos citados: SEC 4694, SEC 6729, SEC 6971 (Informativo-292). Número de páginas: (10). Análise:(JOY). Revisão:(RCO). Inclusão: 06/07/04, (MLR). Alteração: 07/07/04, (NT). Alteração: 16/04/2019, HAC."
73675,[],"[SE 2582, SE 3495, SE 4248, SE 4605, SEC 6304, SEC 6729]","Votação: unânime. Resultado: indeferida Número de páginas: (07). Análise:(MSA). Revisão:(RCO). Alteração: 19/05/05, (SVF). Alteração: 30/04/2019, NSB."
73680,[],"[SEC 6304, SEC 6729, SEC 7218]","Votação: unânime. Resultado: homologada a sentença estrangeira. - Veja Informativo 370 do STF. Número de páginas: (05). Análise:(RDC). Revisão:(ANA). Inclusão: 04/02/05, (SVF). Alteração: 29/11/05, (CSV)."


In [190]:
# Número de acórdãos com MESMAS extrações que coleta anterior ANTES
df_after_i[df_after_i.apply(lambda x: len(x["citacoesObs"]) == len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["citacoesObs", "citacoesObs_before_i", "observacao"]]

,citacoesObs,citacoesObs_before_i,observacao
1,[],[],"Número de páginas: 7. Análise: 11/07/2013, AUR."
4,[],[],"Número de páginas: 5. Análise: 26/05/2016, IMC."
8,[],[],"Número de páginas: 11. Análise: 21/05/2015, AMA."
13,[],[],"Número de páginas: 5. Análise: 11/12/2009, MLM."
18,[],[],"Número de páginas: 6. Análise: 10/05/2016, AMS."
...,...,...,...
74358,[],[],"Número de páginas: 7. Análise: 25/03/2014, TIA."
74367,[],[],"Número de páginas: 5. Análise: 12/01/2017, MJC."
74372,[],[],"Número de páginas: 17. Análise: 08/02/2011, SEV. Revisão: 10/02/2011, KBP."
74373,[],[],"- Legislação estrangeira citada: art. 118º, 1, ""b"", art. 121, I, ""a"", art. 375ª, 1, no art. 205ª, 1, 4, ""a"", e 5, do Código Penal Português e art. 11ª, nº1, alínea ""a"", do Decreto-Lei Português nº 316 de 1997, art. 202 do Código de Processo Penal Português. Número de páginas: 16. Análise: 27/02/2013, IMC. Revisão: 01/03/2013, MMR."


In [193]:
# Número de acórdãos com MESMAS extrações que coleta anterior DEPOIS
df_after_i[df_after_i.apply(lambda x: len(x["NEWcitacoesObs"]) == len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["NEWcitacoesObs", "citacoesObs_before_i", "observacao"]]

,NEWcitacoesObs,citacoesObs_before_i,observacao
1,[],[],"Número de páginas: 7. Análise: 11/07/2013, AUR."
4,[],[],"Número de páginas: 5. Análise: 26/05/2016, IMC."
8,[],[],"Número de páginas: 11. Análise: 21/05/2015, AMA."
13,[],[],"Número de páginas: 5. Análise: 11/12/2009, MLM."
18,[],[],"Número de páginas: 6. Análise: 10/05/2016, AMS."
...,...,...,...
74358,[],[],"Número de páginas: 7. Análise: 25/03/2014, TIA."
74367,[],[],"Número de páginas: 5. Análise: 12/01/2017, MJC."
74372,[],[],"Número de páginas: 17. Análise: 08/02/2011, SEV. Revisão: 10/02/2011, KBP."
74373,[],[],"- Legislação estrangeira citada: art. 118º, 1, ""b"", art. 121, I, ""a"", art. 375ª, 1, no art. 205ª, 1, 4, ""a"", e 5, do Código Penal Português e art. 11ª, nº1, alínea ""a"", do Decreto-Lei Português nº 316 de 1997, art. 202 do Código de Processo Penal Português. Número de páginas: 16. Análise: 27/02/2013, IMC. Revisão: 01/03/2013, MMR."


In [320]:
# Número de acórdãos com MAIS extrações que coleta anterior ANTES
df_after_i[df_after_i.apply(lambda x: len(x["citacoesObs"]) > len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["acordaoId", "citacoesObs", "citacoesObs_before_i", "observacao"]]

,acordaoId,citacoesObs,citacoesObs_before_i,observacao
0,2ºJULG AGR AI 699063,"[AGR RE 716896, ED AGR AI 670715]",[],"- Acórdão(s) citado(s): (COMPETÊNCIA, JUSTIÇA DO TRABALHO) AI 670715 AgR-ED (2ªT), RE 716896 AgR (2ªT). Número de páginas: 7. Análise: 11/07/2013, AUR."
2,2ºJULG AGR ARE 774137,"[AGR AI 733063, AGR AI 776070, AGR AI 820065, AGR ARE 647186, AGR ARE 649046, AGR ARE 661070, AGR RE 735918]",[],"- Acórdão(s) citado(s): (CONCURSO PÚBLICO, PRETERIÇÃO DE CANDIDATO) AI 776070 AgR (2ªT), ARE 661070 AgR (2ªT), AI 820065 AgR (1ªT), ARE 649046 AgR (1ªT), RE 735918 AgR (2ªT). (PREQUESTIONAMENTO) AI 733063 AgR (2ªT), ARE 647186 AgR (1ªT). Número de páginas: 12. Análise: 04/11/2014, MAR."
3,2ºJULG AGR HC 88759,"[AI 430526, HC 69599, HC 73556, HC 81134]",[],"- Acórdãos citados: HC 69599, HC 73556, HC 81134, AI 430526. Número de páginas: 10. Análise: 01/09/2010, MMR. Revisão: 06/09/2010, ACG."
5,2ºJULG ED ED ED AGR AI 489856,"[ED ED AGR AI 586710, ED HC 80566]",[],"- Acórdãos citados: HC 80566 ED (RTJ 180/974), AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
6,2ºJULG ED ED ED AGR AI 531494,"[AGR AGR ED RE 281287, AGR RE 475421, ED ED AGR AI 586710]",[],"- Acórdãos citados: RE 281287 ED-AgR-AgR, RE 475421 AgR, AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
...,...,...,...,...
74375,TERCEIROS DÉCIMOS EI AP 470,"[AP 396, AP 481, EXTN HC 101118, HC 101118, HC 109596, HC 115098, HC 63765, HC 70391, HC 70774, HC 72992, HC 72992, HC 73120, HC 96590, RC 1453, RE 89063, RHC 115213]",[],"- Acórdão(s) citado(s): (PENA-BASE, CRIME, QUADRILHA, COMPARAÇÃO, PENA-BASE, CRIME, CORRELAÇÃO, CORRUPÇÃO POLÍTICA) AP 396 (TP), AP 481 (TP). (PRESCRIÇÃO DA PRETENSÃO PUNITIVA, QUESTÃO PREJUDICIAL, MÉRITO, AÇÃO PENAL) HC 73120 (1ªT), HC 115098 (1ªT), HC 63765 (2ªT), RC 1453 (2ªT). (PROFUNDIDADE, EFEITO DEVOLUTIVO, EMBARGOS INFRINGENTES) RE 89063. (CRITÉRIO, FIXAÇÃO, PENA) HC 101118 (2ªT). (STF, EXAME, DOSIMETRIA DA PENA, DIVERSIDADE, TRIBUNAL) HC 70391 (2ªT), HC 72992 (1ªT), HC 109596 (2ªT), RHC 115213 (1ªT). (SEVERIDADE, SANÇÃO, GRAVIDADE, FATO) HC 70774 (1ªT). (REQUISITO, MOTIVAÇÃO, DECISÃO JUDICIAL, EXACERBAÇÃO, PENA-BASE) HC 96590 (2ªT), HC 101118 Extn (2ªT), RTJ 143/600. (REQUISITOS, CRIME, QUADRILHA) HC 72992 (1ªT), RTJ 102/614, RT 565/406, RT 582/348, RT 600/383, RT 580/328, RT 588/323, RT 615/272. (AUTONOMIA, CRIME DE QUADRILHA) RTJ 88/468. (QUADRILHA, CONCURSO MATERIAL, CRIMES) RTJ 104/104, RTJ 128/325, RT 505/352. - Acórdão(s) citado(s) - outros tribunais: STJ: AP 514, AP 549, REsp 1198829. TRF4: Apelação Criminal 94.04.05616-2. TJRJ: RT 745/628. RSTJ 110/354. - Veja AP 470, AP 470 EI, AP EI-décimos-quartos, AP 470 EI-sétimos, AP 470 EI-nonos, AP 470 EI-décimos-primeiros, AP 470 EI-quintos, AP 470 EI-décimos do STF. Número de páginas: 117. Análise: 24/09/2014, IVA. Revisão: 06/10/2014, JOS."
74376,TERCEIROS ED RE 463624,"[AGR ED AI 709440, AGR RE 601914, AI 733387, ED AGR AI 153147, ED AGR AI 776295, ED AGR AI 825520, ED AGR ARE 679031, ED RE 177599, HC 54513, HC 69438, HC 69987, QO AGR RE 227089]",[],"- Acórdão(s) citado(s): (ED, EFEITOS INFRINGENTES) RE 177599 ED, AI 153147 AgR-ED, RTJ 191/694-695, RTJ 134/1296, RTJ 134/836. (DEVIDO PROCESSO LEGAL) AI 733387 (2ªT), AI 709440 ED-AgR (1ªT), RE 227089 AgR-QO (TP), RE 601914 AgR (2ªT), AI 776295 AgR-ED (1ªT), ARE 679031 AgR-ED (2ªT), RTJ 137/1053, RTJ 158/272. (MOTIVAÇÃO “PER RELATIONEM”) HC 69438 (1ªT), HC 69987 (TP). (MOTIVAÇÃO POR REFERÊNCIA OU POR REMISSÃO) HC 54513, AI 825520 AgR-ED (2ªT). (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 13. Análise: 26/03/2015, MAD."
74377,TERCEIROS ED SEGUNDO AGR CC 7706,"[AGR CC 7699, AGR CC 7730, AGR RCL 11250, AGR RCL 13508, QO CC 7094, RCL 10793, RE 586453]",[],"- Acórdão(s) citado(s): (PREVIDÊNCIA PRIVADA, COMPLEMENTAÇÃO DE APOSENTADORIA, COMPETÊNCIA) RE 586453 (TP). (CONFLITO DE COMPETÊNCIA, NECESSIDADE, MANIFESTAÇÃO, JUÍZO) CC 7094 QO (1ªT), CC 7699 AgR (TP). (RECLAMAÇÃO, AGRAVO, DECISÃO, APLI

In [191]:
# Número de acórdãos com MAIS extrações que coleta anterior DEPOIS
df_after_i[df_after_i.apply(lambda x: len(x["NEWcitacoesObs"]) > len(x["citacoesObs_before_i"]),
                            axis=1
                           )][["NEWcitacoesObs", "citacoesObs_before_i", "observacao"]]

,NEWcitacoesObs,citacoesObs_before_i,observacao
0,"[AGR RE 716896, ED AGR AI 670715]",[],"- Acórdão(s) citado(s): (COMPETÊNCIA, JUSTIÇA DO TRABALHO) AI 670715 AgR-ED (2ªT), RE 716896 AgR (2ªT). Número de páginas: 7. Análise: 11/07/2013, AUR."
2,"[AGR AI 733063, AGR AI 776070, AGR AI 820065, AGR ARE 647186, AGR ARE 649046, AGR ARE 661070, AGR RE 735918]",[],"- Acórdão(s) citado(s): (CONCURSO PÚBLICO, PRETERIÇÃO DE CANDIDATO) AI 776070 AgR (2ªT), ARE 661070 AgR (2ªT), AI 820065 AgR (1ªT), ARE 649046 AgR (1ªT), RE 735918 AgR (2ªT). (PREQUESTIONAMENTO) AI 733063 AgR (2ªT), ARE 647186 AgR (1ªT). Número de páginas: 12. Análise: 04/11/2014, MAR."
3,"[AI 430526, HC 69599, HC 73556, HC 81134]",[],"- Acórdãos citados: HC 69599, HC 73556, HC 81134, AI 430526. Número de páginas: 10. Análise: 01/09/2010, MMR. Revisão: 06/09/2010, ACG."
5,"[ED ED AGR AI 586710, ED HC 80566]",[],"- Acórdãos citados: HC 80566 ED (RTJ 180/974), AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
6,"[AGR AGR ED RE 281287, AGR RE 475421, ED ED AGR AI 586710]",[],"- Acórdãos citados: RE 281287 ED-AgR-AgR, RE 475421 AgR, AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
...,...,...,...
74375,"[AP 396, AP 481, EXTN HC 101118, HC 101118, HC 109596, HC 115098, HC 63765, HC 70391, HC 70774, HC 72992, HC 72992, HC 73120, HC 96590, RC 1453, RE 89063, RHC 115213]",[],"- Acórdão(s) citado(s): (PENA-BASE, CRIME, QUADRILHA, COMPARAÇÃO, PENA-BASE, CRIME, CORRELAÇÃO, CORRUPÇÃO POLÍTICA) AP 396 (TP), AP 481 (TP). (PRESCRIÇÃO DA PRETENSÃO PUNITIVA, QUESTÃO PREJUDICIAL, MÉRITO, AÇÃO PENAL) HC 73120 (1ªT), HC 115098 (1ªT), HC 63765 (2ªT), RC 1453 (2ªT). (PROFUNDIDADE, EFEITO DEVOLUTIVO, EMBARGOS INFRINGENTES) RE 89063. (CRITÉRIO, FIXAÇÃO, PENA) HC 101118 (2ªT). (STF, EXAME, DOSIMETRIA DA PENA, DIVERSIDADE, TRIBUNAL) HC 70391 (2ªT), HC 72992 (1ªT), HC 109596 (2ªT), RHC 115213 (1ªT). (SEVERIDADE, SANÇÃO, GRAVIDADE, FATO) HC 70774 (1ªT). (REQUISITO, MOTIVAÇÃO, DECISÃO JUDICIAL, EXACERBAÇÃO, PENA-BASE) HC 96590 (2ªT), HC 101118 Extn (2ªT), RTJ 143/600. (REQUISITOS, CRIME, QUADRILHA) HC 72992 (1ªT), RTJ 102/614, RT 565/406, RT 582/348, RT 600/383, RT 580/328, RT 588/323, RT 615/272. (AUTONOMIA, CRIME DE QUADRILHA) RTJ 88/468. (QUADRILHA, CONCURSO MATERIAL, CRIMES) RTJ 104/104, RTJ 128/325, RT 505/352. - Acórdão(s) citado(s) - outros tribunais: STJ: AP 514, AP 549, REsp 1198829. TRF4: Apelação Criminal 94.04.05616-2. TJRJ: RT 745/628. RSTJ 110/354. - Veja AP 470, AP 470 EI, AP EI-décimos-quartos, AP 470 EI-sétimos, AP 470 EI-nonos, AP 470 EI-décimos-primeiros, AP 470 EI-quintos, AP 470 EI-décimos do STF. Número de páginas: 117. Análise: 24/09/2014, IVA. Revisão: 06/10/2014, JOS."
74376,"[AGR ED AI 709440, AGR RE 601914, AI 733387, ED AGR AI 153147, ED AGR AI 776295, ED AGR AI 825520, ED AGR ARE 679031, ED RE 177599, HC 54513, HC 69438, HC 69987, QO AGR RE 227089]",[],"- Acórdão(s) citado(s): (ED, EFEITOS INFRINGENTES) RE 177599 ED, AI 153147 AgR-ED, RTJ 191/694-695, RTJ 134/1296, RTJ 134/836. (DEVIDO PROCESSO LEGAL) AI 733387 (2ªT), AI 709440 ED-AgR (1ªT), RE 227089 AgR-QO (TP), RE 601914 AgR (2ªT), AI 776295 AgR-ED (1ªT), ARE 679031 AgR-ED (2ªT), RTJ 137/1053, RTJ 158/272. (MOTIVAÇÃO “PER RELATIONEM”) HC 69438 (1ªT), HC 69987 (TP). (MOTIVAÇÃO POR REFERÊNCIA OU POR REMISSÃO) HC 54513, AI 825520 AgR-ED (2ªT). (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 13. Análise: 26/03/2015, MAD."
74377,"[AGR CC 7699, AGR CC 7730, AGR RCL 11250, AGR RCL 13508, QO CC 7094, RCL 10793, RE 586453]",[],"- Acórdão(s) citado(s): (PREVIDÊNCIA PRIVADA, COMPLEMENTAÇÃO DE APOSENTADORIA, COMPETÊNCIA) RE 586453 (TP). (CONFLITO DE COMPETÊNCIA, NECESSIDADE, MANIFESTAÇÃO, JUÍZO) CC 7094 QO (1ªT), CC 7699 AgR (TP). (RECLAMAÇÃO, AGRAVO, DECISÃO, APLICAÇÃO, REPERCUSSÃO GERAL) Rcl 10793 (TP), Rcl 11250 AgR (TP), Rcl 13508 AgR (TP). (CONFLITO DE COMPETÊNCIA, SUCEDÂNEO, RECURSO) CC 7730 AgR (TP). - Decisão monocrática citada: (CONFLITO DE COMPETÊNCIA) CC 7123 MC. - Ac

In [194]:
# Número de acórdãos com MAIS extrações que análise original
df_after_i[(df_after_i["citacoesObs"].str.len() != 0) &
           (df_after_i["citacoesObs"].str.len() < df_after_i["NEWcitacoesObs"].str.len())
          ][["citacoesObs", "NEWcitacoesObs", "observacao"]]

,citacoesObs,NEWcitacoesObs,observacao


In [321]:
# Número de acórdãos com MENOS extrações que análise original
df_after_i[(df_after_i["citacoesObs"].str.len() != 0) &
           (df_after_i["citacoesObs"].str.len() > df_after_i["NEWcitacoesObs"].str.len()) &
           ~(~df_after_i["NEWcitacoesObs"].progress_apply(lambda x: True if re.search(r"S\wMULA|VINCULANTE|SUV|INFORMATIVO|STF|DE\s+|RTJ", ",".join(x)) else False) &
           df_after_i["citacoesObs"].progress_apply(lambda x: True if re.search(r"S\wMULA|VINCULANTE|SUV|INFORMATIVO|STF|DE\s+|RTJ", ",".join(x)) else False))
          ][["acordaoId", "citacoesObs", "NEWcitacoesObs", "observacao"]]

,acordaoId,citacoesObs,NEWcitacoesObs,observacao
315,ADI 1919,"[AGR ADI 203, RMS STJ 11824]",[AGR ADI 203],"Acórdãos citados: ADI 203 AgR (RTJ 131/1001); STJ RMS 11824. Número de páginas: (22). Análise:(MML). Revisão:(). Inclusão: 28/11/03, (MLR). Alteração: 09/12/03, (MLR). Alteração: 24/10/2018, HAC."
758,ADI 3106,"[CITADA ESTRANGEIRA LEGISLAÇÃO MC ADI 2667, MC ADI 2024]",[MC ADI 2024],"- Acórdãos citados: ADI 2024 MC - Tribunal Pleno, ADI 2667 MC - Legislação estrangeira citada: art. 10 da Constituição de Weimar. Número de páginas: 70. Análise: 01/10/2010, MMR. Revisão: 14/10/2010, ACG."
766,ADI 3128,"[8 º N O V I T A M R O F N I , ADC 1, ADC 3, ADI 1141, ADI 1430, ADI 159, ADI 2110, ADI 2111, ADI 2777, ADI 2874, ADI 790, ADI 939, AGR RE 250590, AGR SS 775, AGR SS 819, EDV ED RE 72509, MC ADI 1303, MC ADI 1402, MC ADI 1430, MC ADI 1441, MC ADI 2010, MC ADI 2087, MC ADI 223, MC ADI 939, MS 1447, RE 105137, RE 105137, RE 105322, RE 116683, RE 146733, RE 158577, RE 163094, RE 163204, RE 163989, RE 166772, RE 199753, RE 225602, RE 226855, RE 298694, RE 325822, RE 70009, RE 77131, RE 92232, RE 94020, RE 99955, RP 1451]","[ADC 1, ADC 3, ADI 1141, ADI 1430, ADI 159, ADI 2110, ADI 2111, ADI 2777, ADI 2874, ADI 790, ADI 939, AGR RE 250590, AGR SS 775, AGR SS 819, EDV ED RE 72509, MC ADI 1303, MC ADI 1402, MC ADI 1430, MC ADI 1441, MC ADI 2010, MC ADI 2087, MC ADI 223, MC ADI 939, MS 1447, RE 105137, RE 105137, RE 105322, RE 116683, RE 146733, RE 158577, RE 163094, RE 163204, RE 163989, RE 166772, RE 199753, RE 225602, RE 226855, RE 298694, RE 325822, RE 70009, RE 77131, RE 92232, RE 94020, RE 99955, RP 1451]","- Acórdãos citados: ADC 1 (RTJ 156/721), ADC 3 (RTJ 187/3), ADI 159 (RTJ 147/376), ADI 223 MC (RTJ 132/571), SS 775 AgR (Informativo nº-8), ADI 790 (RTJ 147/921), SS 819 AgR, ADI 939 (RTJ 151/755), ADI 939 MC, ADI 1141, ADI 1303 MC (RTJ 174/743), ADI 1402 MC, ADI 1430, ADI 1430 MC (RTJ 164/98), ADI 1441 MC (RTJ 166/890), MS 1447, Rp 1451 (RTJ 127/789), ADI 2010 MC (RTJ 181/73), ADI 2087 MC (RTJ 189/68), ADI 2110, ADI 2111, ADI 2777, ADI 2874 (RTJ 189/200), RE 70009 (RTJ 83/74), RE 72509 ED-Edv, RE 77131 (RTJ 72/155), RE 92232 (RTJ 97/842),RE 94020 (RTJ 104/269), RE 99955 (RTJ 116/1065), RE 105137 (RTJ 15/379), RE 105137 (RTJ 119/783), RE 105322 (RTJ 118/709), RE 116683 (RTJ 137/398), RE 146733 (RTJ 143/684), RE 158577 (RTJ 149/654), RE 163094 (RTJ 148/932), RE 163204 (RTJ 166/267), RE 163989 (RTJ 151/646), RE 166772 (RTJ 156/666), RE 199753, RE 225602 (RTJ 178/428), RE 226855 (RTJ 174/916), RE 250590 AgR, RE 298694, RE 325822 (RTJ 190/333); RTJ 134/1112, RTJ 143/724, RTJ 156/451, RTJ 166/267, RTJ 166/893, RTJ 178/428, RTJ 181/73. -VEJA Informativos 349 e 357 do STF; Estatuto do Servidor Público de 1952; Exposição de motivos da Proposta de Emenda n° 41/03, itens 64, 65, 66, 67, 68, 69, 70, 71; art. 28 da Declaração dos Direitos do Homem e do Cidadão; Acórdão n° 173/2001 do Tribunal Constitucional Português; Acórdão n° 39/84 do Tribunal Constitucional português; ""Acórdãos do Tribunal Constitucional"", vol. 3/95-131, 117-118, 1984, Imprensa Nacional, Lisboa. - Legislação estrangeira citada: art. 2° da Lei Constitucional Francesa de 14/8/1884. Número de páginas: 327. Análise: 15/05/2009, MMR. Revisão: 27/05/2009, JBM. Alteração: 20/08/2009, TBS."
1028,ADI 395,[RESP STJ 431760],[],"Acórdão citado: STJ - REsp 431760. Número de páginas: 20 Análise: 05/09/2007, ACL. Revisão: 28/09/2007, RCO."
2606,AGR AI 263218,[STJ DO AGR 127337],[],"Acórdão citado: AGR 127337 do STJ. Número de páginas: (05). Análise:(FLO). Revisão:(RCO/AAF). Inclusão: 18/09/01, (MLR). Alteração: 26/02/2018, JRM."
...,...,...,...,...
70540,RE 305186,"[RE 149466, RE 158430, RESP STJ 89015]","[RE 149466, RE 158430]","Acórdãos citados: RE 149466 (RTJ 147/1021), RE 158430; STJ-RESP 89015. Número de páginas: (9). Análise: (VAS). Revisão: (RCO/AAF). Inclusão: 28/05/03, (SVF). Alteração: 19/11/03, (SVF). Alteração: 25/07/2018, JLS."
70647,RE 338840,"[EXÉRCITO DO DISCIPLINA

In [198]:
# Número de acórdãos com MESMAS extrações que análise original
df_after_i[(df_after_i["citacoesObs"].str.len() != 0) &
           (df_after_i["citacoesObs"].str.len() == df_after_i["NEWcitacoesObs"].str.len())
          ][["citacoesObs", "NEWcitacoesObs", "observacao"]]

,citacoesObs,NEWcitacoesObs,observacao
0,"[AGR RE 716896, ED AGR AI 670715]","[AGR RE 716896, ED AGR AI 670715]","- Acórdão(s) citado(s): (COMPETÊNCIA, JUSTIÇA DO TRABALHO) AI 670715 AgR-ED (2ªT), RE 716896 AgR (2ªT). Número de páginas: 7. Análise: 11/07/2013, AUR."
2,"[AGR AI 733063, AGR AI 776070, AGR AI 820065, AGR ARE 647186, AGR ARE 649046, AGR ARE 661070, AGR RE 735918]","[AGR AI 733063, AGR AI 776070, AGR AI 820065, AGR ARE 647186, AGR ARE 649046, AGR ARE 661070, AGR RE 735918]","- Acórdão(s) citado(s): (CONCURSO PÚBLICO, PRETERIÇÃO DE CANDIDATO) AI 776070 AgR (2ªT), ARE 661070 AgR (2ªT), AI 820065 AgR (1ªT), ARE 649046 AgR (1ªT), RE 735918 AgR (2ªT). (PREQUESTIONAMENTO) AI 733063 AgR (2ªT), ARE 647186 AgR (1ªT). Número de páginas: 12. Análise: 04/11/2014, MAR."
3,"[AI 430526, HC 69599, HC 73556, HC 81134]","[AI 430526, HC 69599, HC 73556, HC 81134]","- Acórdãos citados: HC 69599, HC 73556, HC 81134, AI 430526. Número de páginas: 10. Análise: 01/09/2010, MMR. Revisão: 06/09/2010, ACG."
5,"[ED ED AGR AI 586710, ED HC 80566]","[ED ED AGR AI 586710, ED HC 80566]","- Acórdãos citados: HC 80566 ED (RTJ 180/974), AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
6,"[AGR AGR ED RE 281287, AGR RE 475421, ED ED AGR AI 586710]","[AGR AGR ED RE 281287, AGR RE 475421, ED ED AGR AI 586710]","- Acórdãos citados: RE 281287 ED-AgR-AgR, RE 475421 AgR, AI 586710 AgR-ED-ED. Número de páginas: 6. Análise: 26/06/2008, SOF."
...,...,...,...
74376,"[AGR ED AI 709440, AGR RE 601914, AI 733387, ED AGR AI 153147, ED AGR AI 776295, ED AGR AI 825520, ED AGR ARE 679031, ED RE 177599, HC 54513, HC 69438, HC 69987, QO AGR RE 227089]","[AGR ED AI 709440, AGR RE 601914, AI 733387, ED AGR AI 153147, ED AGR AI 776295, ED AGR AI 825520, ED AGR ARE 679031, ED RE 177599, HC 54513, HC 69438, HC 69987, QO AGR RE 227089]","- Acórdão(s) citado(s): (ED, EFEITOS INFRINGENTES) RE 177599 ED, AI 153147 AgR-ED, RTJ 191/694-695, RTJ 134/1296, RTJ 134/836. (DEVIDO PROCESSO LEGAL) AI 733387 (2ªT), AI 709440 ED-AgR (1ªT), RE 227089 AgR-QO (TP), RE 601914 AgR (2ªT), AI 776295 AgR-ED (1ªT), ARE 679031 AgR-ED (2ªT), RTJ 137/1053, RTJ 158/272. (MOTIVAÇÃO “PER RELATIONEM”) HC 69438 (1ªT), HC 69987 (TP). (MOTIVAÇÃO POR REFERÊNCIA OU POR REMISSÃO) HC 54513, AI 825520 AgR-ED (2ªT). (OFENSA REFLEXA) RTJ 120/912, RTJ 132/455. Número de páginas: 13. Análise: 26/03/2015, MAD."
74377,"[AGR CC 7699, AGR CC 7730, AGR RCL 11250, AGR RCL 13508, QO CC 7094, RCL 10793, RE 586453]","[AGR CC 7699, AGR CC 7730, AGR RCL 11250, AGR RCL 13508, QO CC 7094, RCL 10793, RE 586453]","- Acórdão(s) citado(s): (PREVIDÊNCIA PRIVADA, COMPLEMENTAÇÃO DE APOSENTADORIA, COMPETÊNCIA) RE 586453 (TP). (CONFLITO DE COMPETÊNCIA, NECESSIDADE, MANIFESTAÇÃO, JUÍZO) CC 7094 QO (1ªT), CC 7699 AgR (TP). (RECLAMAÇÃO, AGRAVO, DECISÃO, APLICAÇÃO, REPERCUSSÃO GERAL) Rcl 10793 (TP), Rcl 11250 AgR (TP), Rcl 13508 AgR (TP). (CONFLITO DE COMPETÊNCIA, SUCEDÂNEO, RECURSO) CC 7730 AgR (TP). - Decisão monocrática citada: (CONFLITO DE COMPETÊNCIA) CC 7123 MC. - Acórdão(s) citado(s) - outros tribunais: STJ: CC 110996, CC 48106, REsp 1.413.016, CC 129.502 AgRg, CC 128.051 AgRg. - Veja AI 834551 do STF, Resp 961407 do STJ e RR 114500-77.2005.5.02.0049 do TST. Número de páginas: 56. Análise: 28/05/2015, IMC."
74378,"[AGR AI 734689, AGR AI 738982, AGR AI 813692, AGR ARE 657355, AGR ED AGR RE 406432, AGR RE 585932, ED AGR AI 153147, ED AGR AI 825520, ED EXT 662, ED INQ 1070, ED INQ 2424, ED PETA AP 552, ED RE 177599, ED RE 592905, ED RMS 27920, HC 54513, RE 172292, RE 223904]","[AGR AI 734689, AGR AI 738982, AGR AI 813692, AGR ARE 657355, AGR ED AGR RE 406432, AGR RE 585932, ED AGR AI 153147, ED AGR AI 825520, ED EXT 662, ED INQ 1070, ED INQ 2424, ED PETA AP 552, ED RE 177599, ED RE 592905, ED RMS 27920, HC 54513, RE 172292, RE 223904]","- Acórdão(s) citado(s): (REVISÃO, CANCELAMENTO, NOTA TAQUIGRÁFICA) RE 406432 AgR-ED-AgR (2ªT), RMS 27920 ED (2ªT), RE 592905 ED (TP), Inq 2424 ED (TP), AP 552 PetA-ED (TP). (CONDIÇÃO, 

In [319]:
### Checar ids estranhos
set().union(*df_after_i["NEWcitacoesObs"].apply(lambda ids: set(re.sub("\s+\d+.*$", "", id) for id in ids)).values)

{'2',
 '7',
 '8',
 '9',
 'AC',
 'AC AC ED AGR RE',
 'ACI',
 'ACO',
 'ACORA',
 'AD',
 'ADC',
 'ADCMC',
 'ADCQO',
 'ADI',
 'ADI ESTA A APENSADAS FORAM ADI',
 'ADIM',
 'ADIMC',
 'ADIN',
 'ADINMC',
 'ADIQO',
 'ADMIC',
 'ADO',
 'ADPF',
 'AG',
 'AGAED',
 'AGAEDE',
 'AGCRA',
 'AGCRAE DOS OBJETIVO FOI AGCRA',
 'AGCRQO',
 'AGED',
 'AGQO',
 'AGR',
 'AGR A AI',
 'AGR AC',
 'AGR ACO',
 'AGR ADC',
 'AGR ADI',
 'AGR ADPF',
 'AGR AG',
 'AGR AG AR',
 'AGR AGR ADI',
 'AGR AGR AGR ADI',
 'AGR AGR AGR AGR AGR AI',
 'AGR AGR AGR AGR AI',
 'AGR AGR AGR AI',
 'AGR AGR AGR RCL',
 'AGR AGR AGR RE',
 'AGR AGR AGR SS',
 'AGR AGR AI',
 'AGR AGR AO',
 'AGR AGR AR',
 'AGR AGR ARE',
 'AGR AGR CR',
 'AGR AGR ED AGR ARE',
 'AGR AGR ED AGR RE',
 'AGR AGR ED AI',
 'AGR AGR ED MS',
 'AGR AGR ED RE',
 'AGR AGR EDV ED ED ED AGR AI',
 'AGR AGR EDV ED ED RE',
 'AGR AGR EDV RE',
 'AGR AGR HC',
 'AGR AGR MC MS',
 'AGR AGR MC RCL',
 'AGR AGR MC RMS',
 'AGR AGR MI',
 'AGR AGR MS',
 'AGR AGR PET',
 'AGR AGR RCL',
 'AGR AGR RE',


In [315]:
df_after_i[df_after_i["NEWcitacoesObs"].apply(lambda x: True if re.search('EM PROFERIDO', ",".join(x)) else False)][["observacao", "NEWcitacoesObs"]]

,observacao,NEWcitacoesObs
41356,"-Acórdãos citados: O AI 395285 AgR e o RE 439515 AgR são ""no sentido de ser considerado extemporâneo o recurso extraordinário protocolizado antes da publicação do acórdão proferido em embargos infringentes, sem posterior ratificação"". Número de páginas: 14 Análise: 25/09/2007, CEL. Revisão: 26/09/2007, CEL.","[INFRINGENTES EMBARGOS EM PROFERIDO ACÓRDÃO DO PUBLICAÇÃO DA ANTES PROTOCOLIZADO EXTRAORDINÁRIO RECURSO O EXTEMPORÂNEO CONSIDERADO SER DE SENTIDO ""NO SÃO AGR RE 439515, O E AGR AI 395285]"


In [278]:
df_after_i[df_after_i["NEWcitacoesObs"].apply(lambda ids: "".join(set(re.sub("\s+\d+.*$", "", id) for id in ids))).str.contains("9")]

,acordaoId,_id,publicacao,citacoes_revistas,partesTexto,observacao,citacoesObs,index,observacao_lab,citacoesObs_before_i,citacoesObs_,NEWcitacoesObs
37527,AGR RE 238690,5e32b1cbd0a3cf19aa5bbb3b,DJ 21-03-2003 PP-00067 EMENT VOL-02103-02 PP-00342,[],AGTE.(S) : VAN LEER EMBALAGENS INDUSTRIAIS DO BRASIL LTDA \nADVDO.(A/S) : LUIZ PAULO ROMANO \nAGDO.(A/S) : UNIÃO \nADVDA. : PFN - ELYADIR FERREIRA BORGES,"Acórdãos citados: (TRF) argüição de inconstitucionalidade nº 83449. Número de páginas: (05). Análise:(VAS). Revisão:(ANA). Inclusão: 15/03/04, (MLR). Alteração: 16/03/04, (NT). Alteração: 21/09/2018, CLS.",[9 4 4 3 8 º N E D A D I L A N O I C U T I T S N O C N I E D O Ã Ç I Ü G R A ],6807,,[RE 140612],[9 4 4 3 8 º N ],[9 4 4 3 8 º N ]


In [199]:
# Decisões ainda não detectadas
df_after_i[df_after_i["observacao"].str.contains("[Cc]it") &
         (df_after_i["NEWcitacoesObs"].str.len() == 0)][["acordaoId", "observacao"]].head(40)

,acordaoId,observacao
198,ADI 1454,"Número de páginas: 41 -Acórdão citado: ADI 1155. Análise: 16/08/2007, JBM."
498,ADI 2503,"Acpordãos citados: ADI 766, ADI 2061, ADI 2115. Número de páginas: (7). Análise:(COF). Revisão:(AAF). Inclusão: 30/04/02, (SVF). Alteração: 03/05/02, (SVF). Alteração: 23/04/2018, CLS."
693,ADI 2939,"Ácórdãos citados: ADI 1690 MC (RTJ 170/75), ADI 1854 (RTJ 177/697). Número de páginas: (09). Análise:(JOY). Revisão:(RCO). Inclusão: 31/05/04, (SVF). Alteração: 01/06/04, (NT). Alteração: 18/02/2019, LRC."
695,ADI 2944,"- Acõrdçãos citados: ADI 13 - Tribunal Pleno, ADI 104 - Tribunal Pleno, Rp 164 - Tribunal Pleno, ADI 186 - Tribunal Pleno, Pleno, Rp 195 - Tribunal Pleno, ADI 368 - Tribunal Pleno, ADI 483 MC - Tribunal Pleno, ADI 507 - Tribunal Pleno, ADI 665 - Tribunal Pleno, ADI 850 MC - Tribunal Pleno, ADI 951 - Tribunal Pleno, ADI 1050 MC - Tribunal Pleno, ADI 1230 - Tribunal Pleno, ADI 1594 - Tribunal Pleno, ADI 1611 MC - Tribunal Pleno, ADI 1854 - Tribunal Pleno, ADI 2079 - Tribunal Pleno, ADI 2145 MC - Tribunal Pleno, ADI 2170 - Tribunal Pleno, ADI 2364 MC, ADI 2433 MC - Tribunal Pleno, ADI 2569 - Tribunal Pleno, ADI 3114 - Tribunal Pleno. Número de páginas: 26. Análise: 29/09/2011, KBP. Revisão: 07/10/2011, MMR."
829,ADI 3277,"-Aórdãos citados: ADI 2847 (RTJ 192/575), ADI 2948 (RTJ 194/160), ADI 2995, ADI 2996, ADI 3063, ADI 3147, ADI 3148, ADI 3189, ADI 3259, ADI 3293. Número de páginas: 20 Análise: 06/06/2007, JBM."
1201,ADI 80,"Número de páginas: 57 -Acórdão citado: RE 106007. Análise: 14/08/2007, JBM."
1293,AGR AC 196,"Decisões monocráticas citadas: Pet 2140, Pet 2592, Pet 2713, Pet 2800. Número de páginas: (07). Análise:(CEL). Revisão:(ANA). Inclusão: 22/06/04, (MLR). Alteração: 11/06/2019, MTH."
1297,AGR AC 2006,"- Decisão monocrática citada: RE 573098. Número de páginas: 9. Análise: 18/02/2009, RHP."
1311,AGR AC 2211,"- Acórdãos citados: RTJ 116/428, RTJ 127/4, RTJ 140/756, RTJ 172/419, RTJ 176/653, RTJ 191/123. - Decisões monocráticas citadas: Pet 914, Pet 965, Pet 1841, Pet 1865. Número de páginas: 10. Análise: 17/07/2009, CLM. Revisão: 31/07/2009, JBM."
1339,AGR AC 2751,"- Decisões monocráticas citadas: Rcl 2179 MC, Pet 2592 MC. Número de páginas: 6. Análise: 15/03/2011, SEV. Revisão: 24/03/2011, ACG."


In [57]:
for index, row in df_after_i.iterrows():
    db["acordaos"].update_one({"acordaoId": row["acordaoId"]}, {"$set": {"citacoesObs": row["NEWcitacoesObs"]}})

# db["acordaos"].update_one({"acordaoId": "RE 305186"}, {"$set": {"citacoesObs": df_after_i[df_after_i["acordaoId"] == "RE 305186"]["NEWcitacoesObs"].values[0]}})
# df_after_i[df_after_i["acordaoId"] == "RE 305186"]["NEWcitacoesObs"].values[0]

In [79]:
db.collection_names()

['stf_pr_1_acordaos_10_rel_8_removed_7',
 'stf_pr_1_acordaos_10_1',
 'stf_pr_2_acordaos_10_rel_8_7',
 'stf_pr_2_acordaos_10_rel_10_9',
 'stf_pr_1_acordaos_10_no_similars_10',
 'stf_kleinberg_acordaos_10_rel_10_8',
 'stf_kleinberg_acordaos_10_rel_9_removed_9',
 'stf_pr_2_acordaos_30_n_rem_invalid_5',
 'stf_pr_2_acordaos_10_rel_3_n_rem_invalid_3',
 'stf_pr_2_acordaos_10_rel_3_n_rem_invalid_removed_7',
 'stf_pr_2_acordaos_10_rel_3_n_rem_invalid_removed_10',
 'stf_pr_1_acordaos_10_rel_7_n_rem_invalid_removed_5',
 'stf_pr_2_acordaos_10_rel_2_n_rem_invalid_2',
 'stf_pr_1_acordaos_10_rel_9_n_rem_invalid_removed_2',
 'stf_pr_2_acordaos_10_rel_9_removed_2',
 'stf_kleinberg_acordaos_10_rel_3_9',
 'stf_pr_1_acordaos_30_removed_3',
 'stf_kleinberg_acordaos_10_rel_6_removed_5',
 'stf_pr_1_acordaos_10_rel_10_n_rem_invalid_removed_2',
 'stf_pr_2_acordaos_10_rel_9_n_rem_invalid_4',
 'stf_pr_1_acordaos_10_no_similars_5',
 'stf_pr_2_acordaos_10_rel_9_removed_10',
 'stf_pr_1_acordaos_10_rel_3_8',
 'stf_p

In [135]:
collection_name_lst = db.collection_names()
for pr in [1, 2]:
    for perc in [10, 20, 30]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_loop_",
                len([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_loop_" in c) and ("removed" in c)])
             )

print()
for pr in [1, 2]:
    for perc in [10]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop",
                len([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop" in c) and ("removed" in c)])
             )

print("\n")
for pr in [1, 2]:
    for perc in [10, 20, 30]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_loop_",
                len([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_loop_" in c) and ("removed" not in c)])
             )

print()
for pr in [1, 2]:
    for perc in [10]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop",
                len([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop" in c) and ("removed" not in c)])
             )

print()
for pr in [1, 2]:
    for perc in [10]:
        for rel in range(1, 11):
            print(
                    f"stf_pr_{pr}_acordaos_{perc}_rel_{rel}_no_loop_",
                    len([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_rel_{rel}_no_loop_" in c) and ("removed" not in c)])
                 )


stf_pr_1_acordaos_10_no_loop_ 10
stf_pr_1_acordaos_20_no_loop_ 10
stf_pr_1_acordaos_30_no_loop_ 10
stf_pr_2_acordaos_10_no_loop_ 10
stf_pr_2_acordaos_20_no_loop_ 10
stf_pr_2_acordaos_30_no_loop_ 10

stf_pr_1_acordaos_10_no_similars_no_loop 10
stf_pr_2_acordaos_10_no_similars_no_loop 10


stf_pr_1_acordaos_10_no_loop_ 10
stf_pr_1_acordaos_20_no_loop_ 10
stf_pr_1_acordaos_30_no_loop_ 10
stf_pr_2_acordaos_10_no_loop_ 10
stf_pr_2_acordaos_20_no_loop_ 10
stf_pr_2_acordaos_30_no_loop_ 10

stf_pr_1_acordaos_10_no_similars_no_loop 10
stf_pr_2_acordaos_10_no_similars_no_loop 10

stf_pr_1_acordaos_10_rel_1_no_loop_ 10
stf_pr_1_acordaos_10_rel_2_no_loop_ 10
stf_pr_1_acordaos_10_rel_3_no_loop_ 10
stf_pr_1_acordaos_10_rel_4_no_loop_ 10
stf_pr_1_acordaos_10_rel_5_no_loop_ 10
stf_pr_1_acordaos_10_rel_6_no_loop_ 10
stf_pr_1_acordaos_10_rel_7_no_loop_ 10
stf_pr_1_acordaos_10_rel_8_no_loop_ 10
stf_pr_1_acordaos_10_rel_9_no_loop_ 10
stf_pr_1_acordaos_10_rel_10_no_loop_ 10
stf_pr_2_acordaos_10_rel_1_no_lo

In [101]:
experiments_run_lst = []
for pr in [2]:
    for perc in [10, 20, 30]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_loop_",
                experiments_run_lst.extend([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_loop_" in c) and ("removed" not in c)])
             )

print()
for pr in [2]:
    for perc in [10]:
        print(
                f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop",
                experiments_run_lst.extend([c for c in collection_name_lst if (f"stf_pr_{pr}_acordaos_{perc}_no_similars_no_loop" in c) and ("removed" not in c)])
             )
experiments_run_lst

stf_pr_2_acordaos_10_no_loop_ None
stf_pr_2_acordaos_20_no_loop_ None
stf_pr_2_acordaos_30_no_loop_ None

stf_pr_2_acordaos_10_no_similars_no_loop None


['stf_pr_2_acordaos_10_no_loop_3',
 'stf_pr_2_acordaos_10_no_loop_8',
 'stf_pr_2_acordaos_10_no_loop_10',
 'stf_pr_2_acordaos_10_no_loop_7',
 'stf_pr_2_acordaos_10_no_loop_4',
 'stf_pr_2_acordaos_10_no_loop_9',
 'stf_pr_2_acordaos_10_no_loop_5',
 'stf_pr_2_acordaos_10_no_loop_6',
 'stf_pr_2_acordaos_20_no_loop_3',
 'stf_pr_2_acordaos_20_no_loop_10',
 'stf_pr_2_acordaos_20_no_loop_1',
 'stf_pr_2_acordaos_20_no_loop_5',
 'stf_pr_2_acordaos_20_no_loop_9',
 'stf_pr_2_acordaos_20_no_loop_8',
 'stf_pr_2_acordaos_20_no_loop_4',
 'stf_pr_2_acordaos_20_no_loop_2',
 'stf_pr_2_acordaos_20_no_loop_6',
 'stf_pr_2_acordaos_20_no_loop_7',
 'stf_pr_2_acordaos_30_no_loop_7',
 'stf_pr_2_acordaos_30_no_loop_9',
 'stf_pr_2_acordaos_30_no_loop_5',
 'stf_pr_2_acordaos_30_no_loop_8',
 'stf_pr_2_acordaos_30_no_loop_10',
 'stf_pr_2_acordaos_30_no_loop_6',
 'stf_pr_2_acordaos_10_no_similars_no_loop_8',
 'stf_pr_2_acordaos_10_no_similars_no_loop_7',
 'stf_pr_2_acordaos_10_no_similars_no_loop_6',
 'stf_pr_2_acord

In [104]:
query = {}
collections_name = "acordaos"
compute_similars = "S"
page_rank_iters = []

# experimento page rank todas as decisões
percentages = [10, 20, 30]
page_ranks = [1]
for percentage in percentages:
    for page_rank in page_ranks:
        collection_out_iter_name = "stf_pr_{}_acordaos_{}_no_loop".format(
            page_rank, percentage
        )
        page_rank_iters.extend(
            [
                (
                    i,
                    query,
                    collections_name,
                    percentage,
                    compute_similars,
                    collection_out_iter_name,
                    page_rank,
                )
                for i in range(1, 11)
                if collection_out_iter_name + "_{}".format(i) not in experiments_run_lst
            ]
        )

compute_similars = "N"
percentage = 10
for page_rank in page_ranks:
    collection_out_iter_name = "stf_pr_{}_acordaos_{}_no_similars_no_loop".format(
        page_rank, percentage
    )
    page_rank_iters.extend(
        [
            (
                i,
                query,
                collections_name,
                percentage,
                compute_similars,
                collection_out_iter_name,
                page_rank,
            )
            for i in range(1, 11)
            if collection_out_iter_name + "_{}".format(i) not in experiments_run_lst
        ]
    )
page_rank_iters

[(1, {}, 'acordaos', 10, 'S', 'stf_pr_2_acordaos_10_no_loop', 2),
 (2, {}, 'acordaos', 10, 'S', 'stf_pr_2_acordaos_10_no_loop', 2),
 (1, {}, 'acordaos', 30, 'S', 'stf_pr_2_acordaos_30_no_loop', 2),
 (2, {}, 'acordaos', 30, 'S', 'stf_pr_2_acordaos_30_no_loop', 2),
 (3, {}, 'acordaos', 30, 'S', 'stf_pr_2_acordaos_30_no_loop', 2),
 (4, {}, 'acordaos', 30, 'S', 'stf_pr_2_acordaos_30_no_loop', 2),
 (1, {}, 'acordaos', 10, 'N', 'stf_pr_2_acordaos_10_no_similars_no_loop', 2),
 (2, {}, 'acordaos', 10, 'N', 'stf_pr_2_acordaos_10_no_similars_no_loop', 2)]

In [131]:
def get_top_10_relatores():
    MONGO_URI = os.getenv("MONGO_URI")
    MONGO_DATABASE = os.getenv("MONGO_DATABASE")
    client = MongoClient(MONGO_URI)
    db = client[MONGO_DATABASE]
    docs = db["acordaos"].aggregate(
        [
            {"$group": {"_id": "$relator", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$limit": 10},
        ]
    )
    relatores = [doc["_id"] for doc in docs]

    return relatores
